# Customer Churn Prediction in the Spanish Energy Retail Market

## 1. Data Sources

Core Customer Data
- `customer_attributes.csv`: Customer demographics and metadata
- `customer_contracts.csv`: Contract lifecycle details
- `churn_label.csv`: Target variable (churn indicator)

Pricing & Cost Data
- `price_history.csv`: Historical tariff pricing
- `costs_by_province_month.csv`: Regional energy cost data

Behavioral & Interaction Data
- `consumption_hourly_2024.csv` / `.parquet`: Hourly energy consumption
- `customer_interactions.json`: Customer service interaction summaries

### Data Loading & Optimization
We load all datasets. For the large `consumption` file, we verify if a Parquet version exists; if not, we create it to optimize future reads.

In [1]:
# ============================================================
# PATH SETUP  —  run this cell first; edit nothing below
# ============================================================
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)
pd.set_option('display.width', 160)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# ── Raw data directory (where CSVs / JSON live) ───────────────────────────
_NOTEBOOK_DIR = Path.cwd()

_RAW_FILES = [
    'churn_label.csv',
    'customer_attributes.csv',
    'customer_contracts.csv',
    'price_history.csv',
    'costs_by_province_month.csv',
    'customer_interactions.json',
]

_RAW_CANDIDATES = [
    _NOTEBOOK_DIR / 'data' / 'Data and dictionary',
    _NOTEBOOK_DIR / 'Data and dictionary',
    _NOTEBOOK_DIR / 'data' / 'raw',
    _NOTEBOOK_DIR / 'data',
    _NOTEBOOK_DIR,
]

DATA_DIR = None
for _c in _RAW_CANDIDATES:
    if all((_c / f).exists() for f in _RAW_FILES):
        DATA_DIR = _c
        break

if DATA_DIR is None:
    _detail = '\n  '.join(
        str(p) + '  missing=' + str([f for f in _RAW_FILES if not (p / f).exists()])
        for p in _RAW_CANDIDATES
    )
    raise FileNotFoundError(
        f'Cannot find all raw data files. Searched:\n  {_detail}'
    )

print(f'DATA_DIR  -> {DATA_DIR.resolve()}')
for _f in _RAW_FILES:
    print(f"  {'OK' if (DATA_DIR / _f).exists() else 'MISSING'}  {_f}")

# ── Artefacts directory (bronze / silver / gold parquet) ─────────────────
ARTIFACTS_DIR = _NOTEBOOK_DIR / 'data'
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)
print(f'\nARTIFACTS_DIR -> {ARTIFACTS_DIR.resolve()}')


DATA_DIR  -> /Users/crissafrancisco/Desktop/IE/Third Term/Capstone/data/raw
  OK  churn_label.csv
  OK  customer_attributes.csv
  OK  customer_contracts.csv
  OK  price_history.csv
  OK  costs_by_province_month.csv
  OK  customer_interactions.json

ARTIFACTS_DIR -> /Users/crissafrancisco/Desktop/IE/Third Term/Capstone/data


In [2]:
def load_or_convert_consumption(data_dir):
    parquet_path = str(Path(data_dir) / 'consumption_hourly_2024.parquet')
    csv_path_gz = str(Path(data_dir) / 'consumption_hourly_2024.csv.gz')
    csv_path = str(Path(data_dir) / 'consumption_hourly_2024.csv')
    
    if os.path.exists(parquet_path):
        print(f"Loading optimized Parquet file: {parquet_path}")
        return pd.read_parquet(parquet_path)
    
    print("Parquet file not found. checking for CSV...")
    source_path = csv_path_gz if os.path.exists(csv_path_gz) else (csv_path if os.path.exists(csv_path) else None)
    
    if source_path:
        print(f"Converting {source_path} to Parquet for performance...")
        df = pd.read_csv(source_path)
        df.to_parquet(parquet_path)
        print("Conversion complete.")
        return df
    else:
        raise FileNotFoundError("Consumption file not found!")

# Load Datasets
files = {
    'churn': 'churn_label.csv',
    'attributes': 'customer_attributes.csv',
    'contracts': 'customer_contracts.csv',
    'prices': 'price_history.csv',
    'costs': 'costs_by_province_month.csv',
    'interactions': 'customer_interactions.json'
}

dfs = {}
for key, filename in files.items():
    path = DATA_DIR / filename
    print(f"Loading {filename}...")
    if filename.endswith('.json'):
        dfs[key] = pd.read_json(path)
    else:
        dfs[key] = pd.read_csv(path)

# Load Consumption
dfs['consumption'] = load_or_convert_consumption(DATA_DIR)

Loading churn_label.csv...
Loading customer_attributes.csv...
Loading customer_contracts.csv...
Loading price_history.csv...
Loading costs_by_province_month.csv...
Loading customer_interactions.json...
Loading optimized Parquet file: /Users/crissafrancisco/Desktop/IE/Third Term/Capstone/data/raw/consumption_hourly_2024.parquet


In [3]:
# Dataframes creation
df_churn = dfs["churn"]
df_attributes = dfs["attributes"]
df_contracts = dfs["contracts"]
df_prices = dfs["prices"]
df_costs = dfs["costs"]
df_interactions = dfs["interactions"]
df_consumption = dfs["consumption"]

## 2. Data Architecture

### Bronze Layer
Raw ingested datasets without transformation.

### Silver Layer
Cleaned and standardized datasets:
- Date normalization
- Price imputation
- Removal of impossible segment combinations
- Renewal and tenure bucket creation
- Interaction intent classification
- Sentiment timestamp validation

### Gold Layer
Customer-level modeling table including:
- Aggregated consumption features
- Profitability metrics
- Lifecycle variables
- Sentiment indicators
- Target label

### 2.1 Bronze Layer
- Consolidated raw datasets into two structured backbones:  
  `bronze_customer` (1 row per customer) and `bronze_customer_month` (1 row per customer-month).
- Enforced strict grain integrity through primary key validation, duplicate checks, and validated merge types (one-to-one / many-to-one).
- Standardized and aggregated hourly consumption into monthly usage, corrected invalid negative values (set to 0), and encoded tariff tiers and calendar signals (weekend, holiday).
- Integrated monthly pricing and provincial cost data at consistent month-level granularity to enable downstream margin computation.
- Applied systematic data quality controls (missing key removal, date normalization, merge validation) to ensure structural consistency before Silver-layer feature engineering.


In [4]:
# Customer-level Bronze (1 row per customer_id)
# churn + attributes + contracts + interactions

KEY = "customer_id"

# 1) Basic checks
for name, df in {
    "df_churn": df_churn,
    "df_attributes": df_attributes,
    "df_contracts": df_contracts,
    "df_interactions": df_interactions,
}.items():
    if KEY not in df.columns:
        raise KeyError(f"{name} missing join key '{KEY}'. Columns: {list(df.columns)[:50]}")

# 2) Ensure churn is unique per customer (backbone)
if df_churn.duplicated(subset=[KEY]).any():
    raise ValueError("df_churn has duplicate customer_id values; backbone must be 1 row per customer.")

# 3) De-dupe other tables to 1 row/customer (keep most recent interaction if date exists)
df_attr_1 = df_attributes.drop_duplicates(subset=[KEY], keep="first").copy()
df_con_1  = df_contracts.drop_duplicates(subset=[KEY], keep="first").copy()

df_int_1 = df_interactions.copy()
# If you have a timestamp/date column for interactions, set it here if it differs
INTERACTION_DATE_COL = "interaction_date"  # change if needed, or set to None

if INTERACTION_DATE_COL in df_int_1.columns:
    df_int_1[INTERACTION_DATE_COL] = pd.to_datetime(df_int_1[INTERACTION_DATE_COL], errors="coerce")
    df_int_1 = (
        df_int_1.sort_values([KEY, INTERACTION_DATE_COL])
               .drop_duplicates(subset=[KEY], keep="last")
    )
else:
    df_int_1 = df_int_1.drop_duplicates(subset=[KEY], keep="first")

# 4) Merge (left-joins from churn)
bronze_customer = (
    df_churn.copy()
    .merge(df_attr_1, on=KEY, how="left", validate="one_to_one")
    .merge(df_con_1,  on=KEY, how="left", validate="one_to_one")
    .merge(df_int_1,  on=KEY, how="left", validate="one_to_one")
)

# 5) Final grain check
if bronze_customer.duplicated(subset=[KEY]).any():
    raise ValueError("bronze_customer has duplicate customer_id rows; merge created row multiplication.")

print("bronze_customer shape:", bronze_customer.shape)
print("unique customers:", bronze_customer[KEY].nunique())


bronze_customer shape: (20099, 17)
unique customers: 20099


##### Consumption Processing (Bronze - Customer-Month)

- Converted hourly consumption to monthly aggregates to align with billing and pricing granularity.
- Identified and corrected invalid negative consumption values (set to 0 to maintain industry-consistent treatment).
- Merged monthly pricing data to attach tariff information at customer-month level.
- Added province from `bronze_customer` to enable geographic cost attribution.
- Integrated provincial cost data using (province, month) keys to prepare for margin computation.


In [5]:
# check negative values in df_consumption
neg_elec = df_consumption[df_consumption["consumption_elec_kwh"] < 0]
neg_gas = df_consumption[df_consumption["consumption_gas_m3"] < 0]

print("Elec negatives %:", len(neg_elec) / len(df_consumption))
print("Gas negatives %:", len(neg_gas) / len(df_consumption))

print("Elec total negative kWh:", neg_elec["consumption_elec_kwh"].sum())
print("Gas total negative m3:", neg_gas["consumption_gas_m3"].sum())


Elec negatives %: 0.002592155170702835
Gas negatives %: 0.00011073940823524645
Elec total negative kWh: -412328.5811701021
Gas total negative m3: -286.9179469833537


In [6]:
neg_elec["consumption_elec_kwh"].describe()
neg_gas["consumption_gas_m3"].describe()


count   19551.0000
mean       -0.0147
std         0.0220
min        -0.2300
25%        -0.0169
50%        -0.0061
75%        -0.0020
max        -0.0000
Name: consumption_gas_m3, dtype: float64

In [7]:
# Impute negative consumption. Negative consumption is not “missing data.” It is invalid data.
# --- Fix invalid negative consumption in df_consumption (hourly) ---
# Columns confirmed in your df:
ELEC_COL = "consumption_elec_kwh"
GAS_COL_M3 = "consumption_gas_m3"

cons_clean = df_consumption.copy()

# Flag rows that had any negative consumption (useful for QA / later features)
cons_clean["had_negative_consumption"] = (
    (cons_clean[ELEC_COL] < 0) | (cons_clean[GAS_COL_M3] < 0)
).astype(int)

# Optional: keep original values for audit (comment out if you don’t want extra cols)
cons_clean[f"{ELEC_COL}__orig"] = cons_clean[ELEC_COL]
cons_clean[f"{GAS_COL_M3}__orig"] = cons_clean[GAS_COL_M3]

# Replace negatives with 0 (industry-standard for invalid delivered energy)
cons_clean[ELEC_COL] = np.where(cons_clean[ELEC_COL] < 0, 0.0, cons_clean[ELEC_COL])
cons_clean[GAS_COL_M3] = np.where(cons_clean[GAS_COL_M3] < 0, 0.0, cons_clean[GAS_COL_M3])

# Quick sanity checks
print("Neg elec after fix:", int((cons_clean[ELEC_COL] < 0).sum()))
print("Neg gas after fix:", int((cons_clean[GAS_COL_M3] < 0).sum()))
print("Rows flagged had_negative_consumption:", int(cons_clean["had_negative_consumption"].sum()))

Neg elec after fix: 0
Neg gas after fix: 0
Rows flagged had_negative_consumption: 477175


In [8]:
KEY = "customer_id"
TS_COL = "timestamp"
ELEC_COL = "consumption_elec_kwh"
GAS_COL_M3 = "consumption_gas_m3"

# Use the cleaned hourly df (cons_clean) if you created it; otherwise df_consumption
base = cons_clean if "cons_clean" in globals() else df_consumption

# Keep only required cols (BIG memory saver)
cons = base[[KEY, TS_COL, ELEC_COL, GAS_COL_M3]].copy()

# Parse timestamp once
cons[TS_COL] = pd.to_datetime(cons[TS_COL], errors="coerce")

# Calendar fields (use small dtypes where possible)
cons["month"] = cons[TS_COL].dt.to_period("M").astype(str)   # YYYY-MM
weekday = cons[TS_COL].dt.weekday.astype("int8")
hour = cons[TS_COL].dt.hour.astype("int8")

cons["is_weekend"] = weekday.isin([5, 6])

In [9]:
# Holiday encoding
SPANISH_PUBLIC_HOLIDAYS_MD = {101, 106, 501, 815, 1012, 1101, 1206, 1208, 1225}

# Use numpy arrays where possible to avoid pandas overhead/copies
ts = cons[TS_COL]

md_int = (ts.dt.month.to_numpy(dtype="int16") * 100 + ts.dt.day.to_numpy(dtype="int16")).astype("int16", copy=False)
cons["is_holiday"] = np.isin(md_int, list(SPANISH_PUBLIC_HOLIDAYS_MD))

# Tier assignment (minimize intermediate allocations)
is_weekday_nonholiday = (~cons["is_weekend"].to_numpy()) & (~cons["is_holiday"].to_numpy())

h = hour.to_numpy(copy=False)  # hour is already int8 from your previous cell

peak = is_weekday_nonholiday & (((h >= 10) & (h < 14)) | ((h >= 18) & (h < 22)))
standard = is_weekday_nonholiday & (((h >= 8) & (h < 10)) | ((h >= 14) & (h < 18)) | ((h >= 22) & (h < 24)))

# Build tier with a single output array (less memory than np.select + fewer temps)
tier = np.full(len(cons), "tier_3_offpeak", dtype=object)
tier[standard] = "tier_2_standard"
tier[peak] = "tier_1_peak"

cons["tier"] = pd.Series(tier, index=cons.index).astype("category")

# (optional) free large temp arrays sooner
del tier


In [10]:
# Gas converted to kWh (using ~11 kWh per m3)
cons["consumption_gas_kwh"] = cons[GAS_COL_M3] * 11.0

# Monthly totals (elec + gas)
monthly_total = (
    cons.groupby([KEY, "month"], as_index=False)
        .agg(
            monthly_elec_kwh=(ELEC_COL, "sum"),
            monthly_gas_m3=(GAS_COL_M3, "sum"),
            monthly_gas_kwh=("consumption_gas_kwh", "sum"),
        )
)

In [11]:
# Monthly by tier for each measure
def tier_pivot(value_col: str, prefix: str) -> pd.DataFrame:
    p = (
        cons.pivot_table(
            index=[KEY, "month"],
            columns="tier",
            values=value_col,
            aggfunc="sum",
            fill_value=0
        )
        .reset_index()
    )
    # ensure columns exist
    for c in ["tier_1_peak", "tier_2_standard", "tier_3_offpeak"]:
        if c not in p.columns:
            p[c] = 0.0
    return p.rename(columns={
        "tier_1_peak": f"{prefix}_tier_1_peak",
        "tier_2_standard": f"{prefix}_tier_2_standard",
        "tier_3_offpeak": f"{prefix}_tier_3_offpeak",
    })

In [12]:
# 1) Fast tier splits (NO pivot_table / pivot_table-like call)
cons["elec_kwh_t1"] = np.where(cons["tier"] == "tier_1_peak",     cons[ELEC_COL], 0.0)
cons["elec_kwh_t2"] = np.where(cons["tier"] == "tier_2_standard", cons[ELEC_COL], 0.0)
cons["elec_kwh_t3"] = np.where(cons["tier"] == "tier_3_offpeak",  cons[ELEC_COL], 0.0)

cons["gas_m3_t1"]   = np.where(cons["tier"] == "tier_1_peak",     cons[GAS_COL_M3], 0.0)
cons["gas_m3_t2"]   = np.where(cons["tier"] == "tier_2_standard", cons[GAS_COL_M3], 0.0)
cons["gas_m3_t3"]   = np.where(cons["tier"] == "tier_3_offpeak",  cons[GAS_COL_M3], 0.0)

cons["gas_kwh_t1"]  = np.where(cons["tier"] == "tier_1_peak",     cons["consumption_gas_kwh"], 0.0)
cons["gas_kwh_t2"]  = np.where(cons["tier"] == "tier_2_standard", cons["consumption_gas_kwh"], 0.0)
cons["gas_kwh_t3"]  = np.where(cons["tier"] == "tier_3_offpeak",  cons["consumption_gas_kwh"], 0.0)

In [13]:
# 2) One aggregation: totals + tiers + holiday hours
monthly_consumption = (
    cons.groupby([KEY, "month"], as_index=False)
        .agg(
            monthly_elec_kwh=(ELEC_COL, "sum"),
            monthly_gas_m3=(GAS_COL_M3, "sum"),
            monthly_gas_kwh=("consumption_gas_kwh", "sum"),

            elec_kwh_tier_1_peak=("elec_kwh_t1", "sum"),
            elec_kwh_tier_2_standard=("elec_kwh_t2", "sum"),
            elec_kwh_tier_3_offpeak=("elec_kwh_t3", "sum"),

            gas_m3_tier_1_peak=("gas_m3_t1", "sum"),
            gas_m3_tier_2_standard=("gas_m3_t2", "sum"),
            gas_m3_tier_3_offpeak=("gas_m3_t3", "sum"),

            gas_kwh_tier_1_peak=("gas_kwh_t1", "sum"),
            gas_kwh_tier_2_standard=("gas_kwh_t2", "sum"),
            gas_kwh_tier_3_offpeak=("gas_kwh_t3", "sum"),

            holiday_hours_in_month=("is_holiday", "sum"),
        )
)

In [14]:
# 3) Drop missing customer ids (confirmed as invalid by BCG)
monthly_consumption = monthly_consumption.dropna(subset=[KEY])
monthly_consumption = monthly_consumption[monthly_consumption[KEY] != ""]

# 4) Sanity check
if monthly_consumption.duplicated(subset=[KEY, "month"]).any():
    raise ValueError("monthly_consumption has duplicate (customer_id, month) rows; grain broke.")

print("monthly_consumption shape:", monthly_consumption.shape)

monthly_consumption shape: (241188, 15)


In [15]:
KEY = "customer_id"
PRICE_DATE_COL = "pricing_date"

prices = df_prices.copy()

# Drop missing customer_id rows
print("Missing customer_id before:", prices[KEY].isna().sum())

prices = prices.dropna(subset=[KEY])
prices = prices[prices[KEY] != ""]

print("Missing customer_id after:", prices[KEY].isna().sum())

# Continue normal preparation
prices[PRICE_DATE_COL] = pd.to_datetime(prices[PRICE_DATE_COL], errors="coerce")
prices["month"] = prices[PRICE_DATE_COL].dt.to_period("M").astype(str)

# Sanity check
print("Duplicate customer-month rows:",
      prices.duplicated(subset=[KEY, "month"]).sum())


Missing customer_id before: 252
Missing customer_id after: 0
Duplicate customer-month rows: 0


In [16]:
# Merge cleaned monthly prices into monthly_consumption
# Assumes you already created `prices["month"]` and cleaned missing customer_id rows as shown.

# Drop pricing_date to avoid duplicate date columns
prices_merge = prices.drop(columns=[PRICE_DATE_COL], errors="ignore")

monthly_customer_month = (
    monthly_consumption
        .merge(prices_merge, on=[KEY, "month"], how="left", validate="one_to_one")
)

print("monthly_customer_month shape:", monthly_customer_month.shape)

monthly_customer_month shape: (241188, 23)


In [17]:
# Bring province_code from bronze_customer into monthly_customer_month

KEY = "customer_id"

# Select only the needed columns to avoid duplicating everything
province_lookup = bronze_customer[[KEY, "province_code"]].copy()

# Ensure one row per customer
province_lookup = province_lookup.drop_duplicates(subset=[KEY])

# Merge
monthly_customer_month = (
    monthly_customer_month
        .merge(province_lookup, on=KEY, how="left", validate="many_to_one")
)

print("Shape after adding province_code:", monthly_customer_month.shape)
print("Missing province_code:", monthly_customer_month["province_code"].isna().sum())


Shape after adding province_code: (241188, 24)
Missing province_code: 0


In [18]:
# Merge costs using dictionary keys: month + Province
# df_costs columns: ["month", "Province", ...cost columns...]

# 1) Ensure types/format are consistent
df_costs["month"] = df_costs["month"].astype(str).str[:7]  # YYYY-MM safety
monthly_customer_month["month"] = monthly_customer_month["month"].astype(str).str[:7]

# Province code from bronze_customer is in province_code
# Costs uses Province, so merge left_on province_code -> right_on Province
monthly_customer_month = (
    monthly_customer_month.merge(
        df_costs,
        left_on=["province_code", "month"],
        right_on=["province", "month"],
        how="left",
        validate="many_to_one"
    )
)

# Drop the duplicate Province column after merge
monthly_customer_month = monthly_customer_month.drop(columns=["province"], errors="ignore")

print("Shape after costs merge:", monthly_customer_month.shape)
print("Missing elec_var_cost_eur_kwh:", monthly_customer_month["elec_var_cost_eur_kwh"].isna().sum())



Shape after costs merge: (241188, 27)
Missing elec_var_cost_eur_kwh: 0


In [19]:
# Columns in Bronze Customer (customer-level)
print("\nBronze Customer Columns:")
print(bronze_customer.columns.tolist())
print("Total columns:", len(bronze_customer.columns))


# Columns in Monthly Customer-Month (customer-month level)
print("\nMonthly Customer-Month Columns:")
print(monthly_customer_month.columns.tolist())
print("Total columns:", len(monthly_customer_month.columns))



Bronze Customer Columns:
['customer_id', 'churn', 'contracted_power_kw', 'is_industrial', 'is_second_residence', 'province_code', 'region', 'is_high_competition_province', 'is_urban', 'sales_channel', 'customer_first_activation_date', 'contract_start_date', 'last_product_change_date', 'next_renewal_date', 'date', 'channel', 'interaction_summary']
Total columns: 17

Monthly Customer-Month Columns:
['customer_id', 'month', 'monthly_elec_kwh', 'monthly_gas_m3', 'monthly_gas_kwh', 'elec_kwh_tier_1_peak', 'elec_kwh_tier_2_standard', 'elec_kwh_tier_3_offpeak', 'gas_m3_tier_1_peak', 'gas_m3_tier_2_standard', 'gas_m3_tier_3_offpeak', 'gas_kwh_tier_1_peak', 'gas_kwh_tier_2_standard', 'gas_kwh_tier_3_offpeak', 'holiday_hours_in_month', 'variable_price_tier1_eur_kwh', 'variable_price_tier2_eur_kwh', 'variable_price_tier3_eur_kwh', 'elec_fixed_fee_eur_month', 'elec_fixed_cost_eur_month', 'gas_variable_price_eur_m3', 'gas_fixed_revenue_eur_year', 'gas_fixed_cost_eur_year', 'province_code', 'elec_v

In [20]:
# Drop province_code from monthly_customer_month
monthly_customer_month = monthly_customer_month.drop(columns=["province_code"], errors="ignore")

print("Columns after cleanup:")
print(monthly_customer_month.columns.tolist())
print("Total columns:", len(monthly_customer_month.columns))


Columns after cleanup:
['customer_id', 'month', 'monthly_elec_kwh', 'monthly_gas_m3', 'monthly_gas_kwh', 'elec_kwh_tier_1_peak', 'elec_kwh_tier_2_standard', 'elec_kwh_tier_3_offpeak', 'gas_m3_tier_1_peak', 'gas_m3_tier_2_standard', 'gas_m3_tier_3_offpeak', 'gas_kwh_tier_1_peak', 'gas_kwh_tier_2_standard', 'gas_kwh_tier_3_offpeak', 'holiday_hours_in_month', 'variable_price_tier1_eur_kwh', 'variable_price_tier2_eur_kwh', 'variable_price_tier3_eur_kwh', 'elec_fixed_fee_eur_month', 'elec_fixed_cost_eur_month', 'gas_variable_price_eur_m3', 'gas_fixed_revenue_eur_year', 'gas_fixed_cost_eur_year', 'elec_var_cost_eur_kwh', 'gas_var_cost_eur_m3', 'peaje_elec_eur_kwh']
Total columns: 26


In [21]:
bronze_customer_month = monthly_customer_month.copy()

# Delete old variable to avoid confusion
del monthly_customer_month


In [22]:
(ARTIFACTS_DIR / 'bronze').mkdir(parents=True, exist_ok=True)

bronze_customer.to_parquet(
    ARTIFACTS_DIR / 'bronze' / 'bronze_customer.parquet', index=False
)
bronze_customer_month.to_parquet(
    ARTIFACTS_DIR / 'bronze' / 'bronze_customer_month.parquet', index=False
)
print('Saved to', ARTIFACTS_DIR / 'bronze')


Saved to /Users/crissafrancisco/Desktop/IE/Third Term/Capstone/data/bronze


### 2.2 Silver Layer – Consumption (Usage, Prices, Costs)

- Performed structured imputations on pricing variables (forward-fill within customer, controlled backfill when needed) to ensure tariff continuity without data leakage.
- Validated and aligned consumption, pricing, and cost data at consistent monthly granularity before margin computation.
- Computed monthly revenue, cost, and margin metrics at customer-month level to establish profitability signals.
- Conducted EDA on usage patterns, tariff distribution, and margin behavior to detect anomalies and validate business logic.
- Engineered core consumption-based features (monthly totals, tier splits, margin trends) to prepare for Gold-layer aggregation and modeling.


In [23]:
# Read bronze
bronze_customer = pd.read_parquet(
    ARTIFACTS_DIR / 'bronze' / 'bronze_customer.parquet'
)
bronze_customer_month = pd.read_parquet(
    ARTIFACTS_DIR / 'bronze' / 'bronze_customer_month.parquet'
)


In [24]:
# create silver equivalents

silver_customer = bronze_customer.copy()
silver_customer_month = bronze_customer_month.copy()


#### 2.2.1 Preprocessing & Imputation

##### Missing Prices – Imputation Strategy
Missing tariff prices were imputed using a hierarchical, industry-consistent approach. 
1. First, prices were forward-filled and backward-filled at the customer level, reflecting the industry standard that tariffs remain valid until formally updated. 
2. If customer-level history was unavailable, prices were imputed using the segment-month median (based on industrial vs. residential classification), ensuring alignment with typical tariff structures within similar customer groups. 
3. As a final fallback, the national monthly median was applied to preserve temporal market pricing trends while avoiding distortion from extreme values.



In [25]:
# sort month per customer
silver_customer_month = silver_customer_month.sort_values(["customer_id", "month"])


In [26]:
# impute those with missing or zero prices (but only for those with consumption)

KEY = "customer_id"
MONTH_COL = "month"
SEG_COL = "is_industrial"   # comes from bronze_customer

# Work on a copy, then assign back
scm = silver_customer_month.copy()

# 0) Ensure segment column exists in silver_customer_month (bring from silver_customer if needed)
if SEG_COL not in scm.columns:
    seg_lookup = silver_customer[[KEY, SEG_COL]].drop_duplicates(subset=[KEY])
    scm = scm.merge(seg_lookup, on=KEY, how="left", validate="many_to_one")

# 1) Month sortable
scm[MONTH_COL] = pd.to_datetime(scm[MONTH_COL] + "-01", errors="coerce")
scm = scm.sort_values([KEY, MONTH_COL])

# 2) Define columns
elec_t1 = "variable_price_tier1_eur_kwh"
elec_t2 = "variable_price_tier2_eur_kwh"
elec_t3 = "variable_price_tier3_eur_kwh"
gas_var = "gas_variable_price_eur_m3"
elec_fix = "elec_fixed_fee_eur_month"
gas_fix = "gas_fixed_revenue_eur_year"

price_cols = [elec_t1, elec_t2, elec_t3, gas_var, elec_fix, gas_fix]

elec_kwh_t1 = "elec_kwh_tier_1_peak"
elec_kwh_t2 = "elec_kwh_tier_2_standard"
elec_kwh_t3 = "elec_kwh_tier_3_offpeak"

elec_total = "monthly_elec_kwh"
gas_total_m3 = "monthly_gas_m3"

# 3) Treat zero prices as invalid -> NaN (only for price columns)
for c in price_cols:
    scm[c] = pd.to_numeric(scm[c], errors="coerce")
    scm.loc[scm[c] == 0, c] = np.nan

# 4) Masks: impute ONLY where the relevant consumption exists
mask_elec_t1 = scm[elec_kwh_t1].fillna(0) > 0
mask_elec_t2 = scm[elec_kwh_t2].fillna(0) > 0
mask_elec_t3 = scm[elec_kwh_t3].fillna(0) > 0
mask_gas_var = scm[gas_total_m3].fillna(0) > 0
mask_fixed = (scm[elec_total].fillna(0) > 0) | (scm[gas_total_m3].fillna(0) > 0)

def impute_price_segmented(col: str, mask: pd.Series) -> None:
    # (1) customer ffill/bfill candidates
    filled_cust = (
        scm.groupby(KEY, sort=False)[col]
           .apply(lambda s: s.ffill().bfill())
           .reset_index(level=0, drop=True)
    )

    need = mask & scm[col].isna()
    if need.any():
        scm.loc[need, col] = filled_cust.loc[need]

    # (2) segment-month median fallback
    need2 = mask & scm[col].isna()
    if need2.any():
        seg_month_median = (
            scm.loc[mask, [SEG_COL, MONTH_COL, col]]
               .groupby([SEG_COL, MONTH_COL], sort=False)[col]
               .median()
        )
        # map using (seg, month)
        idx = list(zip(scm.loc[need2, SEG_COL], scm.loc[need2, MONTH_COL]))
        scm.loc[need2, col] = [seg_month_median.get(k, np.nan) for k in idx]

    # (3) national month median fallback
    need3 = mask & scm[col].isna()
    if need3.any():
        nat_month_median = (
            scm.loc[mask, [MONTH_COL, col]]
               .groupby(MONTH_COL, sort=False)[col]
               .median()
        )
        scm.loc[need3, col] = scm.loc[need3, MONTH_COL].map(nat_month_median)


In [27]:
# 5) Apply imputation hierarchy
impute_price_segmented(elec_t1, mask_elec_t1)
impute_price_segmented(elec_t2, mask_elec_t2)
impute_price_segmented(elec_t3, mask_elec_t3)
impute_price_segmented(gas_var, mask_gas_var)
impute_price_segmented(elec_fix, mask_fixed)
impute_price_segmented(gas_fix, mask_fixed)

# 6) Return month to YYYY-MM string for consistency
scm[MONTH_COL] = scm[MONTH_COL].dt.to_period("M").astype(str)

# 7) Assign back to silver_customer_month
silver_customer_month = scm

# 8) Sanity: remaining missing where consumption exists
def remaining(col, msk):
    return int(((silver_customer_month[col].isna()) & msk).sum())

print("Remaining missing (only where consumption exists):")
print("elec tier1:", remaining(elec_t1, silver_customer_month[elec_kwh_t1].fillna(0) > 0))
print("elec tier2:", remaining(elec_t2, silver_customer_month[elec_kwh_t2].fillna(0) > 0))
print("elec tier3:", remaining(elec_t3, silver_customer_month[elec_kwh_t3].fillna(0) > 0))
print("gas var   :", remaining(gas_var, silver_customer_month[gas_total_m3].fillna(0) > 0))
print("elec fixed:", remaining(elec_fix, (silver_customer_month[elec_total].fillna(0) > 0) | (silver_customer_month[gas_total_m3].fillna(0) > 0)))
print("gas fixed :", remaining(gas_fix, (silver_customer_month[elec_total].fillna(0) > 0) | (silver_customer_month[gas_total_m3].fillna(0) > 0)))


Remaining missing (only where consumption exists):
elec tier1: 0
elec tier2: 0
elec tier3: 0
gas var   : 0
elec fixed: 0
gas fixed : 0


##### Customer Segmentation

- Customers were first classified as **Residential** (`is_industrial = 0`) or **Industrial** (`is_industrial = 1`) based on contract type.

- Industrial customers were further segmented into **SME** and **Corporate** using contracted power thresholds:
  - `contracted_power_kw ≤ 10` → SME  
  - `contracted_power_kw > 10` → Corporate  
  (Aligned with tariff clustering and empirical distribution patterns.)

- Residential customers were sub-segmented into:
  - **Primary Residence** (`is_second_residence = 0`)
  - **Secondary Residence** (`is_second_residence = 1`)
  - (Note: This classification is contract-level per supply point, not a full representation of a customer’s entire property portfolio.)

- Segmentation logic reflects industry-standard differentiation in tariff structure, consumption behavior, and churn dynamics.


In [28]:
# Work on silver_customer (do NOT touch bronze_customer)

sc = silver_customer.copy()

# --- Main Segment ---
sc["segment"] = None

# Residential
sc.loc[sc["is_industrial"] == 0, "segment"] = "Residential"


In [29]:
# Check data distribution to determine the threshold (75% below 10)

sc[sc["is_industrial"] == 1]["contracted_power_kw"].describe()


count   2009.0000
mean      14.0916
std       17.8775
min       10.0000
25%       10.0000
50%       10.0000
75%       10.0000
max      150.0000
Name: contracted_power_kw, dtype: float64

In [30]:
# Check counts per contracted power

sc[sc["is_industrial"]==1]["contracted_power_kw"].value_counts().sort_index()


contracted_power_kw
10.0000     1832
15.0000       28
20.0000       10
25.0000        9
30.0000       25
35.0000       12
40.0000        6
45.0000        8
50.0000        8
55.0000        5
60.0000        4
65.0000        5
70.0000        4
75.0000        5
80.0000        8
85.0000        4
90.0000        2
95.0000        4
100.0000       3
105.0000       2
110.0000       2
115.0000       1
120.0000       3
130.0000       5
140.0000       1
145.0000       1
150.0000      12
Name: count, dtype: int64

In [31]:
# Industrial split
sc.loc[
    (sc["is_industrial"] == 1) & (sc["contracted_power_kw"] == 10),
    "segment"
] = "SME"

sc.loc[
    (sc["is_industrial"] == 1) & (sc["contracted_power_kw"] > 10),
    "segment"
] = "Corporate"

In [32]:
# --- Residential sub-type ---
sc["residential_type"] = None

sc.loc[
    (sc["segment"] == "Residential") & (sc["is_second_residence"] == 1),
    "residential_type"
] = "Second_Residence"

sc.loc[
    (sc["segment"] == "Residential") & (sc["is_second_residence"] == 0),
    "residential_type"
] = "Primary_Residence"

# Assign back
silver_customer = sc

# --- Validation ---
print("Segment counts:")
print(silver_customer["segment"].value_counts())

print("\nResidential breakdown:")
print(silver_customer["residential_type"].value_counts())


Segment counts:
segment
Residential    18090
SME             1832
Corporate        177
Name: count, dtype: int64

Residential breakdown:
residential_type
Primary_Residence    17688
Second_Residence       402
Name: count, dtype: int64


In [33]:
# Check contracted power of primary vs secondary residence
res = sc[sc["is_industrial"] == 0]
res.groupby("is_second_residence")["contracted_power_kw"].describe()


,count,mean,std,min,25%,50%,75%,max
is_second_residence,,,,,,,,
0,17688.0000,5.1419,1.9834,2.3000,3.4500,4.6000,6.9000,10.3500
1,402.0000,4.0736,1.9114,2.3000,2.3000,3.4500,5.7500,10.3500


In [34]:
res.groupby("is_second_residence")["contracted_power_kw"].agg(
    customers="count",
    mean_power="mean",
    median_power="median",
    std_power="std"
)


,customers,mean_power,median_power,std_power
is_second_residence,,,,
0,17688,5.1419,4.6000,1.9834
1,402,4.0736,3.4500,1.9114


##### Channel conversion (from Spanish to English)

In [35]:
sc = silver_customer.copy()

# Normalize to lowercase
sc["sales_channel_clean"] = (
    sc["sales_channel"]
        .astype(str)
        .str.strip()
        .str.lower()
)

channel_map = {
    "presencial_comercial": "In-Person Commercial",
    "comparador": "Comparison Website",
    "oficina": "Office",
    "telemarketing": "Telemarketing",
    "web_propia": "Own Website",
    "desconocido": "Unknown",
    "unknown": "Unknown",
}

sc["sales_channel_en"] = sc["sales_channel_clean"].map(channel_map)

# Anything unmapped → keep original cleaned version
sc["sales_channel_en"] = sc["sales_channel_en"].fillna(
    sc["sales_channel_clean"]
)

sc = sc.drop(columns=["sales_channel_clean"])

silver_customer = sc

print(silver_customer["sales_channel_en"].value_counts())


sales_channel_en
In-Person Commercial    9158
Unknown                 5267
Comparison Website      2597
Office                  1836
Telemarketing           1218
Own Website               23
Name: count, dtype: int64


##### Final Checks before EDA

In [36]:
# No impossible values

df = silver_customer_month  # rename for convenience

# ---- Column names (from your data dictionary / current silver_customer_month) ----
ELEC_T1_KWH = "elec_kwh_tier_1_peak"
ELEC_T2_KWH = "elec_kwh_tier_2_standard"
ELEC_T3_KWH = "elec_kwh_tier_3_offpeak"

GAS_T1_KWH  = "gas_kwh_tier_1_peak"
GAS_T2_KWH  = "gas_kwh_tier_2_standard"
GAS_T3_KWH  = "gas_kwh_tier_3_offpeak"

GAS_TOTAL_M3 = "monthly_gas_m3"  # use m3 for gas price per m3

P_ELEC_T1 = "variable_price_tier1_eur_kwh"
P_ELEC_T2 = "variable_price_tier2_eur_kwh"
P_ELEC_T3 = "variable_price_tier3_eur_kwh"
P_GAS_VAR = "gas_variable_price_eur_m3"

# ---- Checks: price missing/zero ONLY where corresponding consumption exists ----
checks = [
    ("elec_tier1", P_ELEC_T1, ELEC_T1_KWH),
    ("elec_tier2", P_ELEC_T2, ELEC_T2_KWH),
    ("elec_tier3", P_ELEC_T3, ELEC_T3_KWH),
    ("gas_var_m3", P_GAS_VAR, GAS_TOTAL_M3),
]

rows = []
for label, price_col, cons_col in checks:
    cons_used = df[cons_col].fillna(0) > 0

    missing_where_used = df[price_col].isna() & cons_used
    zero_where_used    = (df[price_col].fillna(0) == 0) & cons_used
    bad_where_used     = (df[price_col].isna() | (df[price_col] == 0)) & cons_used

    rows.append({
        "item": label,
        "consumption_col": cons_col,
        "price_col": price_col,
        "rows_with_consumption": int(cons_used.sum()),
        "price_missing_where_used": int(missing_where_used.sum()),
        "price_zero_where_used": int(zero_where_used.sum()),
        "price_missing_or_zero_where_used": int(bad_where_used.sum()),
    })

summary = pd.DataFrame(rows).sort_values("price_missing_or_zero_where_used", ascending=False)
display(summary)

# ---- Optional: quickly inspect the offending rows for any item ----
# Example: show the rows where elec tier1 is used but tier1 price is missing/zero
bad_elec_t1 = (df[P_ELEC_T1].isna() | (df[P_ELEC_T1] == 0)) & (df[ELEC_T1_KWH].fillna(0) > 0)
df.loc[bad_elec_t1, ["customer_id", "month", ELEC_T1_KWH, P_ELEC_T1]].head(20)


,item,consumption_col,price_col,rows_with_consumption,price_missing_where_used,price_zero_where_used,price_missing_or_zero_where_used
0,elec_tier1,elec_kwh_tier_1_peak,variable_price_tier1_eur_kwh,234866,0,0,0
1,elec_tier2,elec_kwh_tier_2_standard,variable_price_tier2_eur_kwh,234866,0,0,0
2,elec_tier3,elec_kwh_tier_3_offpeak,variable_price_tier3_eur_kwh,235601,0,0,0
3,gas_var_m3,monthly_gas_m3,gas_variable_price_eur_m3,32671,0,0,0


,customer_id,month,elec_kwh_tier_1_peak,variable_price_tier1_eur_kwh


In [37]:
# No impossible values
print(silver_customer_month[[
    "monthly_elec_kwh",
    "monthly_gas_m3",
    "monthly_gas_kwh"
]].min())


monthly_elec_kwh   0.0000
monthly_gas_m3     0.0000
monthly_gas_kwh    0.0000
dtype: float64


In [38]:
# Grain integrity
silver_customer.duplicated("customer_id").sum()

silver_customer_month.duplicated(["customer_id","month"]).sum()



np.int64(0)

In [39]:
# Segment sanity

silver_customer.groupby("segment")["contracted_power_kw"].describe()


,count,mean,std,min,25%,50%,75%,max
segment,,,,,,,,
Corporate,177.0000,56.4407,40.8474,15.0000,25.0000,40.0000,80.0000,150.0000
Residential,18090.0000,5.1182,1.9881,2.3000,3.4500,4.6000,6.9000,10.3500
SME,1832.0000,10.0000,0.0000,10.0000,10.0000,10.0000,10.0000,10.0000


In [40]:
# basic consistency check
silver_customer_month["monthly_elec_kwh"].mean()


np.float64(1141.640496645022)

In [41]:
# Drop unneccessary columns

silver_customer_month = silver_customer_month.drop(columns=["is_industrial"])
silver_customer = silver_customer.drop(columns=["sales_channel"])
silver_customer = silver_customer.rename(columns={"sales_channel_en": "sales_channel"})

#### 2.2.2 Exploratory Data Analysis

##### Unit Economics and Margin Deterioration Risk

1. Profitability differences are structural by segment. Corporate median monthly margin ≈ -13.6€, Residential ≈ 6.8€, SME ≈ 11.4€, with extreme loss months down to ≈ -3,331€ driving dispersion rather than typical performance.
2. Margin level does not explain churn. Median total margin is nearly identical for non-churn vs churn (≈ 6.96€ vs 7.01€), indicating churners are not systematically less profitable.
3. Volatility does not meaningfully differ by churn status. Margin volatility median is nearly identical (≈ 0.64 non-churn vs ≈ 0.62 churn), indicating no instability pattern before switching. However, bill volatility is structurally higher among retained Corporate customers (≈ 376€ vs 175€), reflecting segment-driven exposure rather than churn risk.
4. No systematic margin deterioration before renewal. Pre-renewal margin trend (0-3m minus 3-6m) shows minimal separation, with electricity margin trend similarly near zero for both groups.
5. Consumption trends do not show systematic disengagement before churn. Median electricity and gas consumption changes in the 0–3m vs 3–6m pre-renewal window are similar across non-churn and churn customers, with no consistent negative shift among churners. This suggests switching is not preceded by gradual usage reduction.

**Net:** Portfolio economics vary by segment, but churn is not driven by margin level, volatility, or short-term profitability/consumption shocks; switching appears lifecycle-triggered rather than financially reactive.


In [42]:
df = silver_customer_month.copy()

# ELECTRICITY

# Ensure numeric for all inputs used in elec P&L
elec_inputs = [
    "elec_kwh_tier_1_peak", "elec_kwh_tier_2_standard", "elec_kwh_tier_3_offpeak",
    "variable_price_tier1_eur_kwh", "variable_price_tier2_eur_kwh", "variable_price_tier3_eur_kwh",
    "elec_fixed_fee_eur_month",
    "monthly_elec_kwh", "elec_var_cost_eur_kwh", "peaje_elec_eur_kwh",
    "elec_fixed_cost_eur_month",
]
for c in elec_inputs:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Treat missing price/cost inputs as 0 for P&L arithmetic (avoids NaN propagation)
elec_fill0 = [
    "elec_kwh_tier_1_peak", "elec_kwh_tier_2_standard", "elec_kwh_tier_3_offpeak",
    "variable_price_tier1_eur_kwh", "variable_price_tier2_eur_kwh", "variable_price_tier3_eur_kwh",
    "elec_fixed_fee_eur_month",
    "monthly_elec_kwh", "elec_var_cost_eur_kwh", "peaje_elec_eur_kwh",
    "elec_fixed_cost_eur_month",
]
df[elec_fill0] = df[elec_fill0].fillna(0)

# Variable revenue (3 tiers)
df["elec_revenue_variable"] = (
    df["elec_kwh_tier_1_peak"] * df["variable_price_tier1_eur_kwh"] +
    df["elec_kwh_tier_2_standard"] * df["variable_price_tier2_eur_kwh"] +
    df["elec_kwh_tier_3_offpeak"] * df["variable_price_tier3_eur_kwh"]
)

# Fixed revenue
df["elec_revenue_fixed"] = df["elec_fixed_fee_eur_month"]

# Variable cost (wholesale + peaje)
df["elec_cost_variable"] = df["monthly_elec_kwh"] * (df["elec_var_cost_eur_kwh"] + df["peaje_elec_eur_kwh"])

# Fixed cost
df["elec_cost_fixed"] = df["elec_fixed_cost_eur_month"]

# Electricity margin
df["elec_margin"] = (
    df["elec_revenue_variable"] +
    df["elec_revenue_fixed"] -
    df["elec_cost_variable"] -
    df["elec_cost_fixed"]
)

In [43]:
# GAS

# Ensure numeric for all inputs used in gas P&L
gas_inputs = [
    "monthly_gas_m3", "gas_variable_price_eur_m3",
    "gas_fixed_revenue_eur_year", "gas_var_cost_eur_m3",
    "gas_fixed_cost_eur_year",
]
for c in gas_inputs:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# IMPORTANT:
# - gas_variable_price_eur_m3 is the correct column name per your silver_customer_month columns screenshot
# - keep annual fields annual, then convert to monthly

# Fill NaNs with 0 for P&L arithmetic (prevents NaN propagation for elec-only customers)
gas_fill0 = [
    "monthly_gas_m3", "gas_variable_price_eur_m3",
    "gas_fixed_revenue_eur_year", "gas_var_cost_eur_m3",
    "gas_fixed_cost_eur_year",
]
df[gas_fill0] = df[gas_fill0].fillna(0)

# Variable revenue
df["gas_revenue_variable"] = df["monthly_gas_m3"] * df["gas_variable_price_eur_m3"]

# Fixed revenue (annual → monthly)
df["gas_revenue_fixed"] = df["gas_fixed_revenue_eur_year"] / 12

# Variable cost
df["gas_cost_variable"] = df["monthly_gas_m3"] * df["gas_var_cost_eur_m3"]

# Fixed cost (annual → monthly)
df["gas_cost_fixed"] = df["gas_fixed_cost_eur_year"] / 12

# Gas margin
df["gas_margin"] = (
    df["gas_revenue_variable"] +
    df["gas_revenue_fixed"] -
    df["gas_cost_variable"] -
    df["gas_cost_fixed"]
)

In [44]:
# TOTAL

# Totals computed from components that are already NaN-safe
df["total_revenue"] = (
    df["elec_revenue_variable"] +
    df["elec_revenue_fixed"] +
    df["gas_revenue_variable"] +
    df["gas_revenue_fixed"]
)

df["total_cost"] = (
    df["elec_cost_variable"] +
    df["elec_cost_fixed"] +
    df["gas_cost_variable"] +
    df["gas_cost_fixed"]
)

df["total_margin"] = df["total_revenue"] - df["total_cost"]

# Assign back (overwrite the table only after successful recompute)
silver_customer_month = df

print("Margin computation complete (NaN-safe totals).")

# Quick QA: confirm NaNs reduced
print("NaN rate total_margin:", silver_customer_month["total_margin"].isna().mean())
print("NaN rate gas_revenue_variable:", silver_customer_month["gas_revenue_variable"].isna().mean())

Margin computation complete (NaN-safe totals).
NaN rate total_margin: 0.0
NaN rate gas_revenue_variable: 0.0


In [45]:
silver_customer_month[[
    "elec_margin",
    "gas_margin",
    "total_margin"
]].describe()


,elec_margin,gas_margin,total_margin
count,241188.0000,241188.0000,241188.0000
mean,-12.5613,7.5970,-4.9643
std,74.0479,41.4276,77.5027
min,-3334.4469,-1454.9657,-3331.3536
25%,-0.4132,5.3114,2.9143
50%,1.9080,5.3114,6.9673
75%,3.1544,5.3174,8.3300
max,3169.1443,2605.6037,3174.4556


In [46]:
# isolate extreme negative months
silver_customer_month.sort_values("total_margin").head(20)[
    ["customer_id", "month", "elec_margin", "gas_margin", "total_margin"]
]


,customer_id,month,elec_margin,gas_margin,total_margin
239538,C19962,2024-07,-3334.4469,3.0933,-3331.3536
239539,C19962,2024-08,-3062.1467,3.0933,-3059.0534
42083,C03507,2024-12,-2982.1529,5.3114,-2976.8415
42072,C03507,2024-01,-2629.0218,5.3114,-2623.7104
42079,C03507,2024-08,-2325.4797,5.3179,-2320.1618
42078,C03507,2024-07,-2298.7319,5.3179,-2293.4140
152514,C12710,2024-07,-2292.4944,2.9835,-2289.5109
52843,C04404,2024-08,-2240.5644,5.3179,-2235.2464
191016,C15919,2024-01,-755.5134,-1454.9657,-2210.4790
152515,C12710,2024-08,-2145.7797,2.9835,-2142.7962


In [47]:
# Validate Per-kWh Margin
(
    silver_customer_month.loc[
        silver_customer_month["monthly_elec_kwh"] > 0,
        "elec_margin"
    ]
    /
    silver_customer_month.loc[
        silver_customer_month["monthly_elec_kwh"] > 0,
        "monthly_elec_kwh"
    ]
).describe()



count   235601.0000
mean         0.7239
std         79.7820
min         -0.0767
25%         -0.0019
50%          0.0262
75%          0.0851
max      36460.4686
dtype: float64

In [48]:
# Margin by segment

silver_customer_month.merge(
    silver_customer[["customer_id","segment"]],
    on="customer_id",
    how="left"
).groupby("segment")["total_margin"].describe()


,count,mean,std,min,25%,50%,75%,max
segment,,,,,,,,
Corporate,2124.0000,-34.5472,183.7920,-1424.8565,-64.4772,-13.6230,18.5203,2012.3568
Residential,217080.0000,-6.2863,79.2073,-3331.3536,2.5898,6.7968,7.9815,3174.4556
SME,21984.0000,10.9476,18.2295,-78.7148,9.3794,11.3768,12.1915,1549.4490


In [49]:
(
    silver_customer_month
        .merge(silver_customer[["customer_id", "segment"]], on="customer_id", how="left")
        .groupby("segment")["total_margin"]
        .median()
)


segment
Corporate     -13.6230
Residential     6.7968
SME            11.3768
Name: total_margin, dtype: float64

In [50]:
# check corporate electricity margin per kWh -> Corporate customers are basically being priced at near-zero margin per kWh
# Corporate pricing is almost wholesale pass-through.
(
    silver_customer_month
        .merge(silver_customer[["customer_id","segment"]], on="customer_id")
        .query("segment == 'Corporate' & monthly_elec_kwh > 0")
        .assign(margin_per_kwh=lambda x: x["elec_margin"] / x["monthly_elec_kwh"])
)["margin_per_kwh"].describe()


count   2124.0000
mean      -0.0039
std        0.0073
min       -0.0282
25%       -0.0089
50%       -0.0037
75%        0.0008
max        0.0480
Name: margin_per_kwh, dtype: float64

In [51]:
# Compare with SME -> SME has a real margin buffer.
(
    silver_customer_month
        .merge(silver_customer[["customer_id","segment"]], on="customer_id")
        .query("segment == 'SME' & monthly_elec_kwh > 0")
        .assign(margin_per_kwh=lambda x: x["elec_margin"] / x["monthly_elec_kwh"])
)["margin_per_kwh"].describe()


count   21744.0000
mean        0.6402
std         9.9341
min        -0.0347
25%         0.0243
50%         0.0772
75%         0.2032
max       401.2080
Name: margin_per_kwh, dtype: float64

In [52]:
# Check if corporate losses concentrated in specific months
(
    silver_customer_month
        .merge(silver_customer[["customer_id","segment"]], on="customer_id")
        .query("segment == 'Corporate'")
        .groupby("month")["total_margin"]
        .sum()
        .sort_values()
        .head(12)
)


month
2024-07   -17033.9476
2024-02   -12065.9557
2024-08    -9280.7098
2024-09    -8538.4208
2024-03    -7942.0024
2024-10    -7596.4064
2024-04    -4070.4474
2024-06    -3554.5672
2024-11    -3438.7745
2024-05    -2362.6347
2024-12     1172.9667
2024-01     1332.6848
Name: total_margin, dtype: float64

In [53]:
# check if corporate churners have worse margins before churn -> identical so margin level is not differentiating churners vs non-churners
(
    silver_customer_month
        .merge(silver_customer[["customer_id","segment","churn"]], on="customer_id")
        .query("segment == 'Corporate'")
        .groupby("churn")["total_margin"]
        .median()
)


churn
0   -14.0617
1   -11.0464
Name: total_margin, dtype: float64

In [54]:
# Overall monthly margin distribution by churn
(
    silver_customer_month
        .merge(
            silver_customer[["customer_id", "churn"]],
            on="customer_id",
            how="left",
            validate="many_to_one"
        )
        .groupby("churn")["total_margin"]
        .describe()
)

,count,mean,std,min,25%,50%,75%,max
churn,,,,,,,,
0,217164.0000,-5.7654,80.8433,-3331.3536,2.8343,6.9604,8.3278,3174.4556
1,24024.0000,2.2774,34.1606,-2235.2464,3.6161,7.0141,8.3576,474.0690


In [55]:
# check median
(
    silver_customer_month
        .merge(silver_customer[["customer_id","churn"]], on="customer_id")
        .groupby("churn")["total_margin"]
        .median()
)

churn
0   6.9604
1   7.0141
Name: total_margin, dtype: float64

In [56]:
# margin vs churn per segment
(
    silver_customer_month
        .merge(
            silver_customer[["customer_id", "segment", "churn"]],
            on="customer_id",
            how="left",
            validate="many_to_one"
        )
        .groupby(["segment","churn"])["total_margin"]
        .describe()
)


count     mean      std        min      25%      50%     75%       max
segment     churn                                                                             
Corporate   0       1908.0000 -36.8877 193.2507 -1424.8565 -71.6611 -14.0617 19.2348 2012.3568
            1        216.0000 -13.8727  42.7313  -269.8167 -33.1972 -11.0464  9.6708  117.5663
Residential 0     195216.0000  -7.1844  82.6322 -3331.3536   2.4896   6.7837  7.9756 3174.4556
            1      21864.0000   1.7326  35.4015 -2235.2464   3.3505   6.8930  8.0364  474.0690
SME         0      20040.0000  11.0201  19.0185   -78.7148   9.3339  11.3697 12.1855 1549.4490
            1       1944.0000  10.1994   5.3668   -41.3612   9.8134  11.4324 12.2427   53.2350

In [57]:
# check median
(
    silver_customer_month
        .merge(
            silver_customer[["customer_id","segment","churn"]],
            on="customer_id"
        )
        .groupby(["segment","churn"])["total_margin"]
        .median()
)


segment      churn
Corporate    0       -14.0617
             1       -11.0464
Residential  0         6.7837
             1         6.8930
SME          0        11.3697
             1        11.4324
Name: total_margin, dtype: float64

Margin level statistically irrelevant. It is not equal to customer perceived attractiveness. So margin is a profitability issue, not a churn issue.

In [58]:
# Compute monthly margin volatility per customer
margin_vol = (
    silver_customer_month
        .groupby("customer_id")["total_margin"]
        .std()
        .reset_index()
        .rename(columns={"total_margin": "margin_volatility"})
)


In [59]:
# Merge With Churn & Segment

margin_vol = margin_vol.merge(
    silver_customer[["customer_id", "segment", "churn"]],
    on="customer_id",
    how="left",
    validate="one_to_one"
)


In [60]:
# Compare Volatility by Churn (Overall)

margin_vol.groupby("churn")["margin_volatility"].describe()


,count,mean,std,min,25%,50%,75%,max
churn,,,,,,,,
0,18097.0000,10.9262,44.1967,0.0000,0.1899,0.6369,2.7259,1268.6942
1,2002.0000,4.9002,24.2185,0.0000,0.1929,0.6167,2.2064,813.6161


In [61]:
# Compare median -> Churners have lower volatility.

margin_vol.groupby("churn")["margin_volatility"].median()


churn
0   0.6369
1   0.6167
Name: margin_volatility, dtype: float64

In [62]:
# Compare by segment -> Non-churners have higher margin volatility.

margin_vol.groupby(["segment", "churn"])["margin_volatility"].median()


segment      churn
Corporate    0       42.0829
             1       19.6796
Residential  0        0.6683
             1        0.6169
SME          0        0.3550
             1        0.4875
Name: margin_volatility, dtype: float64

In [63]:
# Compute bill volatility per customer
bill_vol = (
    silver_customer_month
        .groupby("customer_id")["total_revenue"]
        .std()
        .reset_index()
        .rename(columns={"total_revenue": "bill_volatility"})
)


In [64]:
# Merge with customer table
bill_vol = bill_vol.merge(
    silver_customer[["customer_id", "segment", "churn"]],
    on="customer_id",
    how="left",
    validate="one_to_one"
)


In [65]:
# Compare overall
bill_vol.groupby("churn")["bill_volatility"].describe()


,count,mean,std,min,25%,50%,75%,max
churn,,,,,,,,
0,18097.0000,91.8656,386.3832,0.0000,1.1144,3.4115,17.3533,8014.8599
1,2002.0000,35.0621,146.9369,0.0000,1.1150,3.3723,12.4900,3694.0973


In [66]:
# Compare median
bill_vol.groupby("churn")["bill_volatility"].median()


churn
0   3.4115
1   3.3723
Name: bill_volatility, dtype: float64

In [67]:
# Compare by segment
bill_vol.groupby(["segment","churn"])["bill_volatility"].median()


segment      churn
Corporate    0       375.7326
             1       174.8772
Residential  0         3.4022
             1         3.2672
SME          0         2.8288
             1         3.3617
Name: bill_volatility, dtype: float64

In [68]:
# Merge renewal info into monthly data
monthly_lifecycle = silver_customer_month.merge(
    silver_customer[["customer_id", "next_renewal_date", "churn"]],
    on="customer_id",
    how="left"
)

monthly_lifecycle["month"] = pd.to_datetime(monthly_lifecycle["month"])
monthly_lifecycle["next_renewal_date"] = pd.to_datetime(monthly_lifecycle["next_renewal_date"])

# Calculate months to renewal at each month
monthly_lifecycle["months_to_renewal_at_month"] = (
    (monthly_lifecycle["next_renewal_date"] - monthly_lifecycle["month"]).dt.days / 30
)


In [69]:
pre_renewal = monthly_lifecycle[
    monthly_lifecycle["months_to_renewal_at_month"].between(0, 6)
]


In [70]:
# Bucket into near vs earlier window
pre_renewal["window"] = pd.cut(
    pre_renewal["months_to_renewal_at_month"],
    bins=[0, 3, 6],
    labels=["0_3m", "3_6m"]
)

# Compute customer-level mean margin per window
margin_window = (
    pre_renewal
    .groupby(["customer_id", "churn", "window"])["total_margin"]
    .mean()
    .unstack()
    .dropna()
    .reset_index()
)

# Calculate trend (near minus earlier)
margin_window["margin_trend_pre_renewal"] = (
    margin_window["0_3m"] - margin_window["3_6m"]
)

# Compare trend by churn
margin_window.groupby("churn")["margin_trend_pre_renewal"].describe()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/1898581622.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_renewal["window"] = pd.cut(
/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/1898581622.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["customer_id", "churn", "window"])["total_margin"]


,count,mean,std,min,25%,50%,75%,max
churn,,,,,,,,
0,2550.0000,4.3489,82.0367,-502.4525,-1.4413,-0.2163,0.0060,2196.2665
1,634.0000,-0.5376,32.9378,-181.7323,-1.5812,-0.3202,-0.0236,744.8521


In [71]:
# Electricity-specific trend
elec_window = (
    pre_renewal
    .groupby(["customer_id", "churn", "window"])["elec_margin"]
    .mean()
    .unstack()
    .dropna()
    .reset_index()
)

elec_window["elec_margin_trend"] = (
    elec_window["0_3m"] - elec_window["3_6m"]
)

elec_window.groupby("churn")["elec_margin_trend"].describe()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/3875526461.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["customer_id", "churn", "window"])["elec_margin"]


,count,mean,std,min,25%,50%,75%,max
churn,,,,,,,,
0,2550.0000,-5.5266,56.4901,-508.5657,-1.3529,-0.2201,-0.0021,2135.3097
1,634.0000,-1.2806,31.9001,-181.7323,-1.0834,-0.2684,-0.0192,748.3952


In [72]:
# Volatility Increase before renewal

vol_window = (
    pre_renewal
    .groupby(["customer_id", "churn"])["total_margin"]
    .std()
    .reset_index()
)

vol_window.groupby("churn")["total_margin"].describe()


,count,mean,std,min,25%,50%,75%,max
churn,,,,,,,,
0,5607.0000,9.6290,46.1433,0.0000,0.1474,0.5241,2.2382,1679.3064
1,1444.0000,4.4188,29.3174,0.0000,0.1359,0.4686,1.8865,1001.2629


In [73]:
# Electricity consumption trend

# Compute average consumption per window
cons_window = (
    pre_renewal
    .groupby(["customer_id", "churn", "window"])["monthly_elec_kwh"]
    .mean()
    .unstack()
    .dropna()
    .reset_index()
)

# Calculate trend (near minus earlier)
cons_window["elec_consumption_trend"] = (
    cons_window["0_3m"] - cons_window["3_6m"]
)

# Compare by churn
cons_window.groupby("churn")["elec_consumption_trend"].describe()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/1438678471.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["customer_id", "churn", "window"])["monthly_elec_kwh"]


,count,mean,std,min,25%,50%,75%,max
churn,,,,,,,,
0,2550.0000,305.4321,2570.8169,-89492.6552,1.3321,19.0436,78.9829,26670.5565
1,634.0000,95.4205,1412.9006,-32437.0878,6.7301,24.6076,82.2358,8417.5271


In [74]:
# Gas consumption trend
gas_window = (
    pre_renewal
    .groupby(["customer_id", "churn", "window"])["monthly_gas_m3"]
    .mean()
    .unstack()
    .dropna()
    .reset_index()
)

gas_window["gas_consumption_trend"] = (
    gas_window["0_3m"] - gas_window["3_6m"]
)

gas_window.groupby("churn")["gas_consumption_trend"].describe()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/2447541261.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["customer_id", "churn", "window"])["monthly_gas_m3"]


,count,mean,std,min,25%,50%,75%,max
churn,,,,,,,,
0,2550.0000,144.2819,847.4368,0.0000,0.0000,0.0000,0.0000,17701.8392
1,634.0000,19.4946,120.9836,0.0000,0.0000,0.0000,0.0000,1776.3120


In [75]:
# Relative % change
cons_window["elec_pct_change"] = (
    (cons_window["0_3m"] - cons_window["3_6m"]) /
    (cons_window["3_6m"] + 1e-6)
)

cons_window.groupby("churn")["elec_pct_change"].describe()


,count,mean,std,min,25%,50%,75%,max
churn,,,,,,,,
0,2550.0000,0.3541,0.3875,-1.0000,0.0643,0.4000,0.6558,3.3097
1,634.0000,0.4334,0.3953,-1.0000,0.1814,0.5639,0.6792,1.2112


##### Contract Renewal & Tenure Lifecycle Dynamics

1. Churn is strongly renewal-driven. Customers within 0–3 months of renewal show ~25% churn vs ~6% outside the window (~4x increase). Median months to renewal: churn ≈ 2.7 vs non-churn ≈ 5.1. Churn probability declines sharply as renewal distance increases (0–3m: 25%, 3–6m: 8%, 6–12m: 2%).

2. Tenure reduces structural churn risk. Median tenure: non-churn ≈ 46 months vs churn ≈ 32 months. Baseline churn declines from ~13% in early tenure buckets to ~6% for 5y+ customers.

3. Renewal amplifies churn across tenure groups. Within 3 months of renewal, churn spikes across all tenure buckets (~30–32% for <2y; ~27% for 2–5y; ~16% for 5y+), vs ~4–8% outside the renewal window. Tenure acts as a loyalty buffer but does not eliminate renewal risk.

4. Market and acquisition context moderate lifecycle effects. High-competition provinces show elevated churn across tenure buckets (e.g., ~9% vs ~5% for 5y+). Acquisition channel influences early stability; comparison and telemarketing channels exhibit higher early churn.

5. Structural differences exist across segments and residence type. Corporate customers show higher early-tenure churn (up to ~23%); second residences display higher instability in 0–12m tenure. Contracted power does not materially alter tenure-driven churn once lifecycle is controlled.

6. Expired contracts show structurally elevated churn risk. Although only ~3% of the portfolio is expired, churn among expired contracts (~10.9%) is higher than active contracts (~9.9%), and reaches ~20% in Corporate segments. This suggests post-renewal inaction significantly increases churn probability.

**Net:** Churn is primarily a lifecycle event triggered around renewal, moderated by tenure and amplified by competitive intensity and acquisition channel. The highest-signal features are renewal proximity (months_to_renewal / renewal_bucket / is_within_3m_of_renewal), tenure (tenure_months / tenure_bucket), and their interactions.

In [76]:
# Months to renewal vs churn

(
    silver_customer
        .assign(
            months_to_renewal = (
                pd.to_datetime(silver_customer["next_renewal_date"]) -
                pd.to_datetime("2024-12-31")
            ).dt.days / 30
        )
        .groupby("churn")["months_to_renewal"]
        .describe()
)


,count,mean,std,min,25%,50%,75%,max
churn,,,,,,,,
0,18097.0000,4.8873,2.2923,-0.9667,3.5000,5.1000,6.5000,9.9333
1,2002.0000,3.0632,1.8782,-0.9667,1.7333,2.7333,4.2000,9.5333


In [77]:
# Visual distribution of renewal timing
(
    silver_customer
        .assign(
            months_to_renewal = (
                pd.to_datetime(silver_customer["next_renewal_date"]) -
                pd.to_datetime("2024-12-31")
            ).dt.days / 30
        )
        .groupby("churn")["months_to_renewal"]
        .median()
)


churn
0   5.1000
1   2.7333
Name: months_to_renewal, dtype: float64

In [78]:
# Renewal window effect
sc_lifecycle = silver_customer.copy()

sc_lifecycle["months_to_renewal"] = (
    pd.to_datetime(sc_lifecycle["next_renewal_date"]) -
    pd.to_datetime("2024-12-31")
).dt.days / 30

sc_lifecycle["is_within_3m_of_renewal"] = sc_lifecycle["months_to_renewal"].between(0, 3)

sc_lifecycle.groupby("is_within_3m_of_renewal")["churn"].mean()


is_within_3m_of_renewal
False   0.0602
True    0.2466
Name: churn, dtype: float64

In [79]:
# Time since last product change

sc_lifecycle["months_since_last_change"] = (
    pd.to_datetime("2024-12-31") -
    pd.to_datetime(sc_lifecycle["last_product_change_date"])
).dt.days / 30

sc_lifecycle.groupby("churn")["months_since_last_change"].median()


churn
0   7.4333
1   4.9333
Name: months_since_last_change, dtype: float64

In [80]:
sc_lifecycle.groupby("segment")["is_within_3m_of_renewal"].mean()


segment
Corporate     0.1864
Residential   0.2107
SME           0.2189
Name: is_within_3m_of_renewal, dtype: float64

In [81]:
sc_lifecycle.groupby(["segment","is_within_3m_of_renewal"])["churn"].mean()


segment      is_within_3m_of_renewal
Corporate    False                     0.0764
             True                      0.2121
Residential  False                     0.0607
             True                      0.2506
SME          False                     0.0538
             True                      0.2120
Name: churn, dtype: float64

In [82]:
silver_customer_month["month"].max()


'2024-12'

In [83]:
# churn probability by month to renewal bucket

sc_lifecycle["renewal_bucket"] = pd.cut(
    sc_lifecycle["months_to_renewal"],
    bins=[-1, 0, 3, 6, 12],
    labels=["expired_or_now", "0-3m", "3-6m", "6-12m"]
)

sc_lifecycle.groupby("renewal_bucket")["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/1725118587.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_lifecycle.groupby("renewal_bucket")["churn"].mean()


renewal_bucket
expired_or_now   0.1084
0-3m             0.2467
3-6m             0.0814
6-12m            0.0234
Name: churn, dtype: float64

In [84]:
# Tenure calculation (as of end of observation window)
sc_lifecycle["tenure_months"] = (
    pd.to_datetime("2024-12-31") -
    pd.to_datetime(sc_lifecycle["customer_first_activation_date"])
).dt.days / 30

# Compare tenure by churn
sc_lifecycle.groupby("churn")["tenure_months"].describe()


,count,mean,std,min,25%,50%,75%,max
churn,,,,,,,,
0,18097.0000,57.7011,47.7735,0.0000,20.1000,46.0000,73.0667,182.4667
1,2002.0000,40.1159,33.0018,0.0667,14.5167,31.9667,51.0500,127.6000


In [85]:
sc_lifecycle["tenure_bucket"] = pd.cut(
    sc_lifecycle["tenure_months"],
    bins=[0, 6, 12, 24, 60, 200],
    labels=["0–6m", "6–12m", "1–2y", "2–5y", "5y+"]
)

sc_lifecycle.groupby("tenure_bucket")["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/778137262.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_lifecycle.groupby("tenure_bucket")["churn"].mean()


tenure_bucket
0–6m    0.1339
6–12m   0.1303
1–2y    0.1259
2–5y    0.1136
5y+     0.0612
Name: churn, dtype: float64

In [86]:
sc_lifecycle.groupby(
    ["tenure_bucket", "is_within_3m_of_renewal"]
)["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/1460005373.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_lifecycle.groupby(


tenure_bucket  is_within_3m_of_renewal
0–6m           False                     0.0777
               True                      0.3207
6–12m          False                     0.0782
               True                      0.3035
1–2y           False                     0.0741
               True                      0.3039
2–5y           False                     0.0710
               True                      0.2694
5y+            False                     0.0374
               True                      0.1606
Name: churn, dtype: float64

In [87]:
# Tenure by sales channel
sc_lifecycle.groupby("sales_channel")["tenure_months"].describe()

# Churn rate by sales_channel × tenure_bucket
sc_lifecycle.groupby(
    ["sales_channel", "tenure_bucket"]
)["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/3204251447.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_lifecycle.groupby(


sales_channel         tenure_bucket
Comparison Website    0–6m            0.1111
                      6–12m           0.1000
                      1–2y            0.0623
                      2–5y            0.0665
                      5y+             0.0356
In-Person Commercial  0–6m            0.1710
                      6–12m           0.1569
                      1–2y            0.1575
                      2–5y            0.1445
                      5y+             0.0754
Office                0–6m            0.1226
                      6–12m           0.1210
                      1–2y            0.1475
                      2–5y            0.1234
                      5y+             0.0628
Own Website           0–6m            0.0000
                      6–12m              NaN
                      1–2y            0.1667
                      2–5y            0.0000
                      5y+             0.0000
Telemarketing         0–6m            0.0808
                   

In [88]:
# Tenure distribution by segment
sc_lifecycle.groupby("segment")["tenure_months"].describe()

# Churn by segment × tenure_bucket
sc_lifecycle.groupby(
    ["segment", "tenure_bucket"]
)["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/2968208963.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_lifecycle.groupby(


segment      tenure_bucket
Corporate    0–6m            0.1818
             6–12m           0.2308
             1–2y            0.1071
             2–5y            0.1154
             5y+             0.0556
Residential  0–6m            0.1300
             6–12m           0.1295
             1–2y            0.1262
             2–5y            0.1159
             5y+             0.0628
SME          0–6m            0.1690
             6–12m           0.1295
             1–2y            0.1245
             2–5y            0.0909
             5y+             0.0466
Name: churn, dtype: float64

In [89]:
# Tenure vs competition flag
sc_lifecycle.groupby("is_high_competition_province")["tenure_months"].describe()

# Churn by tenure × competition
sc_lifecycle.groupby(
    ["is_high_competition_province", "tenure_bucket"]
)["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/3836371366.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_lifecycle.groupby(


is_high_competition_province  tenure_bucket
0                             0–6m            0.1127
                              6–12m           0.1097
                              1–2y            0.1039
                              2–5y            0.0915
                              5y+             0.0493
1                             0–6m            0.1813
                              6–12m           0.1777
                              1–2y            0.1746
                              2–5y            0.1658
                              5y+             0.0886
Name: churn, dtype: float64

In [90]:
# Tenure by residence type
sc_lifecycle.groupby("is_second_residence")["tenure_months"].describe()

# Churn by tenure × second residence
sc_lifecycle.groupby(
    ["is_second_residence", "tenure_bucket"]
)["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/2913163308.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_lifecycle.groupby(


is_second_residence  tenure_bucket
0                    0–6m            0.1334
                     6–12m           0.1297
                     1–2y            0.1266
                     2–5y            0.1130
                     5y+             0.0613
1                    0–6m            0.1667
                     6–12m           0.1579
                     1–2y            0.0952
                     2–5y            0.1429
                     5y+             0.0559
Name: churn, dtype: float64

In [91]:
# Bucket contracted power
sc_lifecycle["power_bucket"] = pd.cut(
    sc_lifecycle["contracted_power_kw"],
    bins=[0, 3.5, 5, 8, 20],
    labels=["<=3.5kW", "3.5–5kW", "5–8kW", "8kW+"]
)

# Churn by tenure × power
sc_lifecycle.groupby(
    ["power_bucket", "tenure_bucket"]
)["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/4174021515.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_lifecycle.groupby(


power_bucket  tenure_bucket
<=3.5kW       0–6m            0.1467
              6–12m           0.1309
              1–2y            0.1225
              2–5y            0.1269
              5y+             0.0638
3.5–5kW       0–6m            0.1089
              6–12m           0.1038
              1–2y            0.1497
              2–5y            0.1236
              5y+             0.0677
5–8kW         0–6m            0.1244
              6–12m           0.1557
              1–2y            0.1040
              2–5y            0.1048
              5y+             0.0643
8kW+          0–6m            0.1468
              6–12m           0.1210
              1–2y            0.1357
              2–5y            0.0968
              5y+             0.0507
Name: churn, dtype: float64

In [92]:
# Tenure x Months to renewal

sc_lifecycle.groupby(
    ["tenure_bucket", "renewal_bucket"]
)["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/1599856178.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_lifecycle.groupby(


tenure_bucket  renewal_bucket
0–6m           expired_or_now   0.1053
               0-3m             0.3207
               3-6m             0.1070
               6-12m            0.0285
6–12m          expired_or_now   0.1333
               0-3m             0.3035
               3-6m             0.1054
               6-12m            0.0304
1–2y           expired_or_now   0.1333
               0-3m             0.3043
               3-6m             0.0973
               6-12m            0.0306
2–5y           expired_or_now   0.1171
               0-3m             0.2694
               3-6m             0.0968
               6-12m            0.0274
5y+            expired_or_now   0.0842
               0-3m             0.1606
               3-6m             0.0500
               6-12m            0.0147
Name: churn, dtype: float64

In [93]:
sc_lifecycle.groupby(
    ["is_within_3m_of_renewal", "tenure_bucket"]
)["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/3850903483.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_lifecycle.groupby(


is_within_3m_of_renewal  tenure_bucket
False                    0–6m            0.0777
                         6–12m           0.0782
                         1–2y            0.0741
                         2–5y            0.0710
                         5y+             0.0374
True                     0–6m            0.3207
                         6–12m           0.3035
                         1–2y            0.3039
                         2–5y            0.2694
                         5y+             0.1606
Name: churn, dtype: float64

In [94]:
# Expired contract churn check
sc_lifecycle["is_expired_contract"] = (sc_lifecycle["months_to_renewal"] < 0).astype(int)

print("Expired contract prevalence:")
print(sc_lifecycle["is_expired_contract"].value_counts(normalize=True).round(4))

print("\nChurn rate — expired vs active contracts:")
print(sc_lifecycle.groupby("is_expired_contract")["churn"].agg(
    customers="count",
    churn_rate="mean"
))

print("\nExpired × renewal_bucket × churn (verify signal beyond bucket):")
print(sc_lifecycle.groupby(["renewal_bucket","is_expired_contract"])["churn"].mean().unstack())

print("\nExpired × segment:")
print(sc_lifecycle.groupby(["segment","is_expired_contract"])["churn"].mean().unstack())

Expired contract prevalence:
is_expired_contract
0   0.9711
1   0.0289
Name: proportion, dtype: float64

Churn rate — expired vs active contracts:
                     customers  churn_rate
is_expired_contract                       
0                        19519      0.0993
1                          580      0.1086

Expired × renewal_bucket × churn (verify signal beyond bucket):
is_expired_contract      0      1
renewal_bucket                   
expired_or_now      0.0000 0.1086
0-3m                0.2467    NaN
3-6m                0.0814    NaN
6-12m               0.0234    NaN

Expired × segment:
is_expired_contract      0      1
segment                          
Corporate           0.0988 0.2000
Residential         0.1004 0.1103
SME                 0.0885 0.0862


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/1903213125.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(sc_lifecycle.groupby(["renewal_bucket","is_expired_contract"])["churn"].mean().unstack())


##### Portfolio Structure & Competitive Exposure

1. Acquisition channel materially shapes both retention and economics.  
   Comparison Website customers show the lowest churn (~6%) but highly negative median margins (~ -20€), particularly among Residential (~ -23.7€) and Corporate (~ -59.8€) segments. In-Person Commercial has higher churn (~12%; ~29% within renewal window) but positive margins (~ +2.9€). Office and Telemarketing channels exhibit balanced churn (9–10%) and stable positive margins (~3–4€). Channel quality must be evaluated on lifetime economics, not churn alone.

2. Competition increases churn without materially compressing margins.  
   Median margins remain similar across regions (~2.85€ vs ~2.63€), but churn rises sharply (8% → 14%). Within 3 months of renewal, churn increases from ~22% to ~31% in high-competition provinces. Competition amplifies renewal-triggered switching rather than directly eroding price.

3. Competitive pressure disproportionately affects Residential and SME segments.  
   Residential churn increases from ~8% to ~14%; SME from ~7% to ~12%, while Corporate remains relatively stable (~10–11%). Market intensity erodes loyalty primarily through switching behavior.

4. Dual fuel bundling increases retention but reduces margin.  
   Churn: ~8% (dual) vs ~10% (single). Within renewal window: ~20% (dual) vs ~26% (single). However, median total margin is lower for dual fuel (~2.79€ vs ~4.00€). Bundling dampens renewal risk but dilutes per-customer profitability.

**Net:** Portfolio structure (channel, competition, segment mix, bundling) shapes churn exposure and economic outcomes. Competition and renewal proximity amplify switching risk, while acquisition strategy and bundling create clear retention–profitability trade-offs.


In [95]:
# Merge lifecycle with monthly margin (customer-level aggregation first)
customer_margin = (
    silver_customer_month
    .groupby("customer_id")["total_margin"]
    .median()
    .reset_index()
)

acq_df = (
    sc_lifecycle
    .merge(customer_margin, on="customer_id", how="left")
)

# Channel overview
acq_df.groupby("sales_channel").agg(
    customers=("customer_id", "count"),
    churn_rate=("churn", "mean"),
    median_tenure=("tenure_months", "median"),
    median_margin=("total_margin", "median")
).sort_values("churn_rate", ascending=False)


,customers,churn_rate,median_tenure,median_margin
sales_channel,,,,
In-Person Commercial,9158,0.1248,44.3333,7.0302
Office,1836,0.1046,43.2667,7.4248
Telemarketing,1218,0.0854,44.1667,7.2140
Unknown,5267,0.0769,43.4667,6.9121
Comparison Website,2597,0.0605,45.3333,2.4359
Own Website,23,0.0435,41.6667,8.3503


In [96]:
# Channel x Renewal sensitivity

acq_df.groupby(
    ["sales_channel", "is_within_3m_of_renewal"]
)["churn"].mean()


sales_channel         is_within_3m_of_renewal
Comparison Website    False                     0.0312
                      True                      0.1786
In-Person Commercial  False                     0.0781
                      True                      0.2920
Office                False                     0.0605
                      True                      0.2638
Own Website           False                     0.0000
                      True                      0.1667
Telemarketing         False                     0.0524
                      True                      0.2045
Unknown               False                     0.0461
                      True                      0.1987
Name: churn, dtype: float64

In [97]:
# Channel x Competitive Province

acq_df.groupby(
    ["sales_channel", "is_high_competition_province"]
)["churn"].mean()


sales_channel         is_high_competition_province
Comparison Website    0                              0.0487
                      1                              0.0884
In-Person Commercial  0                              0.1042
                      1                              0.1714
Office                0                              0.0745
                      1                              0.1729
Own Website           0                              0.0556
                      1                              0.0000
Telemarketing         0                              0.0664
                      1                              0.1309
Unknown               0                              0.0642
                      1                              0.1065
Name: churn, dtype: float64

In [98]:
# Channel x Tenure Conversion

acq_df.groupby(
    ["sales_channel", "tenure_bucket"]
)["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/3503758525.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  acq_df.groupby(


sales_channel         tenure_bucket
Comparison Website    0–6m            0.1111
                      6–12m           0.1000
                      1–2y            0.0623
                      2–5y            0.0665
                      5y+             0.0356
In-Person Commercial  0–6m            0.1710
                      6–12m           0.1569
                      1–2y            0.1575
                      2–5y            0.1445
                      5y+             0.0754
Office                0–6m            0.1226
                      6–12m           0.1210
                      1–2y            0.1475
                      2–5y            0.1234
                      5y+             0.0628
Own Website           0–6m            0.0000
                      6–12m              NaN
                      1–2y            0.1667
                      2–5y            0.0000
                      5y+             0.0000
Telemarketing         0–6m            0.0808
                   

In [99]:
# Merge margin volatility into acquisition dataframe
acq_df = acq_df.merge(
    margin_vol[["customer_id", "margin_volatility"]],
    on="customer_id",
    how="left"
)

# Channel profitability stability
acq_df.groupby("sales_channel").agg(
    customers=("customer_id", "count"),
    median_margin=("total_margin", "median"),
    median_margin_volatility=("margin_volatility", "median"),
    churn_rate=("churn", "mean")
).sort_values("churn_rate", ascending=False)


,customers,median_margin,median_margin_volatility,churn_rate
sales_channel,,,,
In-Person Commercial,9158,7.0302,0.6118,0.1248
Office,1836,7.4248,0.2930,0.1046
Telemarketing,1218,7.2140,0.4413,0.0854
Unknown,5267,6.9121,0.6036,0.0769
Comparison Website,2597,2.4359,4.5035,0.0605
Own Website,23,8.3503,0.3014,0.0435


In [100]:
# Aggregate customer-level median margins
customer_margin_split = (
    silver_customer_month
    .groupby("customer_id")
    .agg(
        elec_margin_med=("elec_margin", "median"),
        gas_margin_med=("gas_margin", "median"),
        total_margin_med=("total_margin", "median")
    )
    .reset_index()
)

# Merge into acquisition dataframe
acq_split = acq_df.merge(
    customer_margin_split,
    on="customer_id",
    how="left"
)

# Channel decomposition
acq_split.groupby("sales_channel").agg(
    median_elec_margin=("elec_margin_med", "median"),
    median_gas_margin=("gas_margin_med", "median"),
    median_total_margin=("total_margin_med", "median"),
    churn_rate=("churn", "mean")
).sort_values("median_total_margin")


,median_elec_margin,median_gas_margin,median_total_margin,churn_rate
sales_channel,,,,
Comparison Website,-1.6443,5.3114,2.4359,0.0605
Unknown,1.9361,5.3114,6.9121,0.0769
In-Person Commercial,1.9517,5.3114,7.0302,0.1248
Telemarketing,2.2218,5.3114,7.2140,0.0854
Office,2.3581,5.3114,7.4248,0.1046
Own Website,3.2419,5.3114,8.3503,0.0435


In [101]:
# Channel × Segment distribution
acq_split.groupby(["sales_channel", "segment"]).agg(
    customers=("customer_id", "count"),
    churn_rate=("churn", "mean"),
    median_total_margin=("total_margin_med", "median")
).sort_values(["sales_channel", "segment"])


customers  churn_rate  median_total_margin
sales_channel        segment                                                
Comparison Website   Corporate           79      0.1013             -11.2522
                     Residential       2351      0.0600               1.7432
                     SME                167      0.0479              10.8473
In-Person Commercial Corporate           55      0.0909             -20.5288
                     Residential       8235      0.1268               6.8757
                     SME                868      0.1083              11.4510
Office               Corporate            1      0.0000             -22.5086
                     Residential       1654      0.1034               7.2935
                     SME                181      0.1160              11.5907
Own Website          Residential         21      0.0476               8.3503
                     SME                  2      0.0000               8.5482
Telemarketing        Corporate            1      1.0000              12.3961
                     Residential       1085      0.0839               7.0878
                     SME                132      0.0909              11.4127
Unknown              Corporate           41      0.0976              -9.6665
                     Residential       4744      0.0788               6.7308
                     SME                482      0.0560              11.0622

In [102]:
# Channel x Competition x Margin
acq_split.groupby(
    ["sales_channel", "is_high_competition_province"]
).agg(
    customers=("customer_id", "count"),
    median_total_margin=("total_margin_med", "median"),
    churn_rate=("churn", "mean")
).sort_values(["sales_channel", "is_high_competition_province"])


customers  median_total_margin  churn_rate
sales_channel        is_high_competition_province                                            
Comparison Website   0                                  1828               2.3534      0.0487
                     1                                   769               2.7229      0.0884
In-Person Commercial 0                                  6346               7.0262      0.1042
                     1                                  2812               7.0483      0.1714
Office               0                                  1275               7.4266      0.0745
                     1                                   561               7.4153      0.1729
Own Website          0                                    18               8.5899      0.0556
                     1                                     5               6.5276      0.0000
Telemarketing        0                                   859               7.2097      0.0664
                     1                                   359               7.2504      0.1309
Unknown              0                                  3690               6.9590      0.0642
                     1                                  1577               6.7917      0.1065

In [103]:
# Competition x Margin Decomposition

comp_margin = (
    acq_split
    .groupby("is_high_competition_province")
    .agg(
        customers=("customer_id", "count"),
        median_elec_margin=("elec_margin_med", "median"),
        median_gas_margin=("gas_margin_med", "median"),
        median_total_margin=("total_margin_med", "median"),
        median_margin_vol=("margin_volatility", "median"),
        churn_rate=("churn", "mean")
    )
)

comp_margin


,customers,median_elec_margin,median_gas_margin,median_total_margin,median_margin_vol,churn_rate
is_high_competition_province,,,,,,
0,14016,1.9122,5.3114,6.9434,0.6162,0.0813
1,6083,1.9070,5.3114,6.9362,0.6825,0.1417


In [104]:
# Competition x Segment composition
sc_lifecycle.groupby(
    ["is_high_competition_province", "segment"]
)["customer_id"].count().unstack()


segment,Corporate,Residential,SME
is_high_competition_province,,,
0,120,12608,1288
1,57,5482,544


In [105]:
sc_lifecycle.groupby(
    ["is_high_competition_province", "segment"]
)["churn"].mean()


is_high_competition_province  segment    
0                             Corporate     0.1000
                              Residential   0.0819
                              SME           0.0738
1                             Corporate     0.1053
                              Residential   0.1439
                              SME           0.1232
Name: churn, dtype: float64

In [106]:
# Competition x Channel mix

sc_lifecycle.groupby(
    ["is_high_competition_province", "sales_channel"]
)["customer_id"].count().unstack()


sales_channel,Comparison Website,In-Person Commercial,Office,Own Website,Telemarketing,Unknown
is_high_competition_province,,,,,,
0,1828,6346,1275,18,859,3690
1,769,2812,561,5,359,1577


In [107]:
# Competiton x Renewal sensitivity

sc_lifecycle.groupby(
    ["is_high_competition_province", "is_within_3m_of_renewal"]
)["churn"].mean()


is_high_competition_province  is_within_3m_of_renewal
0                             False                     0.0468
                              True                      0.2171
1                             False                     0.0923
                              True                      0.3065
Name: churn, dtype: float64

In [108]:
# Profit at risk by competition

# Profit at risk proxy
acq_split["profit_at_risk"] = (
    acq_split["total_margin_med"] * acq_split["churn"]
)

acq_split.groupby("is_high_competition_province").agg(
    median_margin=("total_margin_med", "median"),
    churn_rate=("churn", "mean"),
    avg_profit_at_risk=("profit_at_risk", "mean")
)


,median_margin,churn_rate,avg_profit_at_risk
is_high_competition_province,,,
0,6.9434,0.0813,0.2134
1,6.9362,0.1417,0.3689


In [109]:
# Determine fuel type per customer
fuel_flag = (
    silver_customer_month
    .groupby("customer_id")
    .agg(
        total_elec=("monthly_elec_kwh", "sum"),
        total_gas=("monthly_gas_m3", "sum")
    )
    .reset_index()
)

fuel_flag["is_dual_fuel"] = (
    (fuel_flag["total_elec"] > 0) & (fuel_flag["total_gas"] > 0)
)

# Merge into lifecycle
sc_dual = sc_lifecycle.merge(
    fuel_flag[["customer_id", "is_dual_fuel"]],
    on="customer_id",
    how="left"
)


In [110]:
# Basic Stickiness test

sc_dual.groupby("is_dual_fuel").agg(
    customers=("customer_id", "count"),
    churn_rate=("churn", "mean"),
    median_tenure=("tenure_months", "median")
)


,customers,churn_rate,median_tenure
is_dual_fuel,,,
False,16497,0.1036,44.1333
True,3602,0.0813,44.2333


In [111]:
# Dual Fuel x Renewal Sensitivity

sc_dual.groupby(
    ["is_dual_fuel", "is_within_3m_of_renewal"]
)["churn"].mean()


is_dual_fuel  is_within_3m_of_renewal
False         False                     0.0620
              True                      0.2560
True          False                     0.0525
              True                      0.1997
Name: churn, dtype: float64

In [112]:
# Dual Fuel x Competition

sc_dual.groupby(
    ["is_dual_fuel", "is_high_competition_province"]
)["churn"].mean()


is_dual_fuel  is_high_competition_province
False         0                              0.0856
              1                              0.1451
True          0                              0.0618
              1                              0.1263
Name: churn, dtype: float64

In [113]:
# Dual Fuel x Tenure

sc_dual.groupby(
    ["is_dual_fuel", "tenure_bucket"]
)["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/4258335582.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_dual.groupby(


is_dual_fuel  tenure_bucket
False         0–6m            0.1415
              6–12m           0.1362
              1–2y            0.1289
              2–5y            0.1174
              5y+             0.0642
True          0–6m            0.0986
              6–12m           0.1028
              1–2y            0.1118
              2–5y            0.0959
              5y+             0.0479
Name: churn, dtype: float64

In [114]:
# Profitability check

# Merge margin medians
dual_margin = acq_split.merge(
    fuel_flag[["customer_id", "is_dual_fuel"]],
    on="customer_id",
    how="left"
)

dual_margin.groupby("is_dual_fuel").agg(
    median_total_margin=("total_margin_med", "median"),
    churn_rate=("churn", "mean")
)


,median_total_margin,churn_rate
is_dual_fuel,,
False,7.2357,0.1036
True,2.8285,0.0813


##### Interaction & Behavioral Escalation Risk

1. No temporal leakage.  
   All interaction dates occur strictly before renewal. Results remain unchanged under 7-day and 30-day pre-renewal buffers. Median interaction timing occurs months before renewal (Negative ≈ 159 days, Neutral ≈ 212 days, Positive ≈ 235 days).

2. Sentiment is an extremely strong churn discriminator.  
   - Negative sentiment → ~99% churn  
   - Neutral sentiment → ~15–30% churn (timing-dependent)  
   - Positive sentiment → ~0% churn  
   Among negative-sentiment churners, virtually none churn within 7 days of interaction, confirming this is not last-minute cancellation logging.

3. Sentiment overrides pure renewal timing effects.  
   Renewal proximity increases churn, but negative sentiment dominates across all timing buckets (~99–100% churn regardless of renewal distance). Renewal risk becomes severe primarily when sentiment is negative.

4. Sentiment amplifies behavioral intent risk.  
   - Negative + Complaint / Cancellation / Pricing → ~100% churn  
   - Neutral + Complaint → ~78% churn  
   - Positive sentiment → ~0% churn across intents  
   Complaint intent is structurally terminal independent of fuel type or competition.

5. Competition and interaction context magnify sentiment risk.  
   High-competition provinces increase churn materially for pricing, renewal, and service intents. Interaction acts as the activation mechanism through which dissatisfaction and competition translate into churn.

6. Dual fuel does not mitigate negative sentiment.  
   Bundling reduces baseline churn but does not protect against complaint-driven or negative-sentiment exits.

**Net Insight:**  
Sentiment captures a near-deterministic dissatisfaction signal embedded in CRM interaction text. It is not leakage, but it behaves as a renewal-stage escalation flag. Churn in this dataset is intent-driven and sentiment-amplified, with renewal proximity acting as a timing accelerator rather than the root cause.

**Modeling Implication:**  
Use sentiment cautiously. It is highly predictive but may dominate model performance and reflect renewal-stage decisions rather than early warning signals. Combine:
- `customer_intent`
- `sentiment_label`
- `is_within_3m_of_renewal`
- `is_high_competition_province`
- Interaction terms between them


In [115]:
# Interaction variables

# --- Interaction flags (customer-level) ---
sc = silver_customer.copy()

# Parse dates just in case (safe)
if "date" in sc.columns:
    sc["date"] = pd.to_datetime(sc["date"], errors="coerce")

# Define "has interaction" = we have an interaction record (date and/or summary present)
sc["has_interaction"] = (
    sc["date"].notna() |
    sc["interaction_summary"].fillna("").str.strip().ne("")
).astype(int)

# If you want it persisted back into your main df:
silver_customer = sc

# Churn rate by interaction flag
silver_customer.groupby("has_interaction")["churn"].agg(
    customers="count",
    churn_rate="mean"
)


,customers,churn_rate
has_interaction,,
0,13679,0.0660
1,6420,0.1712


In [116]:
# Interaction rate among churners vs non-churners
silver_customer.groupby("churn")["has_interaction"].agg(
    customers="count",
    interaction_rate="mean"
)


,customers,interaction_rate
churn,,
0,18097,0.2940
1,2002,0.5490


In [117]:
# align interaction timing with renewal + product change (fallback to contract start)

sc = silver_customer.copy()

sc["date"] = pd.to_datetime(sc["date"], errors="coerce")
sc["next_renewal_date"] = pd.to_datetime(sc["next_renewal_date"], errors="coerce")

sc["days_to_renewal_at_interaction"] = (
    sc["next_renewal_date"] - sc["date"]
).dt.days

sc["renewal_timing_bucket"] = pd.qcut(
    sc["days_to_renewal_at_interaction"],
    q=5,
    duplicates="drop"
)

sc.groupby("renewal_timing_bucket")["churn"].mean()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/3815765036.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc.groupby("renewal_timing_bucket")["churn"].mean()


renewal_timing_bucket
(3.999, 147.0]   0.3568
(147.0, 196.0]   0.2727
(196.0, 238.0]   0.1176
(238.0, 280.0]   0.0662
(280.0, 406.0]   0.0377
Name: churn, dtype: float64

In [118]:
#Build has_interaction + renewal distance for everyone

sc_all = silver_customer.copy()

# Parse dates
for c in ["date", "next_renewal_date"]:
    if c in sc_all.columns:
        sc_all[c] = pd.to_datetime(sc_all[c], errors="coerce")

# Has interaction (latest interaction)
sc_all["has_interaction"] = (
    sc_all["date"].notna() |
    sc_all["interaction_summary"].fillna("").str.strip().ne("")
).astype(int)

# Choose a "snapshot date" to measure renewal distance for ALL customers
# Option A (best): max interaction date present in dataset (global snapshot)
snapshot_date = sc_all["date"].max()
# Option B: if you prefer "today", use: pd.Timestamp.today().normalize()

sc_all["days_to_renewal_snapshot"] = (
    sc_all["next_renewal_date"] - snapshot_date
).dt.days


In [119]:
# Create renewal buckets for everyone (automatic)
# Keep only customers with a renewal date for bucketing
mask = sc_all["days_to_renewal_snapshot"].notna()

sc_all.loc[mask, "renewal_bucket_snapshot"] = pd.qcut(
    sc_all.loc[mask, "days_to_renewal_snapshot"],
    q=5,
    duplicates="drop"
)


In [120]:
# Interaction frequency by renewal window
sc_all.groupby("renewal_bucket_snapshot")["has_interaction"].agg(
    customers="count",
    interaction_rate="mean"
).sort_index()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/1330752721.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_all.groupby("renewal_bucket_snapshot")["has_interaction"].agg(


,customers,interaction_rate
renewal_bucket_snapshot,,
"(1.999, 106.6]",4020,0.3423
"(106.6, 161.0]",4085,0.3447
"(161.0, 194.0]",4042,0.3073
"(194.0, 234.0]",4014,0.2957
"(234.0, 329.0]",3938,0.3065


In [121]:
# Churn amplification: renewal window × interaction
sc_all.groupby(["renewal_bucket_snapshot", "has_interaction"])["churn"].mean().unstack()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/862551169.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_all.groupby(["renewal_bucket_snapshot", "has_interaction"])["churn"].mean().unstack()


has_interaction,0,1
renewal_bucket_snapshot,,
"(1.999, 106.6]",0.1479,0.3648
"(106.6, 161.0]",0.1061,0.2599
"(161.0, 194.0]",0.0439,0.1039
"(194.0, 234.0]",0.0205,0.0430
"(234.0, 329.0]",0.0172,0.0423


In [122]:
sc_all.groupby(["renewal_bucket_snapshot", "has_interaction"])["customer_id"].count().unstack()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/942651307.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_all.groupby(["renewal_bucket_snapshot", "has_interaction"])["customer_id"].count().unstack()


has_interaction,0,1
renewal_bucket_snapshot,,
"(1.999, 106.6]",2644,1376
"(106.6, 161.0]",2677,1408
"(161.0, 194.0]",2800,1242
"(194.0, 234.0]",2827,1187
"(234.0, 329.0]",2731,1207


For Sentiment Analysis, we used cardiffnlp/twitter-roberta-base-sentiment-latest because it provides a robust, well-validated three-class (positive/neutral/negative) sentiment classification that performs reliably on short, conversational English text similar to our CRM interaction summaries.

In [123]:
import subprocess
subprocess.run(["pip", "install", "transformers", "torch"], check=True)

CompletedProcess(args=['pip', 'install', 'transformers', 'torch'], returncode=0)

In [124]:
# Sentiment analysis

from transformers import pipeline

# 1) Copy working df
sc = silver_customer.copy()

# 2) Text mask
mask_text = sc["interaction_summary"].notna() & sc["interaction_summary"].astype(str).str.strip().ne("")
texts = sc.loc[mask_text, "interaction_summary"].astype(str).tolist()

# 3) Sentiment model pipeline (3-class)
sent_pipe = pipeline(
    "text-classification",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest",
    top_k=None,          # <- returns all labels/scores
    truncation=True
)

# 4) Run inference (you can tune batch_size)
all_scores = sent_pipe(texts, batch_size=32)

# 5) Normalize output to "list of dicts" per text
# Sometimes HF returns: [ [ {...}, {...}, {...} ], ... ]  (expected)
# But sometimes returns: [ {...}, {...}, {...} ] for a single item
if len(texts) == 1 and isinstance(all_scores, list) and len(all_scores) == 3 and isinstance(all_scores[0], dict):
    all_scores = [all_scores]

def scores_to_row(score_list):
    # score_list is list of dicts
    d = {x["label"].lower(): float(x["score"]) for x in score_list}

    # Map label names used by this model:
    # negative / neutral / positive
    neg = d.get("negative", np.nan)
    neu = d.get("neutral", np.nan)
    pos = d.get("positive", np.nan)

    # Argmax label
    label = max([("negative", neg), ("neutral", neu), ("positive", pos)], key=lambda t: t[1])[0]

    return pd.Series({
        "sentiment_neg": neg,
        "sentiment_neu": neu,
        "sentiment_pos": pos,
        "sentiment_label": label
    })

scores_df = pd.DataFrame([scores_to_row(s) for s in all_scores], index=sc.loc[mask_text].index)

# 6) Join back
sc = sc.join(scores_df)

# Optional: fill missing rows (no interaction_summary) with NaNs
silver_customer = sc

# Quick sanity check
silver_customer["sentiment_label"].value_counts(dropna=False)


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: cardiffnlp/twitter-roberta-base-sentiment-latest
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 
roberta.pooler.dense.weight     | UNEXPECTED |  | 
roberta.pooler.dense.bias       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


sentiment_label
NaN         13679
positive     3001
neutral      2740
negative      679
Name: count, dtype: int64

In [125]:
# Validation for Sentiment Analysis
score_cols = ["sentiment_neg", "sentiment_neu", "sentiment_pos"]

# Only rows that actually have scores
scored = silver_customer[silver_customer["sentiment_label"].notna()].copy()

# 1) Check sums
scored["sentiment_sum"] = scored[score_cols].sum(axis=1)

print("Rows with non~1 score sum:",
      (~np.isclose(scored["sentiment_sum"], 1.0, atol=1e-3)).sum())

# 2) Check label matches argmax
argmax_map = {
    "sentiment_neg": "negative",
    "sentiment_neu": "neutral",
    "sentiment_pos": "positive"
}

scored["sentiment_argmax"] = (
    scored[score_cols]
    .idxmax(axis=1)
    .map(argmax_map)
)

print("Rows with label mismatch:",
      (~scored["sentiment_label"].eq(scored["sentiment_argmax"])).sum())


Rows with non~1 score sum: 0
Rows with label mismatch: 0


In [126]:
# Quick spot check

pd.set_option("display.max_colwidth", None)


# Only rows that were scored (non-null sentiment)
sample_df = (
    silver_customer
    .loc[silver_customer["sentiment_label"].notna(),
         ["customer_id",
          "interaction_summary",
          "sentiment_label",
          "sentiment_neg",
          "sentiment_neu",
          "sentiment_pos"]]
    .sample(50, random_state=42)
)

pd.set_option("display.max_colwidth", 200)
sample_df


,customer_id,interaction_summary,sentiment_label,sentiment_neg,sentiment_neu,sentiment_pos
980,C16581,"The customer reached out with general inquiries about their account and upcoming renewal options, expressing satisfaction with their current service. Given their long tenure and stable pricing, th...",positive,0.0061,0.1882,0.8056
4187,C16598,Cust inquired re: renewal options. Not satisfied.,negative,0.8696,0.1206,0.0098
18933,C08853,"The customer inquired about their upcoming renewal, expressing interest in maintaining their current rate without any increases. They mentioned their satisfaction with the service over the past tw...",positive,0.0107,0.3993,0.5900
6425,C05223,"Customer inquired about their upcoming contract renewal and expressed interest in reviewing the current plan options. They seemed satisfied with the existing pricing and service, indicating no imm...",positive,0.0138,0.3418,0.6444
1158,C08259,"The customer inquired about their upcoming renewal, expressing satisfaction with their current plan. As they have been with us for nearly two years, they seemed interested in exploring options but...",neutral,0.0391,0.6429,0.3179
6267,C02246,Cust inquired re: upcoming renewal. Info provided.,neutral,0.0187,0.8935,0.0878
6887,C04792,Cust inquired about services. No price increase.,neutral,0.0461,0.7943,0.1596
11824,C19030,Cust inquired about renewal details. Dissatisfied.,negative,0.8962,0.0970,0.0068
14298,C02512,"Customer inquired about their account details and upcoming renewal options. They expressed satisfaction with the service and indicated no issues with pricing, looking forward to continuing their r...",positive,0.0052,0.1113,0.8835
11629,C03359,Cust inquired about renewal. Dissatisfied with options.,negative,0.8746,0.1176,0.0079


In [127]:
sample_df.to_csv("sentiment_sample_review.csv", index=False)


In [128]:
# Sorted by most negative

silver_customer.loc[
    silver_customer["sentiment_label"].notna(),
    ["interaction_summary",
     "sentiment_label",
     "sentiment_neg",
     "sentiment_neu",
     "sentiment_pos"]
].sort_values("sentiment_neg", ascending=False).head(50)


,interaction_summary,sentiment_label,sentiment_neg,sentiment_neu,sentiment_pos
296,Cust inquired about early contract cancel. Frustrated.,negative,0.9235,0.0704,0.0061
19792,Cust visited office re: service issues. Unhappy.,negative,0.9228,0.0701,0.0072
16320,Cust inq about renewal. Unhappy with service.,negative,0.9221,0.0718,0.0061
8509,Cust inquired about renewal. Misinformed. Unhappy.,negative,0.9218,0.0719,0.0063
14462,Cust inquired about early contract termination. Unhappy.,negative,0.9212,0.0724,0.0064
19541,Cust inquired about early termination. Unhappy.,negative,0.9204,0.0729,0.0067
7166,Cust in store re: account issues. Unhappy.,negative,0.9203,0.0735,0.0062
11385,Cust inquired about contract renewal options. Unhappy.,negative,0.9197,0.0737,0.0066
17089,Cust inquired about account closure. Disappointed.,negative,0.9196,0.0745,0.0059
8811,Cust inquired about renewal offers. Unhappy.,negative,0.9193,0.0733,0.0075


In [129]:
# Sorted by most positive

silver_customer.loc[
    silver_customer["sentiment_label"].notna(),
    ["interaction_summary",
     "sentiment_label",
     "sentiment_neg",
     "sentiment_neu",
     "sentiment_pos"]
].sort_values("sentiment_pos", ascending=False).head(50)


,interaction_summary,sentiment_label,sentiment_neg,sentiment_neu,sentiment_pos
11688,"The customer expressed appreciation for consistent service during their 1.5 years with us, and they are looking forward to their upcoming renewal in a few months. There were no concerns raised abo...",positive,0.0035,0.0275,0.9690
18489,The customer expressed satisfaction with their current plan during the recent visit. They inquired about potential future savings and were pleased to learn about available options ahead of their r...,positive,0.0027,0.0300,0.9673
16217,The customer expressed satisfaction with their current service as they have not seen any price increases this year. They are looking forward to their upcoming renewal and mentioned they appreciate...,positive,0.0036,0.0310,0.9654
19500,"The customer expressed general satisfaction during the call, inquiring about upcoming renewal options and potential benefits available for their account. They appreciated the clarity of the inform...",positive,0.0029,0.0347,0.9624
18459,The customer expressed satisfaction with their current plan and mentioned they have no concerns regarding pricing or competition. They are looking forward to the upcoming renewal opportunity in a ...,positive,0.0029,0.0365,0.9606
5017,Cust inq re: renewal options. Happy. 159 days left.,positive,0.0031,0.0374,0.9596
2962,The customer inquired about their current plan details and was pleased to learn about the upcoming renewal options available to them. They expressed appreciation for our competitive pricing and sh...,positive,0.0027,0.0402,0.9572
6359,"The customer expressed satisfaction during their visit, inquiring about their future energy needs and services. They appreciate the stable pricing and are looking forward to renewing when the time...",positive,0.0032,0.0401,0.9567
19103,"The customer inquired about their upcoming renewal and had a positive experience discussing their energy usage. Overall, they seem satisfied with their service and are looking forward to continuin...",positive,0.0036,0.0417,0.9547
3632,Cust inquired about renewal options. Happy!,positive,0.0037,0.0417,0.9547


In [130]:
# Keep only rows with a sentiment label (i.e., interaction exists + scored)
sc_sent = sc[sc["sentiment_label"].notna()].copy()

# 1) Sentiment vs churn
sc_sent.groupby("sentiment_label")["churn"].agg(
    customers="count",
    churn_rate="mean"
).sort_values("churn_rate", ascending=False)


,customers,churn_rate
sentiment_label,,
negative,679,0.9941
neutral,2740,0.1544
positive,3001,0.0003


In [131]:
# 2) Sentiment distribution by renewal window

sc_sent = sc_sent.merge(
    sc_all[["customer_id", "renewal_bucket_snapshot"]],
    on="customer_id",
    how="left"
)

sc_sent.groupby(
    ["renewal_bucket_snapshot", "sentiment_label"]
)["customer_id"].count().unstack(fill_value=0)


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/2039924611.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_sent.groupby(


sentiment_label,negative,neutral,positive
renewal_bucket_snapshot,,,
"(1.999, 106.6]",306,659,411
"(106.6, 161.0]",217,618,573
"(161.0, 194.0]",83,519,640
"(194.0, 234.0]",37,497,653
"(234.0, 329.0]",36,447,724


In [132]:
# 3) Churn rate by renewal window × sentiment
sc_sent.groupby(
    ["renewal_bucket_snapshot", "sentiment_label"]
)["churn"].mean().unstack()


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/608751846.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_sent.groupby(


sentiment_label,negative,neutral,positive
renewal_bucket_snapshot,,,
"(1.999, 106.6]",1.0000,0.2974,0.0000
"(106.6, 161.0]",0.9954,0.2411,0.0017
"(161.0, 194.0]",1.0000,0.0886,0.0000
"(194.0, 234.0]",0.9730,0.0302,0.0000
"(234.0, 329.0]",0.9444,0.0380,0.0000


In [133]:
# 4) Customers + churn rate in one table
sc_sent.groupby(
    ["renewal_bucket_snapshot", "sentiment_label"]
)["churn"].agg(
    customers="count",
    churn_rate="mean"
).sort_values(["renewal_bucket_snapshot", "churn_rate"], ascending=[True, False])


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/2133182139.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_sent.groupby(


customers  churn_rate
renewal_bucket_snapshot sentiment_label                       
(1.999, 106.6]          negative               306      1.0000
                        neutral                659      0.2974
                        positive               411      0.0000
(106.6, 161.0]          negative               217      0.9954
                        neutral                618      0.2411
                        positive               573      0.0017
(161.0, 194.0]          negative                83      1.0000
                        neutral                519      0.0886
                        positive               640      0.0000
(194.0, 234.0]          negative                37      0.9730
                        neutral                497      0.0302
                        positive               653      0.0000
(234.0, 329.0]          negative                36      0.9444
                        neutral                447      0.0380
                        positive               724      0.0000

In [134]:
sc_chk = sc.copy()

# --- 0) Parse dates safely ---
for c in ["date", "next_renewal_date"]:
    if c in sc_chk.columns:
        sc_chk[c] = pd.to_datetime(sc_chk[c], errors="coerce")

# --- 1) Basic flags: interaction + sentiment present ---
sc_chk["has_interaction"] = (
    sc_chk.get("interaction_summary", pd.Series(index=sc_chk.index, dtype="object"))
      .fillna("").astype(str).str.strip().ne("")
    | sc_chk["date"].notna()
).astype(int)

sc_chk["has_sentiment"] = sc_chk.get("sentiment_label", pd.Series(index=sc_chk.index)).notna().astype(int)

# Keep only rows where interaction exists (for timing checks)
mask_inter = sc_chk["has_interaction"].eq(1) & sc_chk["date"].notna() & sc_chk["next_renewal_date"].notna()

# --- 2) Interaction timing relative to renewal (days) ---
# Positive => interaction BEFORE renewal by that many days
# Negative => interaction AFTER renewal (potential leakage)
sc_chk.loc[mask_inter, "days_to_renewal_at_interaction"] = (
    sc_chk.loc[mask_inter, "next_renewal_date"] - sc_chk.loc[mask_inter, "date"]
).dt.days

# Timing flags
sc_chk["interaction_after_renewal"] = (
    sc_chk["days_to_renewal_at_interaction"].notna() & (sc_chk["days_to_renewal_at_interaction"] < 0)
).astype(int)

sc_chk["interaction_before_renewal"] = (
    sc_chk["days_to_renewal_at_interaction"].notna() & (sc_chk["days_to_renewal_at_interaction"] >= 0)
).astype(int)

# "Safe" pre-event buffers (tune)
sc_chk["pre_event_safe_7d"] = (
    sc_chk["days_to_renewal_at_interaction"].notna() & (sc_chk["days_to_renewal_at_interaction"] >= 7)
).astype(int)

sc_chk["pre_event_safe_30d"] = (
    sc_chk["days_to_renewal_at_interaction"].notna() & (sc_chk["days_to_renewal_at_interaction"] >= 30)
).astype(int)

# --- 3) Leakage check A: how much sentiment occurs after renewal? ---
sent_mask = sc_chk["has_sentiment"].eq(1) & sc_chk["days_to_renewal_at_interaction"].notna()
leakage_a = (
    sc_chk.loc[sent_mask]
      .groupby("sentiment_label")
      .agg(
          customers=("customer_id", "count"),
          pct_after_renewal=("interaction_after_renewal", "mean"),
          median_days_to_renewal=("days_to_renewal_at_interaction", "median"),
      )
      .sort_values("pct_after_renewal", ascending=False)
)
display(leakage_a)

# --- 4) Leakage check B: churn by sentiment (ALL scored rows) ---
sent_only = sc_chk[sc_chk["has_sentiment"].eq(1)].copy()
churn_by_sent_all = (
    sent_only.groupby("sentiment_label")["churn"]
            .agg(customers="count", churn_rate="mean")
            .sort_values("churn_rate", ascending=False)
)
display(churn_by_sent_all)

# --- 5) Leakage check C: churn by sentiment, but ONLY if interaction is strictly BEFORE renewal ---
sent_pre = sent_only[sent_only["interaction_before_renewal"].eq(1)].copy()
churn_by_sent_pre = (
    sent_pre.groupby("sentiment_label")["churn"]
            .agg(customers="count", churn_rate="mean")
            .sort_values("churn_rate", ascending=False)
)
display(churn_by_sent_pre)

# --- 6) Leakage check D: stronger filters (>=7d, >=30d buffer) ---
sent_safe7 = sent_only[sent_only["pre_event_safe_7d"].eq(1)].copy()
sent_safe30 = sent_only[sent_only["pre_event_safe_30d"].eq(1)].copy()

churn_by_sent_safe7 = (
    sent_safe7.groupby("sentiment_label")["churn"]
              .agg(customers="count", churn_rate="mean")
              .sort_values("churn_rate", ascending=False)
)
churn_by_sent_safe30 = (
    sent_safe30.groupby("sentiment_label")["churn"]
               .agg(customers="count", churn_rate="mean")
               .sort_values("churn_rate", ascending=False)
)

display(churn_by_sent_safe7)
display(churn_by_sent_safe30)

# --- 7) Diagnostic table: churn by timing bucket x sentiment (pre-renewal only) ---
# If you already have renewal_bucket_snapshot in sc, this will use it.
if "renewal_bucket_snapshot" in sc_chk.columns:
    tmp = sc_chk[(sc_chk["has_sentiment"].eq(1)) & (sc_chk["interaction_before_renewal"].eq(1))].copy()
    churn_by_bucket_sent = (
        tmp.groupby(["renewal_bucket_snapshot", "sentiment_label"])["churn"]
           .agg(customers="count", churn_rate="mean")
           .reset_index()
           .sort_values(["renewal_bucket_snapshot", "sentiment_label"])
    )
    display(churn_by_bucket_sent)

# --- 8) Quick manual audit: sample suspicious rows (negative sentiment + churners) ---
# Focus on likely leakage: after renewal OR within 0-1 days to renewal
sus = sc_chk[
    (sc_chk["sentiment_label"].eq("negative")) &
    (sc_chk["churn"].eq(1)) &
    (sc_chk["days_to_renewal_at_interaction"].notna()) &
    (
        (sc_chk["days_to_renewal_at_interaction"] < 0) |
        (sc_chk["days_to_renewal_at_interaction"].between(0, 1))
    )
].copy()

cols_show = [c for c in ["customer_id","date","next_renewal_date","days_to_renewal_at_interaction",
                         "sentiment_label","sentiment_neg","sentiment_neu","sentiment_pos",
                         "interaction_summary"] if c in sus.columns]

display(sus[cols_show].sample(n=min(30, len(sus)), random_state=42))

# --- 9) Compact summary: how many scored rows are even eligible (have dates)? ---
summary = pd.Series({
    "rows_total": len(sc_chk),
    "rows_has_sentiment": int(sc_chk["has_sentiment"].sum()),
    "rows_has_sentiment_and_dates": int(sent_mask.sum()),
    "rows_sentiment_after_renewal": int((sc_chk["has_sentiment"].eq(1) & sc_chk["interaction_after_renewal"].eq(1)).sum()),
    "rows_sentiment_pre_renewal": int((sc_chk["has_sentiment"].eq(1) & sc_chk["interaction_before_renewal"].eq(1)).sum()),
    "rows_sentiment_safe_7d": int((sc_chk["has_sentiment"].eq(1) & sc_chk["pre_event_safe_7d"].eq(1)).sum()),
    "rows_sentiment_safe_30d": int((sc_chk["has_sentiment"].eq(1) & sc_chk["pre_event_safe_30d"].eq(1)).sum()),
})
display(summary)


,customers,pct_after_renewal,median_days_to_renewal
sentiment_label,,,
negative,679,0.0000,159.0000
neutral,2740,0.0000,212.0000
positive,3001,0.0000,235.0000


,customers,churn_rate
sentiment_label,,
negative,679,0.9941
neutral,2740,0.1544
positive,3001,0.0003


,customers,churn_rate
sentiment_label,,
negative,679,0.9941
neutral,2740,0.1544
positive,3001,0.0003


,customers,churn_rate
sentiment_label,,
negative,679,0.9941
neutral,2739,0.1544
positive,3001,0.0003


,customers,churn_rate
sentiment_label,,
negative,675,0.9941
neutral,2727,0.1551
positive,2994,0.0003


,customer_id,date,next_renewal_date,days_to_renewal_at_interaction,sentiment_label,sentiment_neg,sentiment_neu,sentiment_pos,interaction_summary


rows_total                      20099
rows_has_sentiment               6420
rows_has_sentiment_and_dates     6420
rows_sentiment_after_renewal        0
rows_sentiment_pre_renewal       6420
rows_sentiment_safe_7d           6419
rows_sentiment_safe_30d          6396
dtype: int64

In [135]:
sc2 = sc.copy()
sc2["date"] = pd.to_datetime(sc2["date"], errors="coerce")
sc2["next_renewal_date"] = pd.to_datetime(sc2["next_renewal_date"], errors="coerce")

# days_to_renewal_at_interaction: positive means interaction BEFORE renewal
sc2["days_to_renewal_at_interaction"] = (sc2["next_renewal_date"] - sc2["date"]).dt.days

neg = sc2[
    (sc2["sentiment_label"] == "negative") &
    sc2["date"].notna() &
    sc2["next_renewal_date"].notna()
].copy()

# "Churn within X days" proxy (since we don't have an actual churn date):
# We interpret it as "interaction occurred within X days BEFORE renewal"
# AND the customer churned.
neg["churn_within_7d"]  = ((neg["days_to_renewal_at_interaction"] >= 0) & (neg["days_to_renewal_at_interaction"] <= 7)  & (neg["churn"] == 1)).astype(int)
neg["churn_within_30d"] = ((neg["days_to_renewal_at_interaction"] >= 0) & (neg["days_to_renewal_at_interaction"] <= 30) & (neg["churn"] == 1)).astype(int)

out = pd.Series({
    "negative_sentiment_customers_with_dates": len(neg),
    "negative_sentiment_churners": int((neg["churn"] == 1).sum()),
    "churn_within_7d_count": int(neg["churn_within_7d"].sum()),
    "churn_within_30d_count": int(neg["churn_within_30d"].sum()),
    "churn_within_7d_pct_of_neg_churners": float(neg.loc[neg["churn"] == 1, "churn_within_7d"].mean()) if (neg["churn"] == 1).any() else np.nan,
    "churn_within_30d_pct_of_neg_churners": float(neg.loc[neg["churn"] == 1, "churn_within_30d"].mean()) if (neg["churn"] == 1).any() else np.nan,
})

out


negative_sentiment_customers_with_dates   679.0000
negative_sentiment_churners               675.0000
churn_within_7d_count                       0.0000
churn_within_30d_count                      4.0000
churn_within_7d_pct_of_neg_churners         0.0000
churn_within_30d_pct_of_neg_churners        0.0059
dtype: float64

In [136]:
sc2.groupby("sentiment_label")["days_to_renewal_at_interaction"].describe()


,count,mean,std,min,25%,50%,75%,max
sentiment_label,,,,,,,,
negative,679.0000,165.3166,62.8765,16.0000,126.0000,159.0000,198.0000,393.0000
neutral,2740.0000,207.7912,74.2815,4.0000,153.0000,212.0000,261.0000,403.0000
positive,3001.0000,230.1973,70.8841,10.0000,185.0000,235.0000,282.0000,406.0000


In [137]:
sc2.groupby("sentiment_label")["days_to_renewal_at_interaction"].median()


sentiment_label
negative   159.0000
neutral    212.0000
positive   235.0000
Name: days_to_renewal_at_interaction, dtype: float64

In [138]:
sc_test = sc2.copy()

# Ensure datetime
sc_test["date"] = pd.to_datetime(sc_test["date"], errors="coerce")
sc_test["next_renewal_date"] = pd.to_datetime(sc_test["next_renewal_date"], errors="coerce")

# Days to renewal at interaction
sc_test["days_to_renewal_at_interaction"] = (
    sc_test["next_renewal_date"] - sc_test["date"]
).dt.days

# Neutral sentiment
neutral = sc_test[
    (sc_test["sentiment_label"] == "neutral") &
    sc_test["date"].notna() &
    sc_test["next_renewal_date"].notna()
].copy()

neutral["churn_within_30d"] = (
    (neutral["days_to_renewal_at_interaction"] >= 0) &
    (neutral["days_to_renewal_at_interaction"] <= 30) &
    (neutral["churn"] == 1)
).astype(int)

# Positive sentiment
positive = sc_test[
    (sc_test["sentiment_label"] == "positive") &
    sc_test["date"].notna() &
    sc_test["next_renewal_date"].notna()
].copy()

positive["churn_within_30d"] = (
    (positive["days_to_renewal_at_interaction"] >= 0) &
    (positive["days_to_renewal_at_interaction"] <= 30) &
    (positive["churn"] == 1)
).astype(int)

# Output summary
out = pd.DataFrame({
    "neutral_customers": [len(neutral)],
    "neutral_churners": [neutral["churn"].sum()],
    "neutral_churn_within_30d_count": [neutral["churn_within_30d"].sum()],
    "neutral_churn_within_30d_pct_of_churners": [
        neutral.loc[neutral["churn"] == 1, "churn_within_30d"].mean()
        if (neutral["churn"] == 1).any() else np.nan
    ],
    "positive_customers": [len(positive)],
    "positive_churners": [positive["churn"].sum()],
    "positive_churn_within_30d_count": [positive["churn_within_30d"].sum()],
    "positive_churn_within_30d_pct_of_churners": [
        positive.loc[positive["churn"] == 1, "churn_within_30d"].mean()
        if (positive["churn"] == 1).any() else np.nan
    ],
})

out


,neutral_customers,neutral_churners,neutral_churn_within_30d_count,neutral_churn_within_30d_pct_of_churners,positive_customers,positive_churners,positive_churn_within_30d_count,positive_churn_within_30d_pct_of_churners
0,2740,423,0,0.0000,3001,1,0,0.0000


In [139]:
import re

# ----------------------------
# Intent rules (update/extend)
# ----------------------------
intent_map = {
    "Cancellation / Switch": [
        r"\bcancel(l?ed|lation)?\b",
        r"\bterminate\b",
        r"\bswitch(ing)?\b",
        r"\bleave\b",
        r"\baccount clos(e|ure)\b",
        r"\bclosing account\b",
    ],

    "Complaint / Escalation": [
        r"\bcomplain(t|ing)?\b",
        r"\bfrustrat(ed|ion)\b",
        r"\bescalat(ed|ion)\b",
        r"\bnot satisfied\b",
        r"\bunsatisfied\b",
        r"\bupset\b",
        r"\bdissatisf(ied|action)\b",
        # PATCH for your remaining examples
        r"\bunhappy\b",
        r"\bdiscontent\b",
    ],

    "Billing / Payment": [
        r"\bbill(ing)?\b",
        r"\bcharge(s|d)?\b",
        r"\bpayment\b",
        r"\boverdue\b",
        r"\bpast due\b",
    ],

    "Contract Renewal": [
        r"\brenew(al|als|ing)?\b",
        r"\bexpir(e|y|ation)\b",
        r"\bnext renewal\b",
        r"\brenewals discussed\b",
        # PATCH for your remaining examples
        r"\brenewed\b",
        r"\brenewed contract\b",
    ],

    "Pricing Offers": [
        r"\bprice(s|d)?\b",
        r"\brate(s)?\b",
        r"\bpricing\b",
        r"\bincrease\b",
        r"\bhike\b",
        r"\bdiscount(s)?\b",
        r"\bsavings?\b",
        r"\bcompetitive\b",
        r"\bbetter (deal|rate|offer)\b",
        # PATCH for your remaining examples
        r"\bcompetitiveness\b",
        r"\bcompetition\b",
        r"\balternatives?\b",
        r"\bseeking alternatives?\b",
        r"\blooking around\b",
    ],

    "Plan / Product Inquiry": [
        r"\bplan(s)?\b",
        r"\bplan options?\b",
        r"\bnew plan\b",
        r"\bfuture plans?\b",
        r"\boptions?\b",
        r"\bexploring options?\b",
    ],

    "Account / Service Inquiry": [
        r"\baccount details?\b",
        r"\baccount questions?\b",
        r"\bservices?\b",
        r"\binquir(y|ed|ies)\b",
        r"\binfo\b",
        r"\bclarified\b",
        r"\bprovided info\b",
        r"\breviewed options\b",
        r"\bdiscussed options\b",
        # PATCH for your remaining examples
        r"\baccount update(s)?\b",
        r"\baccount setup\b",
        r"\bsetup details?\b",
        r"\baccount issues?\b",
        r"\baccount\b.*\bno issues\b",
        r"\bno issues found\b",
        r"\ball good\b",
    ],

    "General / Operational Contact": [
        r"\bgeneral\b",
        r"\bfollow[- ]?up\b",
        r"\broutine\b",
        r"\binbound call\b",
        r"\bno action required\b",
        r"\bissue resolved\b",
        r"\bresolved\b",
        r"\bnormal process\b",
        r"\bstandard interaction\b",
        r"\bcustomer contacted\b",
    ],
}

# Priority order matters (cancel > complaint > billing > renewal > pricing > plan > inquiry > general)
intent_priority = [
    "Cancellation / Switch",
    "Complaint / Escalation",
    "Billing / Payment",
    "Contract Renewal",
    "Pricing Offers",
    "Plan / Product Inquiry",
    "Account / Service Inquiry",
    "General / Operational Contact",
]

# Pre-compile patterns for speed
intent_patterns = {
    k: re.compile("|".join(v), flags=re.IGNORECASE)
    for k, v in intent_map.items()
}

def classify_intent(text):
    t = "" if text is None else str(text)
    for label in intent_priority:
        if intent_patterns[label].search(t):
            return label
    return "Other / Unclassified"

# ----------------------------
# Apply to silver_customer
# ----------------------------
silver_customer["customer_intent"] = silver_customer["interaction_summary"].map(classify_intent)
silver_customer["customer_intent"] = silver_customer["customer_intent"].astype("category")

# Quick QA
print(silver_customer["customer_intent"].value_counts(dropna=False))

print("\nCrosstab with has_interaction:")
print(pd.crosstab(silver_customer["customer_intent"], silver_customer["has_interaction"], margins=True))

print("\nExamples of Other / Unclassified with has_interaction=1:")
display(
    silver_customer.loc[
        (silver_customer["customer_intent"] == "Other / Unclassified") &
        (silver_customer["has_interaction"] == 1),
        ["customer_id", "date", "channel", "interaction_summary"]
    ].head(20)
)


customer_intent
Other / Unclassified         13679
Contract Renewal              5136
Complaint / Escalation         595
Pricing Offers                 230
Account / Service Inquiry      218
Billing / Payment              106
Plan / Product Inquiry          96
Cancellation / Switch           39
Name: count, dtype: int64

Crosstab with has_interaction:
has_interaction                0     1    All
customer_intent                              
Account / Service Inquiry      0   218    218
Billing / Payment              0   106    106
Cancellation / Switch          0    39     39
Complaint / Escalation         0   595    595
Contract Renewal               0  5136   5136
Other / Unclassified       13679     0  13679
Plan / Product Inquiry         0    96     96
Pricing Offers                 0   230    230
All                        13679  6420  20099

Examples of Other / Unclassified with has_interaction=1:


,customer_id,date,channel,interaction_summary


In [140]:
pd.crosstab(
    silver_customer["customer_intent"],
    silver_customer["has_interaction"],
    margins=True
)


has_interaction,0,1,All
customer_intent,,,
Account / Service Inquiry,0,218,218
Billing / Payment,0,106,106
Cancellation / Switch,0,39,39
Complaint / Escalation,0,595,595
Contract Renewal,0,5136,5136
Other / Unclassified,13679,0,13679
Plan / Product Inquiry,0,96,96
Pricing Offers,0,230,230
All,13679,6420,20099


In [141]:
# Extract problematic rows
# Recreate the filtered dataframe (safe in case kernel restarted)
unclassified_with_interaction = silver_customer[
    (silver_customer["customer_intent"] == "Other / Unclassified") &
    (silver_customer["has_interaction"] == 1)
]

# Save to CSV
file_path = "unclassified_with_interaction.csv"
unclassified_with_interaction.to_csv(file_path, index=False)

print(f"Saved {len(unclassified_with_interaction)} rows to {file_path}")


Saved 0 rows to unclassified_with_interaction.csv


In [142]:
# display to spot-check intent classification

silver_customer[silver_customer["has_interaction"] == 1] \
    .sample(20, random_state=42)[
        ["customer_id", "customer_intent", "interaction_summary"]
    ]



,customer_id,customer_intent,interaction_summary
980,C16581,Contract Renewal,"The customer reached out with general inquiries about their account and upcoming renewal options, expressing satisfaction with their current service. Given their long tenure and stable pricing, th..."
4187,C16598,Complaint / Escalation,Cust inquired re: renewal options. Not satisfied.
18933,C08853,Contract Renewal,"The customer inquired about their upcoming renewal, expressing interest in maintaining their current rate without any increases. They mentioned their satisfaction with the service over the past tw..."
6425,C05223,Contract Renewal,"Customer inquired about their upcoming contract renewal and expressed interest in reviewing the current plan options. They seemed satisfied with the existing pricing and service, indicating no imm..."
1158,C08259,Contract Renewal,"The customer inquired about their upcoming renewal, expressing satisfaction with their current plan. As they have been with us for nearly two years, they seemed interested in exploring options but..."
6267,C02246,Contract Renewal,Cust inquired re: upcoming renewal. Info provided.
6887,C04792,Pricing Offers,Cust inquired about services. No price increase.
11824,C19030,Complaint / Escalation,Cust inquired about renewal details. Dissatisfied.
14298,C02512,Contract Renewal,"Customer inquired about their account details and upcoming renewal options. They expressed satisfaction with the service and indicated no issues with pricing, looking forward to continuing their r..."
11629,C03359,Complaint / Escalation,Cust inquired about renewal. Dissatisfied with options.


In [143]:
file_path = "customer_intent_with_interaction_only.csv"

silver_customer[
    silver_customer["has_interaction"] == 1
][
    ["customer_id", "customer_intent", "interaction_summary"]
].to_csv(file_path, index=False)

print(f"Saved to {file_path}")


Saved to customer_intent_with_interaction_only.csv


In [144]:
# churn rate by customer intent

intent_churn = (
    silver_customer
    .groupby("customer_intent")
    .agg(
        customers=("customer_id", "count"),
        churn_rate=("churn", "mean")
    )
    .sort_values("churn_rate", ascending=False)
)

intent_churn


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/1960439148.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("customer_intent")


,customers,churn_rate
customer_intent,,
Complaint / Escalation,595,0.9782
Cancellation / Switch,39,0.5897
Pricing Offers,230,0.2000
Plan / Product Inquiry,96,0.1042
Contract Renewal,5136,0.0820
Other / Unclassified,13679,0.0660
Account / Service Inquiry,218,0.0550
Billing / Payment,106,0.0472


In [145]:
# Churn rate by intent x sentiment

intent_sentiment_churn = (
    silver_customer[silver_customer["has_interaction"] == 1]
    .groupby(["customer_intent", "sentiment_label"])
    .agg(
        customers=("customer_id", "count"),
        churn_rate=("churn", "mean")
    )
    .sort_values("churn_rate", ascending=False)
)

intent_sentiment_churn["churn_rate"] = (
    intent_sentiment_churn["churn_rate"] * 100
).round(2)

intent_sentiment_churn


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/3980896912.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["customer_intent", "sentiment_label"])


,,customers,churn_rate
customer_intent,sentiment_label,,
Account / Service Inquiry,negative,4,100.0000
Pricing Offers,negative,7,100.0000
Cancellation / Switch,negative,10,100.0000
Complaint / Escalation,negative,544,100.0000
Contract Renewal,negative,113,97.3500
Complaint / Escalation,neutral,49,77.5500
Cancellation / Switch,neutral,18,72.2200
Pricing Offers,neutral,136,28.6800
Plan / Product Inquiry,neutral,51,19.6100


In [146]:
# intent x channel churn
intent_channel_churn = (
    silver_customer[silver_customer["has_interaction"] == 1]
    .groupby(["customer_intent", "sales_channel"])
    .agg(
        customers=("customer_id", "count"),
        churn_rate=("churn", "mean")
    )
    .sort_values("churn_rate", ascending=False)
)

intent_channel_churn["churn_rate"] = (
    intent_channel_churn["churn_rate"] * 100
).round(2)

intent_channel_churn


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/353189539.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["customer_intent", "sales_channel"])


customers  churn_rate
customer_intent           sales_channel                              
Complaint / Escalation    Office                       70     98.5700
                          In-Person Commercial        333     97.9000
                          Comparison Website           40     97.5000
                          Unknown                     116     97.4100
                          Telemarketing                36     97.2200
Cancellation / Switch     Comparison Website            4     75.0000
                          In-Person Commercial         20     75.0000
                          Telemarketing                 4     75.0000
                          Office                        2     50.0000
Pricing Offers            Office                       14     28.5700
                          Comparison Website           41     24.3900
                          In-Person Commercial        114     21.9300
Contract Renewal          Own Website                   5     20.0000
Pricing Offers            Telemarketing                11     18.1800
Plan / Product Inquiry    In-Person Commercial         49     16.3300
                          Office                        7     14.2900
                          Telemarketing                 8     12.5000
Cancellation / Switch     Unknown                       9     11.1100
Account / Service Inquiry Comparison Website           10     10.0000
Pricing Offers            Unknown                      50     10.0000
Contract Renewal          In-Person Commercial       2334      9.9800
                          Office                      474      9.2800
Billing / Payment         In-Person Commercial         51      7.8400
Account / Service Inquiry In-Person Commercial         90      7.7800
Contract Renewal          Unknown                    1314      6.8500
                          Telemarketing               321      6.5400
                          Comparison Website          688      4.6500
Account / Service Inquiry Unknown                      74      4.0500
                          Office                       31      3.2300
Billing / Payment         Unknown                      34      2.9400
                          Telemarketing                 5      0.0000
Plan / Product Inquiry    Comparison Website           14      0.0000
Billing / Payment         Office                        4      0.0000
Plan / Product Inquiry    Own Website                   1      0.0000
                          Unknown                      17      0.0000
Billing / Payment         Comparison Website           11      0.0000
Account / Service Inquiry Telemarketing                13      0.0000
Billing / Payment         Own Website                   1      0.0000
Account / Service Inquiry Own Website                   0         NaN
Cancellation / Switch     Own Website                   0         NaN
Complaint / Escalation    Own Website                   0         NaN
Other / Unclassified      Comparison Website            0         NaN
                          In-Person Commercial          0         NaN
                          Office                        0         NaN
                          Own Website                   0         NaN
                          Telemarketing                 0         NaN
                          Unknown                       0         NaN
Pricing Offers            Own Website                   0         NaN

In [147]:
# Chi-square test
from scipy.stats import chi2_contingency

contingency = pd.crosstab(
    silver_customer["customer_intent"],
    silver_customer["churn"]
)

chi2, p, dof, expected = chi2_contingency(contingency)

print("Chi-square statistic:", round(chi2, 2))
print("p-value:", p)


Chi-square statistic: 5448.97
p-value: 0.0


In [149]:
# 1. Create dedicated EDA dataframe (no mutation)

interaction_timing_df = silver_customer.copy()

# Ensure datetime format
interaction_timing_df["date"] = pd.to_datetime(
    interaction_timing_df["date"], errors="coerce"
)

interaction_timing_df["next_renewal_date"] = pd.to_datetime(
    interaction_timing_df["next_renewal_date"], errors="coerce"
)

# 2. Recalculate days to renewal

interaction_timing_df["days_to_renewal_at_interaction"] = (
    interaction_timing_df["next_renewal_date"]
    - interaction_timing_df["date"]
).dt.days

# Keep valid interaction rows before renewal
interaction_timing_df = interaction_timing_df[
    (interaction_timing_df["has_interaction"] == 1) &
    (interaction_timing_df["days_to_renewal_at_interaction"].notna()) &
    (interaction_timing_df["days_to_renewal_at_interaction"] >= 0)
].copy()

interaction_timing_df["days_to_renewal_at_interaction"].describe()


count   6420.0000
mean     213.7726
std       74.2298
min        4.0000
25%      160.0000
50%      217.0000
75%      268.0000
max      406.0000
Name: days_to_renewal_at_interaction, dtype: float64

In [150]:
bins = [0, 30, 90, 180, 365, np.inf]
labels = ["0-30d", "30-90d", "90-180d", "180-365d", "365d+"]

interaction_timing_df["renewal_bucket"] = pd.cut(
    interaction_timing_df["days_to_renewal_at_interaction"],
    bins=bins,
    labels=labels,
    right=False
)


In [151]:
# churn rate by renewal proximity
timing_churn_gradient = (
    interaction_timing_df
    .groupby("renewal_bucket")
    .agg(
        customers=("customer_id", "count"),
        churn_rate=("churn", "mean")
    )
)

timing_churn_gradient["churn_rate"] = (
    timing_churn_gradient["churn_rate"] * 100
).round(2)

timing_churn_gradient


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/3232064261.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("renewal_bucket")


,customers,churn_rate
renewal_bucket,,
0-30d,24,16.6700
30-90d,311,29.2600
90-180d,1772,34.4200
180-365d,4248,9.2500
365d+,65,1.5400


In [152]:
# Timing × Sentiment Gradient
timing_sentiment_gradient = (
    interaction_timing_df
    .groupby(["renewal_bucket", "sentiment_label"])
    .agg(
        customers=("customer_id", "count"),
        churn_rate=("churn", "mean")
    )
)

timing_sentiment_gradient["churn_rate"] = (
    timing_sentiment_gradient["churn_rate"] * 100
).round(2)

timing_sentiment_gradient.sort_values("churn_rate", ascending=False)


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/2234169106.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["renewal_bucket", "sentiment_label"])


,,customers,churn_rate
renewal_bucket,sentiment_label,,
0-30d,negative,4,100.0000
30-90d,negative,63,100.0000
365d+,negative,1,100.0000
90-180d,negative,368,99.7300
180-365d,negative,243,98.7700
90-180d,neutral,824,29.3700
30-90d,neutral,142,19.7200
180-365d,neutral,1740,8.7900
90-180d,positive,580,0.1700


In [153]:
# timing x intent gradient

timing_intent_gradient = (
    interaction_timing_df
    .groupby(["renewal_bucket", "customer_intent"])
    .agg(
        customers=("customer_id", "count"),
        churn_rate=("churn", "mean")
    )
)

timing_intent_gradient["churn_rate"] = (
    timing_intent_gradient["churn_rate"] * 100
).round(2)

timing_intent_gradient.sort_values("churn_rate", ascending=False)



/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/3824761069.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["renewal_bucket", "customer_intent"])


customers  churn_rate
renewal_bucket customer_intent                                 
90-180d        Cancellation / Switch             11    100.0000
365d+          Complaint / Escalation             1    100.0000
0-30d          Pricing Offers                     1    100.0000
               Complaint / Escalation             2    100.0000
30-90d         Cancellation / Switch              3    100.0000
               Pricing Offers                     1    100.0000
90-180d        Complaint / Escalation           304     99.6700
30-90d         Complaint / Escalation            59     96.6100
180-365d       Complaint / Escalation           229     95.6300
90-180d        Pricing Offers                    14     92.8600
               Plan / Product Inquiry             7     85.7100
180-365d       Cancellation / Switch             25     36.0000
30-90d         Account / Service Inquiry          3     33.3300
90-180d        Account / Service Inquiry         10     30.0000
               Contract Renewal                1411     19.3500
180-365d       Pricing Offers                   209     14.8300
30-90d         Contract Renewal                 239     12.1300
90-180d        Billing / Payment                 15      6.6700
0-30d          Contract Renewal                  19      5.2600
180-365d       Billing / Payment                 79      5.0600
               Plan / Product Inquiry            84      4.7600
               Account / Service Inquiry        198      4.0400
               Contract Renewal                3424      3.4500
365d+          Plan / Product Inquiry             5      0.0000
               Contract Renewal                  43      0.0000
0-30d          Billing / Payment                  2      0.0000
365d+          Billing / Payment                  4      0.0000
               Account / Service Inquiry          7      0.0000
30-90d         Billing / Payment                  6      0.0000
365d+          Pricing Offers                     5      0.0000
0-30d          Account / Service Inquiry          0         NaN
               Cancellation / Switch              0         NaN
               Other / Unclassified               0         NaN
               Plan / Product Inquiry             0         NaN
30-90d         Other / Unclassified               0         NaN
               Plan / Product Inquiry             0         NaN
90-180d        Other / Unclassified               0         NaN
180-365d       Other / Unclassified               0         NaN
365d+          Cancellation / Switch              0         NaN
               Other / Unclassified               0         NaN

In [154]:
silver_customer.columns

Index(['customer_id', 'churn', 'contracted_power_kw', 'is_industrial', 'is_second_residence', 'province_code', 'region', 'is_high_competition_province',
       'is_urban', 'customer_first_activation_date', 'contract_start_date', 'last_product_change_date', 'next_renewal_date', 'date', 'channel',
       'interaction_summary', 'segment', 'residential_type', 'sales_channel', 'has_interaction', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos',
       'sentiment_label', 'customer_intent'],
      dtype='object')

In [155]:
# 1. Create clean EDA dataset (no mutation)

sc_comp_dual = silver_customer.merge(
    sc_dual[["customer_id", "is_dual_fuel"]].drop_duplicates(subset="customer_id"),
    on="customer_id",
    how="left"
)


In [156]:
# dual fuel x interaction x churn

dual_interaction = (
    sc_comp_dual
    .groupby(["is_dual_fuel", "has_interaction"])
    .agg(
        customers=("customer_id", "count"),
        churn_rate=("churn", "mean")
    )
    .reset_index()
)

dual_interaction["churn_rate"] = (
    dual_interaction["churn_rate"] * 100
).round(2)

dual_interaction.sort_values("churn_rate", ascending=False)


,is_dual_fuel,has_interaction,customers,churn_rate
1,False,1,5281,17.2500
3,True,1,1139,16.5100
0,False,0,11216,7.1100
2,True,0,2463,4.2600


In [157]:
# dual fuel x intent x churn

dual_intent = (
    sc_comp_dual
    .groupby(["is_dual_fuel", "customer_intent"])
    .agg(
        customers=("customer_id", "count"),
        churn_rate=("churn", "mean")
    )
    .reset_index()
)

dual_intent["churn_rate"] = (
    dual_intent["churn_rate"] * 100
).round(2)

# Remove very small groups (noise)
dual_intent = dual_intent[dual_intent["customers"] >= 30]

dual_intent.sort_values("churn_rate", ascending=False)


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/2568274995.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["is_dual_fuel", "customer_intent"])


,is_dual_fuel,customer_intent,customers,churn_rate
11,True,Complaint / Escalation,93,97.8500
3,False,Complaint / Escalation,502,97.8100
2,False,Cancellation / Switch,34,55.8800
7,False,Pricing Offers,174,21.2600
15,True,Pricing Offers,56,16.0700
9,True,Billing / Payment,30,10.0000
12,True,Contract Renewal,891,8.3100
4,False,Contract Renewal,4245,8.1700
6,False,Plan / Product Inquiry,79,7.5900
5,False,Other / Unclassified,11216,7.1100


In [158]:
# High Competition × Interaction × Churn
comp_interaction = (
    sc_comp_dual
    .groupby(["is_high_competition_province", "has_interaction"])
    .agg(
        customers=("customer_id", "count"),
        churn_rate=("churn", "mean")
    )
    .reset_index()
)

comp_interaction["churn_rate"] = (
    comp_interaction["churn_rate"] * 100
).round(2)

comp_interaction.sort_values("churn_rate", ascending=False)



,is_high_competition_province,has_interaction,customers,churn_rate
3,1,1,1986,24.0200
1,0,1,4434,14.0300
2,1,0,4097,9.4000
0,0,0,9582,5.4100


In [159]:
# high competition x intent x churn

comp_intent = (
    sc_comp_dual
    .groupby(["is_high_competition_province", "customer_intent"])
    .agg(
        customers=("customer_id", "count"),
        churn_rate=("churn", "mean")
    )
    .reset_index()
)

comp_intent["churn_rate"] = (
    comp_intent["churn_rate"] * 100
).round(2)

comp_intent = comp_intent[comp_intent["customers"] >= 30]

comp_intent.sort_values("churn_rate", ascending=False)


/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/2196348794.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["is_high_competition_province", "customer_intent"])


,is_high_competition_province,customer_intent,customers,churn_rate
3,0,Complaint / Escalation,330,99.3900
11,1,Complaint / Escalation,265,95.8500
15,1,Pricing Offers,126,28.5700
14,1,Plan / Product Inquiry,38,15.7900
8,1,Account / Service Inquiry,51,15.6900
12,1,Contract Renewal,1481,11.1400
7,0,Pricing Offers,104,9.6200
13,1,Other / Unclassified,4097,9.4000
4,0,Contract Renewal,3655,7.0000
6,0,Plan / Product Inquiry,58,6.9000


#### 2.2.3 Feature Engineering

In [160]:
# ── Behavioral_Base: Customer universe (one row per customer) ─────────────────
# This is the base DataFrame that all feature tiers merge into.
# Initialized from silver_customer unique customer_ids so every customer
# is represented, even those with no interactions.

Behavioral_Base = (
    silver_customer[["customer_id"]]
    .drop_duplicates("customer_id")
    .reset_index(drop=True)
)

print(f"Behavioral_Base initialized: {len(Behavioral_Base):,} unique customers")

Behavioral_Base initialized: 20,099 unique customers


#### Tier 1 — Lifecycle & Timing Signals

This tier captures **structural contract positioning** and customer maturity within the lifecycle.

- **Renewal Proximity:** Measures distance to next renewal (continuous and bucketed) to capture the contract decision window.

- **Expired Status:** Separates lapsed contracts (negative months_to_renewal) as structurally distinct from imminently renewing customers.

- **Tenure Maturity:** Encodes months since activation and tenure buckets to reflect structural stickiness over time.

- **Renewal × Tenure Structure:** Allows renewal risk to vary by customer maturity stage.

**EDA Linkage:** Churn spikes sharply within 0–3 months of renewal (~25% vs ~6% outside), while longer-tenure customers exhibit materially lower baseline churn (~6% for 5+ years vs ~13% early tenure). Renewal risk is moderated by tenure.

**Model Implications:** Tier 1 features form the structural backbone of the model, capturing the primary churn trigger (renewal) and its moderation by tenure, enabling nonlinear lifecycle separation.

In [161]:
AS_OF_DATE = pd.Timestamp("2024-12-31")

In [162]:
# 1) Lifecycle core 
#    Uses silver_customer columns per your schema:
#    - next_renewal_date
#    - customer_first_activation_date
# -----------------------------
need_cols = ["customer_id", "next_renewal_date", "customer_first_activation_date"]
sc = silver_customer[[c for c in need_cols if c in silver_customer.columns]].copy()

# Parse dates safely
sc["next_renewal_date"] = pd.to_datetime(sc["next_renewal_date"], errors="coerce")
sc["customer_first_activation_date"] = pd.to_datetime(sc["customer_first_activation_date"], errors="coerce")

# months_to_renewal
sc["months_to_renewal"] = (sc["next_renewal_date"] - AS_OF_DATE).dt.days / 30

# is_within_3m_of_renewal (True/False)
sc["is_within_3m_of_renewal"] = sc["months_to_renewal"].between(0, 3)

# renewal_bucket (UPDATED: full coverage incl 12m+ and expired)
sc["renewal_bucket"] = pd.cut(
    sc["months_to_renewal"],
    bins=[-np.inf, 0, 3, 6, 12, np.inf],
    labels=["expired", "0-3m", "3-6m", "6-12m", "12m+"],
)

# tenure_months
sc["tenure_months"] = (AS_OF_DATE - sc["customer_first_activation_date"]).dt.days / 30

# tenure_bucket (your reference bins/labels)
sc["tenure_bucket"] = pd.cut(
    sc["tenure_months"],
    bins=[0, 6, 12, 24, 60, 200],
    labels=["0-6m", "6-12m", "1-2y", "2-5y", "5y+"],
    include_lowest=True
)

lifecycle_feats = sc[
    ["customer_id", "months_to_renewal", "renewal_bucket", "is_within_3m_of_renewal", "tenure_months", "tenure_bucket"]
].drop_duplicates("customer_id")

Behavioral_Base = Behavioral_Base.merge(
    lifecycle_feats,
    on="customer_id",
    how="left"
)

In [163]:
# QA — Tier 1 Lifecycle & Timing

print("AS_OF_DATE:", AS_OF_DATE.date())
print("Behavioral_Base shape:", Behavioral_Base.shape)

print("\nMissing rates (key lifecycle features):")
print(
    Behavioral_Base[["months_to_renewal", "tenure_months"]]
        .isna()
        .mean()
        .round(4)
)

print("\nRenewal bucket distribution (incl NaN):")
print(Behavioral_Base["renewal_bucket"].value_counts(dropna=False))

print("\nTenure bucket distribution (incl NaN):")
print(Behavioral_Base["tenure_bucket"].value_counts(dropna=False))

Behavioral_Base.head()

AS_OF_DATE: 2024-12-31
Behavioral_Base shape: (20099, 6)

Missing rates (key lifecycle features):
months_to_renewal   0.0000
tenure_months       0.0000
dtype: float64

Renewal bucket distribution (incl NaN):
renewal_bucket
3-6m       9212
6-12m      6062
0-3m       4244
expired     581
12m+          0
Name: count, dtype: int64

Tenure bucket distribution (incl NaN):
tenure_bucket
5y+      7157
2-5y     6779
1-2y     2963
0-6m     1604
6-12m    1596
Name: count, dtype: int64


,customer_id,months_to_renewal,renewal_bucket,is_within_3m_of_renewal,tenure_months,tenure_bucket
0,C19280,5.9000,3-6m,False,57.5667,2-5y
1,C15961,7.1667,6-12m,False,150.6667,5y+
2,C13697,6.4000,6-12m,False,69.3000,5y+
3,C03064,4.0667,3-6m,False,34.9000,2-5y
4,C18425,4.1000,3-6m,False,23.7333,1-2y


##### Tier 2 — Behavioral Triggers Near Renewal

This tier captures **behavioral escalation signals** that amplify churn risk around the renewal window.

- **Interaction Presence & Timing**: Uses latest interaction and proximity to renewal to capture behavioral activity during the contract decision window.

- **Intent Severity**: Encodes complaint, cancellation, and pricing-related intents to reflect escalation stage of dissatisfaction.

- **Compound Escalation (Intent × Renewal)**: Flags complaints occurring near renewal, representing the highest-risk churn cluster.

- **Lifecycle Recency**: Measures time since last contract/product change as a secondary signal of renegotiation instability.

**EDA Linkage**: EDA shows churn is renewal-driven (~25% inside 0–3m vs ~6% outside), amplified by interaction, and escalated by complaint/cancellation intent.  
These features operationalize that mechanism:  Renewal Trigger × Behavioral Escalation.

**Model Implications**: Tier 2 features act as high-precision churn amplifiers, especially within renewal windows, and are expected to drive ranking performance rather than broad baseline separation.

In [164]:
AS_OF_DATE = pd.Timestamp("2024-12-31")

# Preconditions
required_cols = [
    "customer_id",
    "customer_intent",
    "date",
    "next_renewal_date",
]

missing = [c for c in required_cols if c not in silver_customer.columns]
if missing:
    raise ValueError(f"silver_customer missing required columns: {missing}")

# Pull intent + dates
sc = silver_customer[["customer_id", "customer_intent", "date", "next_renewal_date"]].copy()

sc["date"] = pd.to_datetime(sc["date"], errors="coerce")
sc["next_renewal_date"] = pd.to_datetime(sc["next_renewal_date"], errors="coerce")

# Keep most recent interaction per customer
sc_sorted = sc.sort_values(["customer_id", "date"], ascending=[True, False])
sc_latest = sc_sorted.drop_duplicates("customer_id")

In [165]:
# Tier 2 — Behavioral Trigger: has_interaction (any interaction observed)

has_interaction = (
    df_interactions[["customer_id"]]
    .drop_duplicates()
    .assign(has_interaction=1)
)

Behavioral_Base = Behavioral_Base.merge(
    has_interaction,
    on="customer_id",
    how="left"
)

Behavioral_Base["has_interaction"] = (
    Behavioral_Base["has_interaction"]
        .fillna(0)
        .astype("Int64")
)

In [166]:
# Tier 2 — Behavioral Triggers: intent + complaint + interaction near renewal (latest interaction)

# 1) customer_intent (latest)
Behavioral_Base = Behavioral_Base.merge(
    sc_latest[["customer_id", "customer_intent"]],
    on="customer_id",
    how="left"
)

# 2) recent_complaint_flag (include cancellation/switch as escalation)
Behavioral_Base["recent_complaint_flag"] = (
    Behavioral_Base["customer_intent"].isin(["Complaint / Escalation", "Cancellation / Switch"])
).astype("Int64")

# 3) interaction_within_3m_of_renewal (based on latest interaction)
tmp = sc_latest[["customer_id", "date", "next_renewal_date"]].copy()

tmp["months_to_renewal_at_interaction"] = (
    (tmp["next_renewal_date"] - tmp["date"]).dt.days / 30.44
)

tmp["interaction_within_3m_of_renewal"] = (
    tmp["months_to_renewal_at_interaction"].between(0, 3, inclusive="both")
).astype("Int64")

# 3b) is_interaction_within_30d_of_renewal — tighter window for high-urgency signal
tmp["is_interaction_within_30d_of_renewal"] = (
    tmp["months_to_renewal_at_interaction"].between(0, 1, inclusive="both")
).astype("Int64")

Behavioral_Base = Behavioral_Base.merge(
    tmp[["customer_id", "interaction_within_3m_of_renewal", "is_interaction_within_30d_of_renewal"]],
    on="customer_id",
    how="left"
)

Behavioral_Base["interaction_within_3m_of_renewal"] = (
    Behavioral_Base["interaction_within_3m_of_renewal"].fillna(0).astype("Int64")
)
Behavioral_Base["is_interaction_within_30d_of_renewal"] = (
    Behavioral_Base["is_interaction_within_30d_of_renewal"].fillna(0).astype("Int64")
)

# Tier 2 amplifier: complaint near renewal
Behavioral_Base["complaint_near_renewal"] = (
    (Behavioral_Base["recent_complaint_flag"] == 1) &
    (Behavioral_Base["interaction_within_3m_of_renewal"] == 1)
).astype("Int64")

# 4) is_cancellation_intent — binary flag for highest-risk intent class
Behavioral_Base["is_cancellation_intent"] = (
    Behavioral_Base["customer_intent"].astype("string") == "Cancellation / Switch"
).astype("Int64")

# 5) is_complaint_intent — binary flag for complaint / escalation
Behavioral_Base["is_complaint_intent"] = (
    Behavioral_Base["customer_intent"].astype("string") == "Complaint / Escalation"
).astype("Int64")

# 6) intent_severity_score — ordinal encoding of intent urgency
#    Cancellation=3, Complaint=2, Pricing Offers=1, everything else=0
_severity_map = {
    "Cancellation / Switch":  3,
    "Complaint / Escalation": 2,
    "Pricing Offers":         1,
}
Behavioral_Base["intent_severity_score"] = (
    Behavioral_Base["customer_intent"]
    .astype("string")
    .map(_severity_map)
    .fillna(0)
    .astype(int)
)

# 7) months_since_last_change — time since last product/contract change
sc_change = silver_customer[["customer_id", "last_product_change_date"]].copy()
sc_change["last_product_change_date"] = pd.to_datetime(
    sc_change["last_product_change_date"], errors="coerce"
)
sc_change["months_since_last_change"] = (
    AS_OF_DATE - sc_change["last_product_change_date"]
).dt.days / 30

Behavioral_Base = Behavioral_Base.merge(
    sc_change[["customer_id", "months_since_last_change"]],
    on="customer_id",
    how="left"
)


In [167]:
# QA — Tier 2 Behavioral Triggers

print("Behavioral_Base shape:", Behavioral_Base.shape)

print("\nIntent distribution:")
print(Behavioral_Base["customer_intent"].value_counts(dropna=False))

print("\nComplaint flag distribution:")
print(Behavioral_Base["recent_complaint_flag"].value_counts(dropna=False))

print("\nInteraction within 3m distribution:")
print(Behavioral_Base["interaction_within_3m_of_renewal"].value_counts(dropna=False))

print("\nComplaint near renewal distribution:")
print(Behavioral_Base["complaint_near_renewal"].value_counts(dropna=False))

Behavioral_Base.head()

Behavioral_Base shape: (20099, 16)

Intent distribution:
customer_intent
Other / Unclassified         13679
Contract Renewal              5136
Complaint / Escalation         595
Pricing Offers                 230
Account / Service Inquiry      218
Billing / Payment              106
Plan / Product Inquiry          96
Cancellation / Switch           39
Name: count, dtype: int64

Complaint flag distribution:
recent_complaint_flag
0    19465
1      634
Name: count, dtype: Int64

Interaction within 3m distribution:
interaction_within_3m_of_renewal
0    19738
1      361
Name: count, dtype: Int64

Complaint near renewal distribution:
complaint_near_renewal
0    20031
1       68
Name: count, dtype: Int64


,customer_id,months_to_renewal,renewal_bucket,is_within_3m_of_renewal,tenure_months,tenure_bucket,has_interaction,customer_intent,recent_complaint_flag,interaction_within_3m_of_renewal,is_interaction_within_30d_of_renewal,complaint_near_renewal,is_cancellation_intent,is_complaint_intent,intent_severity_score,months_since_last_change
0,C19280,5.9000,3-6m,False,57.5667,2-5y,0,Other / Unclassified,0,0,0,0,0,0,0,10.5000
1,C15961,7.1667,6-12m,False,150.6667,5y+,1,Contract Renewal,0,0,0,0,0,0,0,NaN
2,C13697,6.4000,6-12m,False,69.3000,5y+,0,Other / Unclassified,0,0,0,0,0,0,0,9.7000
3,C03064,4.0667,3-6m,False,34.9000,2-5y,0,Other / Unclassified,0,0,0,0,0,0,0,NaN
4,C18425,4.1000,3-6m,False,23.7333,1-2y,0,Other / Unclassified,0,0,0,0,0,0,0,6.3000


#### Tier 3 — Sentiment & Interaction Effects

This tier captures **nonlinear behavioral amplification effects** between sentiment, intent, renewal timing, and market context.

- **Sentiment Signals:** Incorporates NLP-derived sentiment from the latest interaction to capture emotional escalation.

- **Sentiment × Lifecycle Interactions:** Combines sentiment with renewal timing and intent to model compounded risk during decision windows.

- **Behavioral Interaction Strings:** Encodes intent × renewal and intent × tenure combinations to allow tree models to learn high-risk joint splits.

- **Contextual Amplifiers:** Adds interaction recency, competition effects, and dual-fuel conditioning to reflect market-driven escalation.

**EDA Linkage:** Negative sentiment approaches deterministic churn, especially when paired with complaint intent or occurring near renewal. Interaction and competition further amplify renewal-stage switching behavior.

**Model Implications:** Tier 3 features capture high-order nonlinear effects and are expected to enhance tree-based models after Tier 1–2 structural signals establish baseline separation.

In [168]:
# Tier 3 — Sentiment signals (customer-level)

# Preconditions
required_cols = ["customer_id", "sentiment_label"]
missing = [c for c in required_cols if c not in silver_customer.columns]
if missing:
    raise ValueError(f"silver_customer missing required columns: {missing}")

# Pull one sentiment row per customer (latest by date if available)
sent_cols = ["customer_id", "sentiment_label"]
if "date" in silver_customer.columns:
    sc_sent = silver_customer[sent_cols + ["date"]].copy()
    sc_sent["date"] = pd.to_datetime(sc_sent["date"], errors="coerce")
    sc_sent = (
        sc_sent.sort_values(["customer_id", "date"], ascending=[True, False])
              .drop_duplicates("customer_id")
              .drop(columns=["date"])
    )
else:
    sc_sent = silver_customer[sent_cols].drop_duplicates("customer_id").copy()

Behavioral_Base = Behavioral_Base.merge(sc_sent, on="customer_id", how="left")

Behavioral_Base["sentiment_label"] = Behavioral_Base["sentiment_label"].astype("string")

# is_negative_sentiment — binary flag (highest churn predictor per EDA)
Behavioral_Base["is_negative_sentiment"] = (
    Behavioral_Base["sentiment_label"] == "negative"
).astype("Int64")

# complaint_x_negative_sentiment — compound highest-risk signal
# EDA: complaint near renewal + negative sentiment → near-100% churn
Behavioral_Base["complaint_x_negative_sentiment"] = (
    (Behavioral_Base["recent_complaint_flag"] == 1) &
    (Behavioral_Base["is_negative_sentiment"] == 1)
).astype("Int64")


In [169]:
# Tier 3 — Sentiment interaction effects

# Ensure safe string conversion
Behavioral_Base["sentiment_label"] = Behavioral_Base["sentiment_label"].astype("string")
Behavioral_Base["customer_intent"] = Behavioral_Base["customer_intent"].astype("string")
Behavioral_Base["renewal_bucket"] = Behavioral_Base["renewal_bucket"].astype("string")

# 1) sentiment × renewal_bucket
Behavioral_Base["sentiment_x_renewal_bucket"] = (
    Behavioral_Base["sentiment_label"].fillna("Unknown")
    + "_x_"
    + Behavioral_Base["renewal_bucket"].fillna("Unknown")
)

# 2) intent × sentiment
Behavioral_Base["intent_x_sentiment"] = (
    Behavioral_Base["customer_intent"].fillna("Unknown")
    + "_x_"
    + Behavioral_Base["sentiment_label"].fillna("Unknown")
)

In [170]:
# Tier 3 — Behavioral interaction effects + recency (behavioral-only)

def make_interaction_safe(df, a, b):
    return (
        df[a].astype("string").fillna("Unknown")
        + "_x_"
        + df[b].astype("string").fillna("Unknown")
    )

# 1) intent × renewal_bucket
Behavioral_Base["intent_x_renewal_bucket"] = make_interaction_safe(
    Behavioral_Base, "customer_intent", "renewal_bucket"
)

# 2) intent × tenure_bucket
Behavioral_Base["intent_x_tenure_bucket"] = make_interaction_safe(
    Behavioral_Base, "customer_intent", "tenure_bucket"
)

# 3) tenure × renewal_bucket — EDA: churn gradient 16–32% by tenure at renewal
Behavioral_Base["tenure_x_renewal_bucket"] = make_interaction_safe(
    Behavioral_Base, "tenure_bucket", "renewal_bucket"
)

# 4) is_price_sensitive (intent-based)
Behavioral_Base["is_price_sensitive"] = (
    Behavioral_Base["customer_intent"].astype("string") == "Pricing Offers"
).astype("Int64")

# 5) last_interaction_days_ago (using latest interaction date from sc_latest)
last_dt = sc_latest[["customer_id", "date"]].copy()
last_dt["date"] = pd.to_datetime(last_dt["date"], errors="coerce")

Behavioral_Base = Behavioral_Base.merge(last_dt, on="customer_id", how="left")

Behavioral_Base["last_interaction_days_ago"] = (
    AS_OF_DATE - pd.to_datetime(Behavioral_Base["date"], errors="coerce")
).dt.days

# Drop raw date after recency feature is created
Behavioral_Base = Behavioral_Base.drop(columns=["date"], errors="ignore")

# ── Lifecycle compound signals ────────────────────────────────────────────────
# These are pre-computed interaction flags for tree models to use as shortcuts.

# 6) is_high_risk_lifecycle — short tenure + near/at renewal (highest churn gradient)
Behavioral_Base["is_high_risk_lifecycle"] = (
    Behavioral_Base["renewal_bucket"].astype("string").isin(["expired", "0-3m"]) &
    Behavioral_Base["tenure_bucket"].astype("string").isin(["0-6m", "6-12m"])
).astype("Int64")

# 7) is_competition_x_renewal — competition province amplifies renewal risk
# Requires is_high_competition_province from silver_customer
_comp = silver_customer[["customer_id", "is_high_competition_province"]].copy()
Behavioral_Base = Behavioral_Base.merge(_comp, on="customer_id", how="left")

Behavioral_Base["is_competition_x_renewal"] = (
    (Behavioral_Base["is_high_competition_province"] == 1) &
    (Behavioral_Base["is_within_3m_of_renewal"] == True)
).astype("Int64")

# Drop is_high_competition_province — it will be merged in from silver_customer in gold build
Behavioral_Base = Behavioral_Base.drop(columns=["is_high_competition_province"], errors="ignore")

# 8) dual_fuel_x_renewal — dual fuel customers less likely to churn at renewal, but flag the ones that do
# Requires is_dual_fuel — merge from Tier1A_Features if available
# 8) dual_fuel_x_renewal — dual fuel customers less likely to churn at renewal, but flag the ones that do
# Compute is_dual_fuel inline (Tier1A_Features not yet defined at this stage)
_dual_inline = (
    silver_customer_month
    .groupby("customer_id", as_index=False)
    .agg(total_gas=("monthly_gas_m3", "sum"))
)
_dual_inline["is_dual_fuel"] = (_dual_inline["total_gas"] > 0).astype("Int64")

Behavioral_Base = Behavioral_Base.merge(
    _dual_inline[["customer_id", "is_dual_fuel"]], on="customer_id", how="left"
)

Behavioral_Base["dual_fuel_x_renewal"] = (
    (Behavioral_Base["is_dual_fuel"] == 1) &
    (Behavioral_Base["is_within_3m_of_renewal"] == True)
).astype("Int64")

Behavioral_Base = Behavioral_Base.drop(columns=["is_dual_fuel"], errors="ignore")


# 8) has_interaction_x_renewal_bucket — interaction amplifier at renewal (EDA s4)
Behavioral_Base["has_interaction_x_renewal_bucket"] = (
    Behavioral_Base["has_interaction"].astype("string").fillna("0")
    + "_x_"
    + Behavioral_Base["renewal_bucket"].astype("string").fillna("Unknown")
)

# 9) competition_x_intent — competition amplifies complaint/cancellation risk (EDA s7)
# is_high_competition_province may not be in Behavioral_Base yet; pull from silver_customer
if "is_high_competition_province" not in Behavioral_Base.columns:
    _comp_lookup = silver_customer[["customer_id","is_high_competition_province"]].drop_duplicates("customer_id")
    Behavioral_Base = Behavioral_Base.merge(_comp_lookup, on="customer_id", how="left")

Behavioral_Base["competition_x_intent"] = (
    Behavioral_Base["is_high_competition_province"].astype("string").fillna("0")
    + "_x_"
    + Behavioral_Base["customer_intent"].astype("string").fillna("Unknown")
)

# 10) dual_fuel_x_competition — bundling buffer weakens in competitive regions (EDA s9)
if "is_dual_fuel" not in Behavioral_Base.columns:
    _dual_lookup = silver_customer[["customer_id"]].drop_duplicates()
    _gas_flag = (
        silver_customer_month
        .groupby("customer_id")["monthly_gas_m3"]
        .sum()
        .reset_index()
    )
    _gas_flag["is_dual_fuel"] = (_gas_flag["monthly_gas_m3"] > 0).astype("Int64")
    _dual_lookup = _dual_lookup.merge(_gas_flag[["customer_id","is_dual_fuel"]], on="customer_id", how="left")
    Behavioral_Base = Behavioral_Base.merge(_dual_lookup, on="customer_id", how="left")

Behavioral_Base["dual_fuel_x_competition"] = (
    (Behavioral_Base["is_dual_fuel"] == 1) &
    (Behavioral_Base["is_high_competition_province"] == True)
).astype("Int64")

# 11) dual_fuel_x_intent — bundled customers with high-risk intent (EDA s9)
Behavioral_Base["dual_fuel_x_intent"] = (
    (Behavioral_Base["is_dual_fuel"] == 1) &
    (Behavioral_Base["customer_intent"].isin(["Cancellation / Switch","Complaint / Escalation"]))
).astype("Int64")

In [171]:
# QA — Tier 3 Sentiment & Interaction Effects

print("Behavioral_Base shape:", Behavioral_Base.shape)

print("\nSentiment distribution:")
print(Behavioral_Base["sentiment_label"].value_counts(dropna=False))

print("\nTop sentiment_x_renewal_bucket:")
print(
    Behavioral_Base["sentiment_x_renewal_bucket"]
        .value_counts(dropna=False)
        .head()
)

print("\nTop intent_x_sentiment:")
print(
    Behavioral_Base["intent_x_sentiment"]
        .value_counts(dropna=False)
        .head()
)

Behavioral_Base.head()

Behavioral_Base shape: (20099, 35)

Sentiment distribution:
sentiment_label
<NA>        13679
positive     3001
neutral      2740
negative      679
Name: count, dtype: Int64

Top sentiment_x_renewal_bucket:
sentiment_x_renewal_bucket
Unknown_x_3-6m     6322
Unknown_x_6-12m    4221
Unknown_x_0-3m     2730
positive_x_3-6m    1408
neutral_x_3-6m     1227
Name: count, dtype: Int64

Top intent_x_sentiment:
intent_x_sentiment
Other / Unclassified_x_Unknown       13679
Contract Renewal_x_positive           2707
Contract Renewal_x_neutral            2316
Complaint / Escalation_x_negative      544
Pricing Offers_x_neutral               136
Name: count, dtype: Int64


,customer_id,months_to_renewal,renewal_bucket,is_within_3m_of_renewal,tenure_months,tenure_bucket,has_interaction,customer_intent,recent_complaint_flag,interaction_within_3m_of_renewal,is_interaction_within_30d_of_renewal,complaint_near_renewal,is_cancellation_intent,is_complaint_intent,intent_severity_score,months_since_last_change,sentiment_label,is_negative_sentiment,complaint_x_negative_sentiment,sentiment_x_renewal_bucket,intent_x_sentiment,intent_x_renewal_bucket,intent_x_tenure_bucket,tenure_x_renewal_bucket,is_price_sensitive,last_interaction_days_ago,is_high_risk_lifecycle,is_competition_x_renewal,dual_fuel_x_renewal,has_interaction_x_renewal_bucket,is_high_competition_province,competition_x_intent,is_dual_fuel,dual_fuel_x_competition,dual_fuel_x_intent
0,C19280,5.9000,3-6m,False,57.5667,2-5y,0,Other / Unclassified,0,0,0,0,0,0,0,10.5000,<NA>,<NA>,0,Unknown_x_3-6m,Other / Unclassified_x_Unknown,Other / Unclassified_x_3-6m,Other / Unclassified_x_2-5y,2-5y_x_3-6m,0,NaN,0,0,0,0_x_3-6m,0,0_x_Other / Unclassified,0,0,0
1,C15961,7.1667,6-12m,False,150.6667,5y+,1,Contract Renewal,0,0,0,0,0,0,0,NaN,positive,0,0,positive_x_6-12m,Contract Renewal_x_positive,Contract Renewal_x_6-12m,Contract Renewal_x_5y+,5y+_x_6-12m,0,118.0000,0,0,0,1_x_6-12m,0,0_x_Contract Renewal,1,0,0
2,C13697,6.4000,6-12m,False,69.3000,5y+,0,Other / Unclassified,0,0,0,0,0,0,0,9.7000,<NA>,<NA>,0,Unknown_x_6-12m,Other / Unclassified_x_Unknown,Other / Unclassified_x_6-12m,Other / Unclassified_x_5y+,5y+_x_6-12m,0,NaN,0,0,0,0_x_6-12m,1,1_x_Other / Unclassified,1,1,0
3,C03064,4.0667,3-6m,False,34.9000,2-5y,0,Other / Unclassified,0,0,0,0,0,0,0,NaN,<NA>,<NA>,0,Unknown_x_3-6m,Other / Unclassified_x_Unknown,Other / Unclassified_x_3-6m,Other / Unclassified_x_2-5y,2-5y_x_3-6m,0,NaN,0,0,0,0_x_3-6m,1,1_x_Other / Unclassified,0,0,0
4,C18425,4.1000,3-6m,False,23.7333,1-2y,0,Other / Unclassified,0,0,0,0,0,0,0,6.3000,<NA>,<NA>,0,Unknown_x_3-6m,Other / Unclassified_x_Unknown,Other / Unclassified_x_3-6m,Other / Unclassified_x_1-2y,1-2y_x_3-6m,0,NaN,0,0,0,0_x_3-6m,0,0_x_Other / Unclassified,0,0,0


## Behavioral Intelligence — Feature Summary

All behavioral, lifecycle, sentiment, and interaction features have been built and merged into `Behavioral_Base`. This DataFrame feeds directly into the gold master table.

| Tier | Feature | Type | Signal |
|------|---------|------|--------|
| 1 — Lifecycle | `months_to_renewal` | Continuous | Days-to-renewal scaled to months |
| 1 — Lifecycle | `renewal_bucket` | Ordinal | 5-level bin: expired / 0-3m / 3-6m / 6-12m / 12m+ |
| 1 — Lifecycle | `is_within_3m_of_renewal` | Binary | Primary renewal risk flag |
| 1 — Lifecycle | `tenure_months` | Continuous | Customer relationship length |
| 1 — Lifecycle | `tenure_bucket` | Ordinal | 5-level maturity bin |
| 2 — Behavioral | `has_interaction` | Binary | Any interaction observed in 2024 |
| 2 — Behavioral | `customer_intent` | Categorical | Highest churn rate by category (Cancellation ~99%) |
| 2 — Behavioral | `recent_complaint_flag` | Binary | Combines Complaint + Cancellation intents |
| 2 — Behavioral | `interaction_within_3m_of_renewal` | Binary | Interaction close to renewal decision window |
| 2 — Behavioral | `is_interaction_within_30d_of_renewal` | Binary | High-urgency tighter window |
| 2 — Behavioral | `complaint_near_renewal` | Binary | Amplifier: complaint AND near renewal |
| 2 — Behavioral | `is_cancellation_intent` | Binary | Highest-risk intent class |
| 2 — Behavioral | `is_complaint_intent` | Binary | Complaint / escalation flag |
| 2 — Behavioral | `intent_severity_score` | Ordinal | Urgency: Cancellation=3, Complaint=2, Pricing=1 |
| 2 — Behavioral | `months_since_last_change` | Continuous | Time since last product/contract change |
| 3 — Sentiment & Interactions | `sentiment_label` | Categorical | Emotional tone of last interaction |
| 3 — Sentiment & Interactions | `is_negative_sentiment` | Binary | Negative sentiment flag (near-deterministic churn) |
| 3 — Sentiment & Interactions | `complaint_x_negative_sentiment` | Binary | Compound: complaint + negative sentiment |
| 3 — Sentiment & Interactions | `sentiment_x_renewal_bucket` | String interaction | Sentiment × lifecycle timing |
| 3 — Sentiment & Interactions | `intent_x_sentiment` | String interaction | Intent × emotional tone |
| 3 — Sentiment & Interactions | `intent_x_renewal_bucket` | String interaction | Intent × renewal proximity |
| 3 — Sentiment & Interactions | `intent_x_tenure_bucket` | String interaction | Intent × customer maturity |
| 3 — Sentiment & Interactions | `tenure_x_renewal_bucket` | String interaction | Tenure × renewal proximity |
| 3 — Sentiment & Interactions | `is_price_sensitive` | Binary | Pricing-driven intent flag |
| 3 — Sentiment & Interactions | `last_interaction_days_ago` | Continuous | Recency of customer contact |
| 3 — Compound | `is_high_risk_lifecycle` | Binary | Short tenure + near renewal |
| 3 — Compound | `is_competition_x_renewal` | Binary | Competition province + near renewal |
| 3 — Compound | `dual_fuel_x_renewal` | Binary | Dual fuel customer near renewal |
| 3 — Compound | `has_interaction_x_renewal_bucket` | String interaction | Interaction × renewal bucket |
| 3 — Compound | `competition_x_intent` | String interaction | Competition × intent |
| 3 — Compound | `dual_fuel_x_competition` | Binary | Dual fuel in competitive province |
| 3 — Compound | `dual_fuel_x_intent` | Binary | Dual fuel with high-risk intent |

**Modeling tier mapping** (for experiment ladder in modeling notebook):

| Gold tier | Modeling tier | Purpose |
|-----------|--------------|---------|
| Tier 1 — Lifecycle | `TIER_1A` (partial: lifecycle columns only) | Structural backbone |
| Structural (from silver_customer) | `TIER_1A` (stickiness columns) | Channel, segment, competition |
| Tier 2 — Behavioral | `TIER_2A` | Behavioral amplifiers |
| Tier 3 — Sentiment | `TIER_2B` | Sentiment signals |
| Tier 3 — Interactions | `TIER_3` | Context modulators |
| Tier 3 — Compound | `TIER_LIFECYCLE_INTERACTIONS` | Compound lifecycle flags |
| Tier 3 — Interactions (linear) | `TIER_1B` | Pre-computed strings for linear models |


#### TierMP_Core — Market & Portfolio: Core Structure

**What the code does:** Aggregates consumption, revenue, channel, portfolio, and pricing structure features from `silver_customer_month`, `silver_customer`, `costs_by_province_month.csv`, and `price_history.csv`. All inputs are filtered to ≤ 2024-12-31 to prevent data leakage.

| # | Feature | Description | Source |
|---|---------|-------------|--------|
| 1 | `avg_monthly_elec_kwh` | Mean monthly electricity consumption | silver_customer_month |
| 2 | `total_elec_kwh_2024` | Total electricity consumed in 2024 | silver_customer_month |
| 3 | `avg_monthly_gas_m3` | Mean monthly gas consumption (0 for elec-only) | silver_customer_month |
| 4 | `total_gas_m3_2024` | Total gas consumed in 2024 | silver_customer_month |
| 5 | `avg_monthly_margin` | Mean monthly customer margin (revenue − cost) | silver_customer_month |
| 6 | `total_margin_2024` | Summed margin for full 2024 | silver_customer_month |
| 7 | `is_digital_channel` | 1 if acquired via Comparison Website or Own Website | silver_customer |
| 8 | `is_dual_fuel` | 1 if customer has any gas consumption in 2024 | silver_customer_month |
| 9 | `portfolio_type` | segment × dual-fuel label (e.g. "Residential_DualFuel") | silver_customer + monthly |
| 10 | `gas_share_of_revenue` | Gas revenue as fraction of total revenue | silver_customer_month |
| 11 | `province_avg_elec_cost_2024` | Province-level mean electricity variable cost | costs_by_province_month |
| 12 | `province_avg_gas_cost_2024` | Province-level mean gas variable cost | costs_by_province_month |
| 13 | `price_update_count` | Number of distinct pricing dates applied in 2024 | price_history |

**Note on margin features:** `avg_monthly_margin` and `total_margin_2024` are retained in the gold master table for **retention prioritisation** (expected value at risk = P(churn) × margin). EDA showed these features have no discriminative power for predicting *who churns* (median: €6.96 non-churners vs €7.01 churners). They will be **excluded from model training** in the modeling notebook; inclusion or exclusion will be tested in the experiment ladder.

**Model implication:** Consumption features and `portfolio_type` are expected to be moderate predictors. `is_digital_channel` captures acquisition economics that correlate with price-sensitivity. `price_update_count` serves as a proxy for pricing volatility exposure.

In [172]:
silver_customer.columns

Index(['customer_id', 'churn', 'contracted_power_kw', 'is_industrial', 'is_second_residence', 'province_code', 'region', 'is_high_competition_province',
       'is_urban', 'customer_first_activation_date', 'contract_start_date', 'last_product_change_date', 'next_renewal_date', 'date', 'channel',
       'interaction_summary', 'segment', 'residential_type', 'sales_channel', 'has_interaction', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos',
       'sentiment_label', 'customer_intent'],
      dtype='object')

In [173]:
# ── Tier1A_Features — lifecycle + stickiness + dual fuel ─────────────────────
# This is the canonical customer universe used by TierMP_Risk and the gold build.
# It mirrors the structure from the reference notebook.

Tier1A_Features = (
    silver_customer[["customer_id"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

# Lifecycle: months_to_renewal, renewal_bucket, tenure_months, tenure_bucket
# (already computed in Behavioral_Base — merge from there)
lifecycle_cols = ["customer_id", "months_to_renewal", "renewal_bucket",
                  "is_within_3m_of_renewal", "tenure_months", "tenure_bucket"]
lifecycle_cols = [c for c in lifecycle_cols if c in Behavioral_Base.columns]

Tier1A_Features = Tier1A_Features.merge(
    Behavioral_Base[lifecycle_cols].drop_duplicates("customer_id"),
    on="customer_id", how="left"
)

# Structural stickiness from silver_customer
stickiness_cols = ["segment", "sales_channel", "is_high_competition_province",
                   "has_interaction", "is_second_residence"]
stickiness_cols = [c for c in stickiness_cols if c in silver_customer.columns]
Tier1A_Features = Tier1A_Features.merge(
    silver_customer[["customer_id"] + stickiness_cols].drop_duplicates("customer_id"),
    on="customer_id", how="left"
)

# has_interaction from Behavioral_Base if not in silver_customer
if "has_interaction" not in Tier1A_Features.columns and "has_interaction" in Behavioral_Base.columns:
    Tier1A_Features = Tier1A_Features.merge(
        Behavioral_Base[["customer_id", "has_interaction"]].drop_duplicates("customer_id"),
        on="customer_id", how="left"
    )


# is_expired_contract — derived from renewal_bucket (already in Tier1A_Features)
Tier1A_Features["is_expired_contract"] = (
    Tier1A_Features["renewal_bucket"].astype("string") == "expired"
).astype("Int64")

# is_comparison_channel / is_own_website_channel
# sales_channel is already merged in via the stickiness_cols block above
Tier1A_Features["is_comparison_channel"] = (
    Tier1A_Features["sales_channel"].astype("string") == "Comparison Website"
).astype("Int64")
Tier1A_Features["is_own_website_channel"] = (
    Tier1A_Features["sales_channel"].astype("string") == "Own Website"
).astype("Int64")

# is_dual_fuel (derived from monthly consumption)
_fuel = (
    silver_customer_month
    .groupby("customer_id", as_index=False)
    .agg(total_elec=("monthly_elec_kwh", "sum"), total_gas=("monthly_gas_m3", "sum"))
)
_fuel["is_dual_fuel"] = ((_fuel["total_elec"] > 0) & (_fuel["total_gas"] > 0)).astype("Int64")
Tier1A_Features = Tier1A_Features.merge(_fuel[["customer_id", "is_dual_fuel"]], on="customer_id", how="left")
Tier1A_Features["is_dual_fuel"] = Tier1A_Features["is_dual_fuel"].fillna(0).astype("Int64")

print("Tier1A_Features shape:", Tier1A_Features.shape)
print("Unique customers:", Tier1A_Features["customer_id"].nunique())
Tier1A_Features.head()

# sales_channel_x_renewal_bucket — channel stickiness at renewal (EDA s8)
Tier1A_Features["sales_channel_x_renewal_bucket"] = (
    Tier1A_Features["sales_channel"].astype("string").fillna("Unknown")
    + "_x_"
    + Tier1A_Features["renewal_bucket"].astype("string").fillna("Unknown")
)

Tier1A_Features shape: (20099, 15)
Unique customers: 20099


In [174]:
AS_OF_DATE = pd.Timestamp("2024-12-31")

# ── Preconditions ─────────────────────────────────────────
_req_monthly = ["customer_id", "month", "monthly_elec_kwh", "monthly_gas_m3", "total_margin"]
_miss_monthly = [c for c in _req_monthly if c not in silver_customer_month.columns]
if _miss_monthly:
    raise ValueError(f"silver_customer_month missing columns: {_miss_monthly}")

_req_sc = ["customer_id", "sales_channel", "segment", "province_code"]
_miss_sc = [c for c in _req_sc if c not in silver_customer.columns]
if _miss_sc:
    raise ValueError(f"silver_customer missing columns: {_miss_sc}")

_req_rev = ["customer_id", "total_revenue", "gas_revenue_variable", "gas_revenue_fixed"]
_miss_rev = [c for c in _req_rev if c not in silver_customer_month.columns]

# ── Ensure month is datetime, then filter to cutoff ────────
scm = silver_customer_month.copy()

_m = scm["month"].astype(str).str.strip()
_mask_ym = _m.str.match(r"^\d{4}-\d{2}$")
_m.loc[_mask_ym] = _m.loc[_mask_ym] + "-01"
scm["month"] = pd.to_datetime(_m, errors="coerce")

_bad = scm["month"].isna()
if _bad.any():
    print("WARNING: Unparseable month values (showing up to 10):")
    print(silver_customer_month.loc[_bad, "month"].head(10).to_list())

dfm = scm[scm["month"] <= AS_OF_DATE].copy()

for _c in ["monthly_elec_kwh", "monthly_gas_m3", "total_margin"]:
    dfm[_c] = pd.to_numeric(dfm[_c], errors="coerce").fillna(0)

# ── A1–A4: Consumption basics ─────────────────────────────
_cons = (
    dfm.groupby("customer_id", as_index=False)
    .agg(
        avg_monthly_elec_kwh=("monthly_elec_kwh", "mean"),
        total_elec_kwh_2024=("monthly_elec_kwh", "sum"),
        avg_monthly_gas_m3=("monthly_gas_m3", "mean"),
        total_gas_m3_2024=("monthly_gas_m3", "sum"),
    )
)

# ── A5–A6: Margin level ───────────────────────────────────
_marg = (
    dfm.groupby("customer_id", as_index=False)
    .agg(
        avg_monthly_margin=("total_margin", "mean"),
        total_margin_2024=("total_margin", "sum"),
    )
)

# ── A7: is_digital_channel ────────────────────────────────
DIGITAL_CHANNELS = {"Comparison Website", "Own Website"}

_sc_ch = silver_customer[["customer_id", "sales_channel", "segment"]].copy()
_sc_ch["is_digital_channel"] = (
    _sc_ch["sales_channel"].isin(DIGITAL_CHANNELS)
).astype("Int64")

# ── A8: is_dual_fuel (computed directly from monthly gas consumption) ─────────
# 1 if customer has any positive gas consumption across 2024
# Note: computed here independently so TierMP_Core is self-contained
_dual = (
    dfm.groupby("customer_id")["monthly_gas_m3"]
    .sum()
    .reset_index()
    .rename(columns={"monthly_gas_m3": "_total_gas"})
)
_dual["is_dual_fuel"] = (_dual["_total_gas"] > 0).astype("Int64")

# ── A9: portfolio_type ────────────────────────────────────
# Composite: segment x dual-fuel -> 6 clean portfolio buckets
# e.g. "SME_DualFuel", "Residential_SingleFuel"
_fuel_label = (
    _dual[["customer_id", "is_dual_fuel"]]
    .merge(_sc_ch[["customer_id", "segment"]], on="customer_id", how="left")
)
_fuel_label["portfolio_type"] = (
    _fuel_label["segment"].astype(str)
    + "_"
    + _fuel_label["is_dual_fuel"].map({1: "DualFuel", 0: "SingleFuel"}).fillna("Unknown")
)

# ── A10: gas_share_of_revenue ──────────────────────────────
if not _miss_rev:
    _scm_rev = dfm[_req_rev].copy()
    for _c in ["total_revenue", "gas_revenue_variable", "gas_revenue_fixed"]:
        _scm_rev[_c] = pd.to_numeric(_scm_rev[_c], errors="coerce")

    _scm_rev["_gas_rev_monthly"] = (
        _scm_rev["gas_revenue_variable"].fillna(0)
        + _scm_rev["gas_revenue_fixed"].fillna(0) / 12
    )

    _gas_share = (
        _scm_rev.groupby("customer_id")
        .apply(
            lambda x: (
                x["_gas_rev_monthly"].sum() / x["total_revenue"].sum()
                if x["total_revenue"].sum() > 0
                else np.nan
            )
        )
        .reset_index()
        .rename(columns={0: "gas_share_of_revenue"})
    )
else:
    print(f"WARNING: gas_share_of_revenue skipped — missing cols: {_miss_rev}")
    _gas_share = pd.DataFrame({"customer_id": [], "gas_share_of_revenue": []})

# ── A11–A12: Province average cost ─────────────────────────
_costs = pd.read_csv(DATA_DIR / "costs_by_province_month.csv")
_costs["month"] = pd.to_datetime(_costs["month"].astype(str).str.strip() + "-01", errors="coerce")
_costs = _costs[_costs["month"] <= AS_OF_DATE].copy()
_costs["province"] = pd.to_numeric(_costs["province"], errors="coerce")
for _c in ["elec_var_cost_eur_kwh", "gas_var_cost_eur_m3"]:
    _costs[_c] = pd.to_numeric(_costs[_c], errors="coerce")

_prov_avg = (
    _costs.groupby("province", as_index=False)
    .agg(
        province_avg_elec_cost_2024=("elec_var_cost_eur_kwh", "mean"),
        province_avg_gas_cost_2024=("gas_var_cost_eur_m3", "mean"),
    )
)

_sc_prov = silver_customer[["customer_id", "province_code"]].copy()
_sc_prov["province_code"] = pd.to_numeric(_sc_prov["province_code"], errors="coerce")
_prov_feat = _sc_prov.merge(
    _prov_avg, left_on="province_code", right_on="province", how="left"
)[["customer_id", "province_avg_elec_cost_2024", "province_avg_gas_cost_2024"]]

# ── A13: price_update_count ───────────────────────────────
_ph = pd.read_csv(DATA_DIR / "price_history.csv", parse_dates=["pricing_date"])
_ph = _ph[_ph["pricing_date"] <= AS_OF_DATE].copy()
_price_count = (
    _ph.groupby("customer_id")["pricing_date"]
    .nunique()
    .reset_index()
    .rename(columns={"pricing_date": "price_update_count"})
)

# ── Assemble TierMP_Core ──────────────────────────────────
# customer_id universe: all unique customers from silver_customer_month
_universe = dfm[["customer_id"]].drop_duplicates("customer_id")

TierMP_Core = (
    _universe
    .merge(_cons,                                               on="customer_id", how="left")
    .merge(_marg,                                               on="customer_id", how="left")
    .merge(_sc_ch[["customer_id", "is_digital_channel"]],      on="customer_id", how="left")
    .merge(_dual[["customer_id", "is_dual_fuel"]],             on="customer_id", how="left")
    .merge(_fuel_label[["customer_id", "portfolio_type"]],     on="customer_id", how="left")
    .merge(_gas_share,                                          on="customer_id", how="left")
    .merge(_prov_feat,                                          on="customer_id", how="left")
    .merge(_price_count,                                        on="customer_id", how="left")
)
TierMP_Core["price_update_count"] = TierMP_Core["price_update_count"].fillna(0).astype(int)

# ── QA ────────────────────────────────────────────────────
print("=" * 56)
print("TierMP_Core — QA")
print("=" * 56)
print(f"Shape              : {TierMP_Core.shape}")
print(f"Unique customer_id : {TierMP_Core['customer_id'].nunique()}")
print(f"Duplicate rows     : {TierMP_Core['customer_id'].duplicated().sum()}")

print()
print("Missing rates:")
print(
    TierMP_Core.drop(columns=["customer_id"])
    .isna()
    .mean()
    .sort_values(ascending=False)
    .round(4)
    .to_string()
)

print()
print("Distribution (numeric):")
print(TierMP_Core.describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99]).T.to_string())

print()
print("portfolio_type distribution:")
print(TierMP_Core["portfolio_type"].value_counts(dropna=False).to_string())

TierMP_Core.head()

TierMP_Core — QA
Shape              : (20099, 14)
Unique customer_id : 20099
Duplicate rows     : 0

Missing rates:
avg_monthly_elec_kwh          0.0000
total_elec_kwh_2024           0.0000
avg_monthly_gas_m3            0.0000
total_gas_m3_2024             0.0000
avg_monthly_margin            0.0000
total_margin_2024             0.0000
is_digital_channel            0.0000
is_dual_fuel                  0.0000
portfolio_type                0.0000
gas_share_of_revenue          0.0000
province_avg_elec_cost_2024   0.0000
province_avg_gas_cost_2024    0.0000
price_update_count            0.0000

Distribution (numeric):
                                 count       mean        std         min         1%      25%       50%       75%         99%          max
avg_monthly_elec_kwh        20099.0000  1141.6405  3954.1966      0.0000     0.0121  34.1778   89.7952  293.8959  23042.1631   92558.9873
total_elec_kwh_2024         20099.0000 13699.6860 47450.3587      0.0000     0.1455 410.1342 1077.5422

/var/folders/2y/vz7b97gd01g8625_kn8blf5c0000gn/T/ipykernel_34402/687227828.py:100: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


,customer_id,avg_monthly_elec_kwh,total_elec_kwh_2024,avg_monthly_gas_m3,total_gas_m3_2024,avg_monthly_margin,total_margin_2024,is_digital_channel,is_dual_fuel,portfolio_type,gas_share_of_revenue,province_avg_elec_cost_2024,province_avg_gas_cost_2024,price_update_count
0,C00001,129.0270,1548.3240,0.0000,0.0000,6.1973,74.3673,0,0,Residential_SingleFuel,0.0172,0.0810,0.7358,12
1,C00002,1764.7486,21176.9829,0.0000,0.0000,-45.2739,-543.2871,1,0,Residential_SingleFuel,0.0027,0.0771,0.7997,12
2,C00003,23.6824,284.1887,0.0000,0.0000,3.4179,41.0148,0,0,Residential_SingleFuel,0.0153,0.0771,0.7677,12
3,C00004,106.5836,1279.0028,51.0217,612.2599,3.3818,40.5811,0,1,Residential_DualFuel,0.6570,0.0778,0.7825,12
4,C00005,43.0865,517.0385,0.0000,0.0000,7.2337,86.8050,0,0,Residential_SingleFuel,0.0261,0.0798,0.7765,12


#### TierMP_Risk — Market & Portfolio: Stability & Risk

**What the code does:** Computes usage volatility, margin risk, and price evolution features. All features are derived from the same filtered monthly window (≤ 2024-12-31) used in TierMP_Core. Tier B features have higher structural missingness (especially gas-related, ~30% missing for elec-only customers).

**Tier B — Usage & Margin Stability:**

| # | Feature | Description | Source |
|---|---------|-------------|--------|
| 1 | `std_monthly_elec_kwh` | Month-to-month electricity usage variance | silver_customer_month |
| 2 | `std_monthly_gas_m3` | Month-to-month gas usage variance (NaN for elec-only) | silver_customer_month |
| 3 | `active_months_count` | Count of months with billing data in 2024 | silver_customer_month |
| 4 | `std_margin` | Month-to-month margin volatility | silver_customer_month |
| 5 | `min_monthly_margin` | Worst-case single month margin | silver_customer_month |
| 6 | `max_negative_margin` | Count of loss-making months in 2024 | silver_customer_month |

**Tier B — Price Evolution:**

| # | Feature | Description | Source |
|---|---------|-------------|--------|
| 7 | `elec_price_trend_12m` | Proportional electricity price change (Jan→Dec 2024) | price_history |
| 8 | `gas_price_trend_12m` | Proportional gas price change (NaN for elec-only) | price_history |
| 9 | `elec_price_volatility_12m` | Std of monthly tier-1 electricity price | price_history |

**Tier C — External Market Signals (province-level):**

| # | Feature | Description | Source |
|---|---------|-------------|--------|
| 10 | `province_elec_cost_trend` | Province electricity cost drift Jan→Dec 2024 | costs_by_province_month |
| 11 | `elec_price_vs_province_cost_spread` | Customer price minus province avg cost (negative = margin compression risk) | price_history + costs |

**Model implication:** Margin stability features (`std_margin`, `min_monthly_margin`, `max_negative_margin`) measure pricing volatility and exposure to loss months — distinct from margin level, which shows no churn discriminative power. These are expected to add incremental lift in later experiments. Gas-related features will have high missingness for elec-only customers; imputation strategy (0-fill vs NaN) to be handled in the modeling notebook.

In [175]:
# ============================================================
# TIER MP-RISK — Market & Portfolio: Stability & Risk (Tier B/C)
# ============================================================
# Scope   : usage volatility, margin risk, price evolution,
#           province cost trend, margin compression spread
# Cutoff  : AS_OF_DATE = 2024-12-31  (no future data used)
# Grain   : 1 row per customer_id
# Tier B  : useful but noisier / higher structural missingness
# Tier C  : external market signals (province-level)
# ============================================================

if "TierMP_Core" not in dir():
    raise ValueError("Run TierMP_Core block first — shared inputs required.")

# ── Ensure month is datetime, then filter to cutoff ────────
scm = silver_customer_month.copy()

_m = scm["month"].astype(str).str.strip()
# handle "YYYY-MM" by converting to first day of month
_mask_ym = _m.str.match(r"^\d{4}-\d{2}$")
_m.loc[_mask_ym] = _m.loc[_mask_ym] + "-01"
scm["month"] = pd.to_datetime(_m, errors="coerce")

_bad = scm["month"].isna()
if _bad.any():
    print("WARNING: Unparseable month values (showing up to 10):")
    print(silver_customer_month.loc[_bad, "month"].head(10).to_list())

dfm = scm[scm["month"] <= AS_OF_DATE].copy()

for _c in ["monthly_elec_kwh", "monthly_gas_m3", "total_margin"]:
    dfm[_c] = pd.to_numeric(dfm[_c], errors="coerce").fillna(0)

# ── B1–B3: Usage stability ────────────────────────────────
_usage_stab = (
    dfm.groupby("customer_id", as_index=False)
    .agg(
        std_monthly_elec_kwh=("monthly_elec_kwh", "std"),
        std_monthly_gas_m3=("monthly_gas_m3", "std"),
        active_months_count=("month", "nunique"),
    )
)

# ── B4–B6: Margin stability ───────────────────────────────
_marg_stab = (
    dfm.groupby("customer_id", as_index=False)
    .agg(
        std_margin=("total_margin", "std"),
        min_monthly_margin=("total_margin", "min"),
    )
)

_neg_months = (
    dfm[dfm["total_margin"] < 0]
    .groupby("customer_id")["total_margin"]
    .count()
    .reset_index()
    .rename(columns={"total_margin": "max_negative_margin"})
)

# ── B7–B9: Price evolution  (raw price_history.csv) ───────
_ph = pd.read_csv(DATA_DIR / "price_history.csv", parse_dates=["pricing_date"])
_ph = _ph[_ph["pricing_date"] <= AS_OF_DATE].copy()

for _c in ["variable_price_tier1_eur_kwh", "gas_variable_price_eur_m3"]:
    _ph[_c] = pd.to_numeric(_ph[_c], errors="coerce")

_ph = _ph.sort_values(["customer_id", "pricing_date"])

_price_ends = (
    _ph.groupby("customer_id", as_index=False)
    .agg(
        elec_price_first=("variable_price_tier1_eur_kwh", "first"),
        elec_price_last=("variable_price_tier1_eur_kwh", "last"),
        gas_price_first=("gas_variable_price_eur_m3", "first"),
        gas_price_last=("gas_variable_price_eur_m3", "last"),
    )
)

_price_ends["elec_price_trend_12m"] = (
    (_price_ends["elec_price_last"] - _price_ends["elec_price_first"])
    / _price_ends["elec_price_first"].replace(0, np.nan)
)
_price_ends["gas_price_trend_12m"] = (
    (_price_ends["gas_price_last"] - _price_ends["gas_price_first"])
    / _price_ends["gas_price_first"].replace(0, np.nan)
)

_price_vol = (
    _ph.groupby("customer_id")["variable_price_tier1_eur_kwh"]
    .std()
    .reset_index()
    .rename(columns={"variable_price_tier1_eur_kwh": "elec_price_volatility_12m"})
)

# ── C1: Province electricity cost trend ───────────────────
_costs_c = pd.read_csv(DATA_DIR / "costs_by_province_month.csv")

_cm = _costs_c["month"].astype(str).str.strip()
_mask_ym = _cm.str.match(r"^\d{4}-\d{2}$")
_cm.loc[_mask_ym] = _cm.loc[_mask_ym] + "-01"
_costs_c["month"] = pd.to_datetime(_cm, errors="coerce")

_costs_c = _costs_c[_costs_c["month"] <= AS_OF_DATE].copy()
_costs_c["province"] = pd.to_numeric(_costs_c["province"], errors="coerce")
_costs_c["elec_var_cost_eur_kwh"] = pd.to_numeric(_costs_c["elec_var_cost_eur_kwh"], errors="coerce")

_prov_trend = (
    _costs_c.sort_values("month")
    .groupby("province", as_index=False)
    .agg(
        _e_first=("elec_var_cost_eur_kwh", "first"),
        _e_last=("elec_var_cost_eur_kwh", "last"),
    )
)
_prov_trend["province_elec_cost_trend"] = (
    (_prov_trend["_e_last"] - _prov_trend["_e_first"])
    / _prov_trend["_e_first"].replace(0, np.nan)
)

_sc_prov_c = silver_customer[["customer_id", "province_code"]].copy()
_sc_prov_c["province_code"] = pd.to_numeric(_sc_prov_c["province_code"], errors="coerce")
_prov_trend_feat = _sc_prov_c.merge(
    _prov_trend[["province", "province_elec_cost_trend"]],
    left_on="province_code",
    right_on="province",
    how="left",
)[["customer_id", "province_elec_cost_trend"]]

# ── C2: Margin compression spread ─────────────────────────
_spread_in = (
    _price_ends[["customer_id", "elec_price_last"]]
    .merge(silver_customer[["customer_id", "province_code"]], on="customer_id", how="left")
)
_spread_in["province_code"] = pd.to_numeric(_spread_in["province_code"], errors="coerce")
_spread_in = _spread_in.merge(
    TierMP_Core[["customer_id", "province_avg_elec_cost_2024"]],
    on="customer_id",
    how="left",
)
_spread_in["elec_price_vs_province_cost_spread"] = (
    _spread_in["elec_price_last"] - _spread_in["province_avg_elec_cost_2024"]
)

# ── C3: is_price_increase — customer's electricity price went up over 2024 ──
_price_ends["is_price_increase"] = (
    _price_ends["elec_price_trend_12m"] > 0
).astype("Int64")

# ── Assemble TierMP_Risk ──────────────────────────────────
TierMP_Risk = (
    Tier1A_Features[["customer_id"]]
    .drop_duplicates("customer_id")
    .merge(_usage_stab, on="customer_id", how="left")
    .merge(_marg_stab, on="customer_id", how="left")
    .merge(_neg_months, on="customer_id", how="left")
    .merge(
        _price_ends[["customer_id", "elec_price_trend_12m", "gas_price_trend_12m"]],
        on="customer_id",
        how="left",
    )
    .merge(_price_vol, on="customer_id", how="left")
    .merge(_prov_trend_feat, on="customer_id", how="left")
    .merge(
        _spread_in[["customer_id", "elec_price_vs_province_cost_spread"]],
        on="customer_id",
        how="left",
    )
    .merge(
        _price_ends[["customer_id", "is_price_increase"]],
        on="customer_id",
        how="left",
    )
)

TierMP_Risk["max_negative_margin"] = TierMP_Risk["max_negative_margin"].fillna(0).astype(int)

# ── QA ────────────────────────────────────────────────────
print("=" * 56)
print("TierMP_Risk — QA")
print("=" * 56)
print(f"Shape              : {TierMP_Risk.shape}")
print(f"Unique customer_id : {TierMP_Risk['customer_id'].nunique()}")
print(f"Duplicate rows     : {TierMP_Risk['customer_id'].duplicated().sum()}")

print()
print("Missing rates:")
print(
    TierMP_Risk.drop(columns=["customer_id"])
    .isna()
    .mean()
    .sort_values(ascending=False)
    .round(4)
    .to_string()
)

print()
print("Distribution (numeric):")
print(TierMP_Risk.describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99]).T.to_string())

TierMP_Risk.head()

# ── Rolling margin trend (EDA s1: use rolling trends, not absolute level) ─────
_scm_t = silver_customer_month.copy()
_scm_t["month_dt"] = pd.to_datetime(_scm_t["month"])
_scm_t = _scm_t[_scm_t["month_dt"] <= AS_OF_DATE]

_last3  = _scm_t[_scm_t["month_dt"] >= (AS_OF_DATE - pd.DateOffset(months=3))]
_prior3 = _scm_t[
    (_scm_t["month_dt"] >= (AS_OF_DATE - pd.DateOffset(months=6))) &
    (_scm_t["month_dt"] <  (AS_OF_DATE - pd.DateOffset(months=3)))
]

_avg_last  = _last3.groupby("customer_id")["total_margin"].mean().rename("_m_last3")
_avg_prior = _prior3.groupby("customer_id")["total_margin"].mean().rename("_m_prior3")
_rolling   = pd.concat([_avg_last, _avg_prior], axis=1).reset_index()
_rolling["rolling_margin_trend"] = _rolling["_m_last3"] - _rolling["_m_prior3"]

TierMP_Risk = TierMP_Risk.merge(
    _rolling[["customer_id","rolling_margin_trend"]], on="customer_id", how="left"
)
print(f"rolling_margin_trend: {TierMP_Risk['rolling_margin_trend'].notna().sum()} non-null / {len(TierMP_Risk)}")

TierMP_Risk — QA
Shape              : (20099, 13)
Unique customer_id : 20099
Duplicate rows     : 0

Missing rates:
gas_price_trend_12m                  0.8193
elec_price_trend_12m                 0.0018
elec_price_volatility_12m            0.0001
elec_price_vs_province_cost_spread   0.0001
std_monthly_elec_kwh                 0.0000
std_monthly_gas_m3                   0.0000
active_months_count                  0.0000
std_margin                           0.0000
min_monthly_margin                   0.0000
max_negative_margin                  0.0000
province_elec_cost_trend             0.0000
is_price_increase                    0.0000

Distribution (numeric):
                                        count     mean       std        min        1%     25%     50%     75%       99%        max
std_monthly_elec_kwh               20099.0000 348.8472 1531.4695     0.0000    0.0036  9.2517 25.0061 84.3638 6435.1837 55371.0357
std_monthly_gas_m3                 20099.0000  69.5258  388.2730     

##### 8. Final Checks before building the Gold table

In [176]:
# Tier aliases for grain checks and gold build
# Tier1A_Features is now properly defined above
# Behavioral_Base contains all behavioral/sentiment/interaction features
Tier2A_Behavioral = Behavioral_Base   # behavioral intent + complaint flags
Tier2B_Sentiment  = Behavioral_Base   # sentiment features (same base object)

In [177]:
# Grain check
def grain_check(df, name):
    print(f"\n{name}")
    print("Shape:", df.shape)
    print("Unique customers:", df["customer_id"].nunique())
    print("Duplicate customer_id rows:", df["customer_id"].duplicated().sum())

grain_check(Tier1A_Features,  "Tier1A_Features")
grain_check(TierMP_Core,      "TierMP_Core")
grain_check(TierMP_Risk,      "TierMP_Risk")
grain_check(Behavioral_Base,  "Behavioral_Base")


Tier1A_Features
Shape: (20099, 16)
Unique customers: 20099
Duplicate customer_id rows: 0

TierMP_Core
Shape: (20099, 14)
Unique customers: 20099
Duplicate customer_id rows: 0

TierMP_Risk
Shape: (20099, 14)
Unique customers: 20099
Duplicate customer_id rows: 0

Behavioral_Base
Shape: (20099, 35)
Unique customers: 20099
Duplicate customer_id rows: 0


In [178]:
# Snapshot leakage check (Confirm no feature uses post-2024-12-31 data.)
# monthly
silver_customer_month["month"].max()

'2024-12'

In [179]:
# Snapshot leakage check (Confirm no feature uses post-2024-12-31 data.)
# interactions
pd.to_datetime(silver_customer["date"], errors="coerce").max()

Timestamp('2024-11-30 00:00:00')

In [180]:
# Label integrity check
silver_customer["churn"].value_counts(normalize=True)


churn
0   0.9004
1   0.0996
Name: proportion, dtype: float64

In [181]:
silver_customer["customer_id"].nunique()


20099

In [182]:
# Missingness scan
def missing_scan(df, name):
    print(f"\n{name} missing > 5%:")
    miss = df.isna().mean().sort_values(ascending=False)
    miss = miss[miss > 0.05]
    print(miss if len(miss) > 0 else "None above 5%")

missing_scan(Tier1A_Features,  "Tier1A_Features")
missing_scan(TierMP_Core,      "TierMP_Core")
missing_scan(TierMP_Risk,      "TierMP_Risk")
missing_scan(Behavioral_Base,  "Behavioral_Base")


Tier1A_Features missing > 5%:
None above 5%

TierMP_Core missing > 5%:
None above 5%

TierMP_Risk missing > 5%:
gas_price_trend_12m   0.8193
dtype: float64

Behavioral_Base missing > 5%:
is_negative_sentiment       0.6806
sentiment_label             0.6806
last_interaction_days_ago   0.6806
months_since_last_change    0.3032
dtype: float64


In [183]:
# Distribution sanity checks
TierMP_Core.describe(percentiles=[0.01,0.99]).T

,count,mean,std,min,1%,50%,99%,max
avg_monthly_elec_kwh,20099.0000,1141.6405,3954.1966,0.0000,0.0121,89.7952,23042.1631,92558.9873
total_elec_kwh_2024,20099.0000,13699.6860,47450.3587,0.0000,0.1455,1077.5422,276505.9569,1110707.8477
avg_monthly_gas_m3,20099.0000,52.6921,293.9941,0.0000,0.0000,0.0000,1548.2379,7131.1328
total_gas_m3_2024,20099.0000,632.3053,3527.9286,0.0000,0.0000,0.0000,18578.8552,85573.5937
avg_monthly_margin,20099.0000,-4.9643,65.1190,-1749.6016,-291.4943,6.9578,46.4217,591.3165
total_margin_2024,20099.0000,-59.5718,781.4277,-20995.2189,-3497.9321,83.4935,557.0601,7095.7982
is_digital_channel,20099.0000,0.1304,0.3367,0.0000,0.0000,0.0000,1.0000,1.0000
is_dual_fuel,20099.0000,0.1807,0.3848,0.0000,0.0000,0.0000,1.0000,1.0000
gas_share_of_revenue,20099.0000,0.0894,0.1918,0.0000,0.0001,0.0200,0.8926,0.9856
province_avg_elec_cost_2024,20099.0000,0.0777,0.0010,0.0761,0.0761,0.0775,0.0810,0.0811


In [184]:
# Logical consistency checks — zero gas consumption should align with elec-only customers
TierMP_Core[
    (TierMP_Core["avg_monthly_gas_m3"] == 0)
].head()

,customer_id,avg_monthly_elec_kwh,total_elec_kwh_2024,avg_monthly_gas_m3,total_gas_m3_2024,avg_monthly_margin,total_margin_2024,is_digital_channel,is_dual_fuel,portfolio_type,gas_share_of_revenue,province_avg_elec_cost_2024,province_avg_gas_cost_2024,price_update_count
0,C00001,129.0270,1548.3240,0.0000,0.0000,6.1973,74.3673,0,0,Residential_SingleFuel,0.0172,0.0810,0.7358,12
1,C00002,1764.7486,21176.9829,0.0000,0.0000,-45.2739,-543.2871,1,0,Residential_SingleFuel,0.0027,0.0771,0.7997,12
2,C00003,23.6824,284.1887,0.0000,0.0000,3.4179,41.0148,0,0,Residential_SingleFuel,0.0153,0.0771,0.7677,12
4,C00005,43.0865,517.0385,0.0000,0.0000,7.2337,86.8050,0,0,Residential_SingleFuel,0.0261,0.0798,0.7765,12
6,C00007,67.9008,814.8090,0.0000,0.0000,8.7459,104.9512,0,0,Residential_SingleFuel,0.0183,0.0771,0.7677,12


In [185]:
# Dual fuel check 2 — fraction of customers with positive gas consumption
(TierMP_Core["avg_monthly_gas_m3"] > 0).mean()

np.float64(0.18070550773670332)

In [186]:
# Correlation quick glance
Gold_preview = (
    Tier1A_Features[["customer_id", "months_to_renewal"]]
    .merge(TierMP_Core, on="customer_id")
    .merge(silver_customer[["customer_id", "churn"]], on="customer_id")
)

Gold_preview.corr(numeric_only=True)["churn"].sort_values(ascending=False)

churn                          1.0000
total_margin_2024              0.0370
avg_monthly_margin             0.0370
province_avg_gas_cost_2024     0.0345
province_avg_elec_cost_2024    0.0016
price_update_count            -0.0268
gas_share_of_revenue          -0.0272
is_dual_fuel                  -0.0288
avg_monthly_gas_m3            -0.0417
total_gas_m3_2024             -0.0417
avg_monthly_elec_kwh          -0.0501
total_elec_kwh_2024           -0.0501
is_digital_channel            -0.0508
months_to_renewal             -0.2355
Name: churn, dtype: float64

In [187]:
# Final customer count
tables = {
    "Tier1A_Features":  Tier1A_Features,
    "TierMP_Core":      TierMP_Core,
    "TierMP_Risk":      TierMP_Risk,
    "Behavioral_Base":  Behavioral_Base,
}
for name, t in tables.items():
    print(f"{name}: {t['customer_id'].nunique():,} unique customers")

Tier1A_Features: 20,099 unique customers
TierMP_Core: 20,099 unique customers
TierMP_Risk: 20,099 unique customers
Behavioral_Base: 20,099 unique customers


In [188]:
# Save silver tables to Parquet
(ARTIFACTS_DIR / 'silver').mkdir(parents=True, exist_ok=True)

silver_customer.to_parquet(
    ARTIFACTS_DIR / 'silver' / 'silver_customer.parquet',
    index=False,
)
silver_customer_month.to_parquet(
    ARTIFACTS_DIR / 'silver' / 'silver_customer_month.parquet',
    index=False,
)
print('Silver tables saved to', ARTIFACTS_DIR / 'silver')


Silver tables saved to /Users/crissafrancisco/Desktop/IE/Third Term/Capstone/data/silver


In [189]:
# ================================
# SILVER CLEANUP: Remove duplicate merge columns
# ================================

# 1️⃣ --- Renewal Bucket Fix ---
# Prefer Tier1A (lifecycle) as canonical source

if "renewal_bucket_x" in silver_customer.columns and "renewal_bucket_y" in silver_customer.columns:
    silver_customer = (
        silver_customer
        .drop(columns=["renewal_bucket_y"])
        .rename(columns={"renewal_bucket_x": "renewal_bucket"})
    )

elif "renewal_bucket_x" in silver_customer.columns:
    silver_customer = silver_customer.rename(columns={"renewal_bucket_x": "renewal_bucket"})

elif "renewal_bucket_y" in silver_customer.columns:
    silver_customer = silver_customer.rename(columns={"renewal_bucket_y": "renewal_bucket"})


# 2️⃣ --- Customer Intent Fix ---
# Prefer Tier2A behavioral version as canonical

if "customer_intent_x" in silver_customer.columns and "customer_intent_y" in silver_customer.columns:
    silver_customer = (
        silver_customer
        .drop(columns=["customer_intent_x"])
        .rename(columns={"customer_intent_y": "customer_intent"})
    )

elif "customer_intent_x" in silver_customer.columns:
    silver_customer = silver_customer.rename(columns={"customer_intent_x": "customer_intent"})

elif "customer_intent_y" in silver_customer.columns:
    silver_customer = silver_customer.rename(columns={"customer_intent_y": "customer_intent"})


# 3️⃣ Final sanity check
duplicate_suffix_cols = [c for c in silver_customer.columns if c.endswith("_x") or c.endswith("_y")]

print("Remaining *_x / *_y columns:", duplicate_suffix_cols)
print("Cleanup complete.")


Remaining *_x / *_y columns: []
Cleanup complete.


#### 3. Gold Layer — Final Master Table

**What the code does:** Merges all four tier DataFrames into a single customer-level table (`gold_master`) — one row per customer, no time dimension. This is the direct input to the modeling notebook.

| Layer | DataFrame | Features | Coverage |
|-------|-----------|----------|----------|
| Structural | `silver_customer` | segment, sales_channel, province flags, contracted power | All customers |
| Lifecycle + stickiness | `Tier1A_Features` | Renewal timing, tenure, dual fuel | All customers |
| Market & portfolio | `TierMP_Core` | Consumption, channel, portfolio type, pricing structure | All customers |
| Market & portfolio risk | `TierMP_Risk` | Usage/margin volatility, price evolution, province cost trend | All customers (gas features: ~70%) |
| Behavioral & sentiment | `Behavioral_Base` | Intent, complaints, sentiment, interaction effects | ~32% of customers with recorded interactions |

**Missingness note:** ~68% of customers have no recorded interaction in 2024, so all `Behavioral_Base` features will be NaN for these customers. This is structurally expected — absence of interaction is itself a signal (lower engagement). Imputation strategy (0-fill, median, or explicit "no interaction" category) is handled in the modeling notebook.

**Retained business columns (excluded from model training):** `avg_monthly_margin`, `total_margin_2024`. These are used post-prediction for retention prioritisation: customers are ranked by `P(churn) × avg_monthly_margin` to allocate intervention budget.

In [190]:
# Read silver parquet inputs
silver_customer = pd.read_parquet(
    ARTIFACTS_DIR / 'silver' / 'silver_customer.parquet'
)
silver_customer_month = pd.read_parquet(
    ARTIFACTS_DIR / 'silver' / 'silver_customer_month.parquet'
)
print('silver_customer shape:      ', silver_customer.shape)
print('silver_customer_month shape:', silver_customer_month.shape)


silver_customer shape:       (20099, 25)
silver_customer_month shape: (241188, 39)


In [191]:
# Check for overlaps across the distinct tier DataFrames
# Note: Tier1A_Features, Tier2A_Behavioral, Tier2B_Sentiment all alias
# Behavioral_Base (same object), so we only check it once.
def check_overlap(*dfs):
    from collections import Counter
    all_cols = []
    for df in dfs:
        all_cols += [c for c in df.columns if c != "customer_id"]
    dup = [c for c, n in Counter(all_cols).items() if n > 1]
    print("Overlapping feature names across tiers:", dup if dup else "None — clean!")

check_overlap(
    Behavioral_Base,   # covers Tier1A / Tier2A / Tier2B (all the same object)
    TierMP_Core,
    TierMP_Risk,
)

Overlapping feature names across tiers: ['is_dual_fuel']


In [192]:
# Build Gold master table
# ─────────────────────────────────────────────────────────────────
# Structural columns from silver_customer
_structural = [
    "customer_id", "churn",
    "segment", "sales_channel",
    "is_high_competition_province", "is_second_residence",
    "contracted_power_kw", "is_urban",
]
_sc_cols = [c for c in _structural if c in silver_customer.columns]

gold_master = (
    silver_customer[_sc_cols]
    .merge(Tier1A_Features,  on="customer_id", how="left")   # lifecycle + stickiness + dual fuel
    .merge(TierMP_Core,      on="customer_id", how="left")   # economic core
    .merge(TierMP_Risk,      on="customer_id", how="left")   # stability + risk
    .merge(Behavioral_Base,  on="customer_id", how="left")   # behavioral + sentiment + interactions
)

# Clean up any _x / _y duplicate columns from merges
_dup_sfx = [c for c in gold_master.columns if c.endswith("_x") or c.endswith("_y")]
if _dup_sfx:
    print("Cleaning up duplicate suffix columns:", _dup_sfx)
    for col in _dup_sfx:
        base = col[:-2]
        if base in gold_master.columns:
            gold_master = gold_master.drop(columns=[col])
        else:
            gold_master = gold_master.rename(columns={col: base})

print("gold_master shape:", gold_master.shape)
print("Unique customers:", gold_master["customer_id"].nunique())
print("Duplicate customer_id rows:", gold_master["customer_id"].duplicated().sum())

Cleaning up duplicate suffix columns: ['segment_x', 'sales_channel_x', 'is_high_competition_province_x', 'is_second_residence_x', 'months_to_renewal_x', 'renewal_bucket_x', 'is_within_3m_of_renewal_x', 'tenure_months_x', 'tenure_bucket_x', 'segment_y', 'sales_channel_y', 'is_high_competition_province_y', 'has_interaction_x', 'is_second_residence_y', 'is_dual_fuel_x', 'is_dual_fuel_y', 'months_to_renewal_y', 'renewal_bucket_y', 'is_within_3m_of_renewal_y', 'tenure_months_y', 'tenure_bucket_y', 'has_interaction_y']
gold_master shape: (20099, 70)
Unique customers: 20099
Duplicate customer_id rows: 0


In [193]:
# Quick check: did any merge accidentally explode rows?
assert gold_master["customer_id"].is_unique, "Gold master has duplicate customer_id rows (merge explosion)."


In [194]:
# 1) Any duplicate columns created by merges?
dup_cols = gold_master.columns[gold_master.columns.duplicated()].tolist()
print("Duplicate column names:", dup_cols)


Duplicate column names: []


In [195]:
# 2) Missingness overview (top 15 most-missing features) -> 68% missing because 68% of customers had no interaction
missing = gold_master.isna().mean().sort_values(ascending=False)
display(missing.head(15))


gas_price_trend_12m                  0.8193
last_interaction_days_ago            0.6806
sentiment_label                      0.6806
is_negative_sentiment                0.6806
months_since_last_change             0.3032
elec_price_trend_12m                 0.0018
elec_price_volatility_12m            0.0001
elec_price_vs_province_cost_spread   0.0001
has_interaction_x_renewal_bucket     0.0000
intent_severity_score                0.0000
province_elec_cost_trend             0.0000
is_price_increase                    0.0000
rolling_margin_trend                 0.0000
customer_intent                      0.0000
recent_complaint_flag                0.0000
dtype: float64

In [196]:
# Save GOLD master
(ARTIFACTS_DIR / 'gold').mkdir(parents=True, exist_ok=True)
gold_master.to_parquet(
    ARTIFACTS_DIR / 'gold' / 'gold_master.parquet', index=False
)
print('Saved:', ARTIFACTS_DIR / 'gold' / 'gold_master.parquet')


Saved: /Users/crissafrancisco/Desktop/IE/Third Term/Capstone/data/gold/gold_master.parquet


In [197]:
gold_master.columns

Index(['customer_id', 'churn', 'segment', 'sales_channel', 'is_second_residence', 'contracted_power_kw', 'is_urban', 'months_to_renewal', 'renewal_bucket',
       'is_within_3m_of_renewal', 'tenure_months', 'tenure_bucket', 'has_interaction', 'is_expired_contract', 'is_comparison_channel',
       'is_own_website_channel', 'sales_channel_x_renewal_bucket', 'avg_monthly_elec_kwh', 'total_elec_kwh_2024', 'avg_monthly_gas_m3', 'total_gas_m3_2024',
       'avg_monthly_margin', 'total_margin_2024', 'is_digital_channel', 'portfolio_type', 'gas_share_of_revenue', 'province_avg_elec_cost_2024',
       'province_avg_gas_cost_2024', 'price_update_count', 'std_monthly_elec_kwh', 'std_monthly_gas_m3', 'active_months_count', 'std_margin',
       'min_monthly_margin', 'max_negative_margin', 'elec_price_trend_12m', 'gas_price_trend_12m', 'elec_price_volatility_12m', 'province_elec_cost_trend',
       'elec_price_vs_province_cost_spread', 'is_price_increase', 'rolling_margin_trend', 'customer_intent', 